In [1]:
from pprint import pprint # used for pretty printing
import requests # used to get data from the a URL
import pandas as pd # used to analyse the results
search_url = "https://www.ebi.ac.uk/pdbe/search/pdb/select?" # the rest of the URL used for PDBe's search API.

In [2]:
#Leer la base de datos (Las secuencias con aminoacidos inusuales (grampa.csv) fueron eliminados en bash)
datos_raw=pd.read_csv('grampa_AA.csv', index_col=0)
#Eliminar secuencias con modificaciones inusuales (README del repositorio de witten para más info)
datos_raw=datos_raw[datos_raw["has_unusual_modification"]==False]
#Eliminar columnas que no aportan información relevante 
datos_raw=datos_raw.drop(['url_source','datasource_has_modifications','database','has_unusual_modification'], axis='columns')

In [3]:
sec_raw=datos_raw["sequence"]
#Inicia una lista vacía donde se guardarán las secuencias únicas
sec_0=[]
[sec_0.append(x) for x in sec_raw if x not in sec_0]
len(sec_0)

6169

In [4]:
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor

In [5]:
#Creamos una lista con las secuencias del DF para poder trabajarla en modlAMP 
sec_raw=pd.Series.tolist(datos_raw["sequence"])
#Calcular la longitud de las secuencias
globaldesc=GlobalDescriptor(sec_raw)
globaldesc.length()
lon=globaldesc.descriptor
datos_raw["sequence_length"]=lon

In [6]:
datos_prueba=datos_raw[(datos_raw["sequence_length"]>=10)].copy()

In [7]:
sec_prueba=datos_prueba["sequence"]
#Inicia una lista vacía donde se guardarán las secuencias únicas
sec_1=[]
[sec_1.append(x) for x in sec_prueba if x not in sec_1]
len(sec_1)

5638

We will now define some functions which will use to get the data from PDBe's search API

In [2]:
def make_request_post(search_dict, number_of_rows=10):
    """
    makes a post request to the PDBe API
    :param dict search_dict: the terms used to search
    :param number_of_rows: number or rows to return - initially limited to 10
    :return dict: response JSON
    """
    # make sure we get the number of rows we need
    if 'rows' not in search_dict:
        search_dict['rows'] = number_of_rows
    # set the return type to JSON
    search_dict['wt'] = 'json'

    # do the query
    response = requests.post(search_url, data=search_dict)

    if response.status_code == 200:
        return response.json()
    else:
        print("[No data retrieved - %s] %s" % (response.status_code, response.text))

    return {}

def format_sequence_search_terms(sequence, filter_terms=None):
    """
    Format parameters for a sequence search
    :param str sequence: one letter sequence
    :param lst filter_terms: Terms to filter the results by
    :return str: search string
    """
    # first we set the parameters which we will pass to PDBe's search
    params = {
        'json.nl': 'map',
        'start': '0',
        'sort': 'fasta(e_value) asc',
        'xjoin_fasta': 'true',
        'bf': 'fasta(percentIdentity)',
        'xjoin_fasta.external.expupperlim': '0.1',
        'xjoin_fasta.external.sequence': sequence,
        'q': '*:*',
        'fq': '{!xjoin}xjoin_fasta'
    }
    # we make sure that we add required filter terms if they aren't present
    if filter_terms:
        for term in ['pdb_id', 'entity_id', 'entry_entity', 'chain_id']:
            filter_terms.append(term)
        filter_terms = list(set(filter_terms))
        params['fl'] = ','.join(filter_terms)

    # returns the parameter dictionary
    return params


def run_sequence_search(sequence, filter_terms=None, number_of_rows=10):
    """
    Runs a sequence search and results the results
    :param str sequence: sequence in one letter code
    :param lst filter_terms: terms to filter the results by
    :param int number_of_rows: number of results to return
    :return lst: List of results
    """
    search_dict = format_sequence_search_terms(sequence=sequence, filter_terms=filter_terms)
    response = make_request_post(search_dict=search_dict, number_of_rows=number_of_rows)
    results = response.get('response', {}).get('docs', [])
    print('Number of results {}'.format(len(results)))

    # we now have to go through the FASTA results and join them with the main results

    raw_fasta_results = response.get('xjoin_fasta').get('external')
    fasta_results = {} # results from FASTA will be stored here - key'd by PDB ID and Chain ID

    # go through each FASTA result and get the E value, percentage identity and sequence from the result

    for fasta_row in raw_fasta_results:
        # join_id = fasta_row.get('joinId')
        fasta_doc = fasta_row.get('doc', {})
        percent_identity = fasta_doc.get('percent_identity')
        e_value = fasta_doc.get('e_value')
        return_sequence = fasta_row.get('return_sequence_string')
        pdb_id_chain = fasta_doc.get('pdb_id_chain').split('_')
        pdb_id = pdb_id_chain[0].lower()
        chain_id = pdb_id_chain[-1]
        join_id = '{}_{}'.format(pdb_id, chain_id)
        fasta_results[join_id] = {'e_value': e_value,
                                  'percentage_identity': percent_identity,
                                  'return_sequence': return_sequence}

    # now we go through the main results and add the FASTA results
    ret = [] # final results will be stored here.
    for row in results:
        pdb_id = row.get('pdb_id').lower()
        chain_ids = row.get('chain_id')
        for chain_id in chain_ids:
            search_id = '{}_{}'.format(pdb_id, chain_id)
            entry_fasta_results = fasta_results.get(search_id, {})
            # we will only keep results that match the search ID
            if entry_fasta_results:
                row['e_value'] = entry_fasta_results.get('e_value')
                row['percentage_identity'] = entry_fasta_results.get('percentage_identity')
                row['result_sequence'] = entry_fasta_results.get('return_sequence_string')

                ret.append(row)
    return ret

In [3]:
def multiple_sequence_search(sequences):
    """
    Performs sequence search for multiple entries
    :param lst sequences: a list coontaining all the query sequences
    """
    results=dict.fromkeys(sequences)
    i=0
    while i < len(sequences):
        for values in results:
            results[values]=run_sequence_search(sequences[i], filter_terms=['pdb_id', 'molecule_name'])
            i=i+1
            print(str(i)+" sequences of "+str(len(sequences)))
        
    return results

In [4]:
def results_to_dict(results):
    """
    Delete sequences with no matches inside results dictionary
    :param dict results: Dictionary containing all the results from multiple sequence search
    """
    new_dict = {k: v for k, v in results.items() if len(v) != 0 }
    return new_dict

## First 500

In [9]:
secuencias=sec_1[0:500]

In [29]:
sec_5h_raw=multiple_sequence_search(secuencias)

1 sequences from 500
2 sequences from 500
3 sequences from 500
4 sequences from 500
5 sequences from 500
6 sequences from 500
7 sequences from 500
8 sequences from 500
9 sequences from 500
10 sequences from 500
11 sequences from 500
12 sequences from 500
13 sequences from 500
14 sequences from 500
15 sequences from 500
16 sequences from 500
17 sequences from 500
18 sequences from 500
19 sequences from 500
20 sequences from 500
21 sequences from 500
22 sequences from 500
23 sequences from 500
24 sequences from 500
25 sequences from 500
26 sequences from 500
27 sequences from 500
28 sequences from 500
29 sequences from 500
30 sequences from 500
31 sequences from 500
32 sequences from 500
33 sequences from 500
34 sequences from 500
35 sequences from 500
36 sequences from 500
37 sequences from 500
38 sequences from 500
39 sequences from 500
40 sequences from 500
41 sequences from 500
42 sequences from 500
43 sequences from 500
44 sequences from 500
45 sequences from 500
46 sequences from 5

362 sequences from 500
363 sequences from 500
364 sequences from 500
365 sequences from 500
366 sequences from 500
367 sequences from 500
368 sequences from 500
369 sequences from 500
370 sequences from 500
371 sequences from 500
372 sequences from 500
373 sequences from 500
374 sequences from 500
375 sequences from 500
376 sequences from 500
377 sequences from 500
378 sequences from 500
379 sequences from 500
380 sequences from 500
381 sequences from 500
382 sequences from 500
383 sequences from 500
384 sequences from 500
385 sequences from 500
386 sequences from 500
387 sequences from 500
388 sequences from 500
389 sequences from 500
390 sequences from 500
391 sequences from 500
392 sequences from 500
393 sequences from 500
394 sequences from 500
395 sequences from 500
396 sequences from 500
397 sequences from 500
398 sequences from 500
399 sequences from 500
400 sequences from 500
401 sequences from 500
402 sequences from 500
403 sequences from 500
404 sequences from 500
405 sequenc

In [48]:
res_5h_raw=sec_5h_raw

In [49]:
%store res_5h_raw

Stored 'res_5h_raw' (dict)


In [50]:
res_5h=results_to_dict(sec_5h_raw)

In [51]:
%store res_5h

Stored 'res_5h' (dict)


In [52]:
len(res_5h)

209

## 500 - 1000

In [37]:
sec_1k=sec_1[500:1000]

In [39]:
sec_1k_raw=multiple_sequence_search(sec_1k)

1 sequences from 500
2 sequences from 500
3 sequences from 500
4 sequences from 500
5 sequences from 500
6 sequences from 500
7 sequences from 500
8 sequences from 500
9 sequences from 500
10 sequences from 500
11 sequences from 500
12 sequences from 500
13 sequences from 500
14 sequences from 500
15 sequences from 500
16 sequences from 500
17 sequences from 500
18 sequences from 500
19 sequences from 500
20 sequences from 500
21 sequences from 500
22 sequences from 500
23 sequences from 500
24 sequences from 500
25 sequences from 500
26 sequences from 500
27 sequences from 500
28 sequences from 500
29 sequences from 500
30 sequences from 500
31 sequences from 500
32 sequences from 500
33 sequences from 500
34 sequences from 500
35 sequences from 500
36 sequences from 500
37 sequences from 500
38 sequences from 500
39 sequences from 500
40 sequences from 500
41 sequences from 500
42 sequences from 500
43 sequences from 500
44 sequences from 500
45 sequences from 500
46 sequences from 5

362 sequences from 500
363 sequences from 500
364 sequences from 500
365 sequences from 500
366 sequences from 500
367 sequences from 500
368 sequences from 500
369 sequences from 500
370 sequences from 500
371 sequences from 500
372 sequences from 500
373 sequences from 500
374 sequences from 500
375 sequences from 500
376 sequences from 500
377 sequences from 500
378 sequences from 500
379 sequences from 500
380 sequences from 500
381 sequences from 500
382 sequences from 500
383 sequences from 500
384 sequences from 500
385 sequences from 500
386 sequences from 500
387 sequences from 500
388 sequences from 500
389 sequences from 500
390 sequences from 500
391 sequences from 500
392 sequences from 500
393 sequences from 500
394 sequences from 500
395 sequences from 500
396 sequences from 500
397 sequences from 500
398 sequences from 500
399 sequences from 500
400 sequences from 500
401 sequences from 500
402 sequences from 500
403 sequences from 500
404 sequences from 500
405 sequenc

In [40]:
%store sec_1k_raw

Stored 'sec_1k_raw' (dict)


In [45]:
res_1k=results_to_dict(sec_1k_raw)

In [47]:
%store res_1k

Stored 'res_1k' (dict)


In [3]:
%store -r res_1k

In [72]:
len(res_1k)

205

## 1000 - 1500

In [53]:
sec_15h=sec_1[1001:1500]

In [54]:
sec_15h_raw=multiple_sequence_search(sec_15h)

1 sequences from 500
2 sequences from 500
3 sequences from 500
4 sequences from 500
5 sequences from 500
6 sequences from 500
7 sequences from 500
8 sequences from 500
9 sequences from 500
10 sequences from 500
11 sequences from 500
12 sequences from 500
13 sequences from 500
14 sequences from 500
15 sequences from 500
16 sequences from 500
17 sequences from 500
18 sequences from 500
19 sequences from 500
20 sequences from 500
21 sequences from 500
22 sequences from 500
23 sequences from 500
24 sequences from 500
25 sequences from 500
26 sequences from 500
27 sequences from 500
28 sequences from 500
29 sequences from 500
30 sequences from 500
31 sequences from 500
32 sequences from 500
33 sequences from 500
34 sequences from 500
35 sequences from 500
36 sequences from 500
37 sequences from 500
38 sequences from 500
39 sequences from 500
40 sequences from 500
41 sequences from 500
42 sequences from 500
43 sequences from 500
44 sequences from 500
45 sequences from 500
46 sequences from 5

362 sequences from 500
363 sequences from 500
364 sequences from 500
365 sequences from 500
366 sequences from 500
367 sequences from 500
368 sequences from 500
369 sequences from 500
370 sequences from 500
371 sequences from 500
372 sequences from 500
373 sequences from 500
374 sequences from 500
375 sequences from 500
376 sequences from 500
377 sequences from 500
378 sequences from 500
379 sequences from 500
380 sequences from 500
381 sequences from 500
382 sequences from 500
383 sequences from 500
384 sequences from 500
385 sequences from 500
386 sequences from 500
387 sequences from 500
388 sequences from 500
389 sequences from 500
390 sequences from 500
391 sequences from 500
392 sequences from 500
393 sequences from 500
394 sequences from 500
395 sequences from 500
396 sequences from 500
397 sequences from 500
398 sequences from 500
399 sequences from 500
400 sequences from 500
401 sequences from 500
402 sequences from 500
403 sequences from 500
404 sequences from 500
405 sequenc

In [55]:
# %store sec_15h_raw

Stored 'sec_15h_raw' (dict)


In [56]:
res_15h=results_to_dict(sec_15h_raw)

In [58]:
len(res_15h)

186

In [84]:
#Running sequence search for sec_1[1000] due to indexing mistake
sec_error=sec_1[1000]

In [85]:
error_list=[]
error_list.append(sec_error)

In [87]:
error_dict=multiple_sequence_search(error_list)

1 sequences from 500


In [90]:
error_dict.update(res_15h)

In [93]:
res_15h=error_dict

In [95]:
%store res_15h

Stored 'res_15h' (dict)


In [71]:
len(res_15h)

187

## 1501 - 2000

In [69]:
sec_2k=sec_1[1500:2000]

In [97]:
sec_2k_raw=multiple_sequence_search(sec_2k)

1 sequences of 500
2 sequences of 500
3 sequences of 500
4 sequences of 500
5 sequences of 500
6 sequences of 500
7 sequences of 500
8 sequences of 500
9 sequences of 500
10 sequences of 500
11 sequences of 500
12 sequences of 500
13 sequences of 500
14 sequences of 500
15 sequences of 500
16 sequences of 500
17 sequences of 500
18 sequences of 500
19 sequences of 500
20 sequences of 500
21 sequences of 500
22 sequences of 500
23 sequences of 500
24 sequences of 500
25 sequences of 500
26 sequences of 500
27 sequences of 500
28 sequences of 500
29 sequences of 500
30 sequences of 500
31 sequences of 500
32 sequences of 500
33 sequences of 500
34 sequences of 500
35 sequences of 500
36 sequences of 500
37 sequences of 500
38 sequences of 500
39 sequences of 500
40 sequences of 500
41 sequences of 500
42 sequences of 500
43 sequences of 500
44 sequences of 500
45 sequences of 500
46 sequences of 500
47 sequences of 500
48 sequences of 500
49 sequences of 500
50 sequences of 500
51 sequen

397 sequences of 500
398 sequences of 500
399 sequences of 500
400 sequences of 500
401 sequences of 500
402 sequences of 500
403 sequences of 500
404 sequences of 500
405 sequences of 500
406 sequences of 500
407 sequences of 500
408 sequences of 500
409 sequences of 500
410 sequences of 500
411 sequences of 500
412 sequences of 500
413 sequences of 500
414 sequences of 500
415 sequences of 500
416 sequences of 500
417 sequences of 500
418 sequences of 500
419 sequences of 500
420 sequences of 500
421 sequences of 500
422 sequences of 500
423 sequences of 500
424 sequences of 500
425 sequences of 500
426 sequences of 500
427 sequences of 500
428 sequences of 500
429 sequences of 500
430 sequences of 500
431 sequences of 500
432 sequences of 500
433 sequences of 500
434 sequences of 500
435 sequences of 500
436 sequences of 500
437 sequences of 500
438 sequences of 500
439 sequences of 500
440 sequences of 500
441 sequences of 500
442 sequences of 500
443 sequences of 500
444 sequences

In [98]:
%store sec_2k_raw

Stored 'sec_2k_raw' (dict)


In [99]:
res_2k=results_to_dict(sec_2k_raw)

In [100]:
%store res_2k

Stored 'res_2k' (dict)


In [70]:
len(res_2k)

189

## 2000 - 2500

In [18]:
sec_25h=sec_1[2000:2500]

In [21]:
sec_25h_raw=multiple_sequence_search(sec_25h)

1 sequences of 500
2 sequences of 500
3 sequences of 500
4 sequences of 500
5 sequences of 500
6 sequences of 500
7 sequences of 500
8 sequences of 500
9 sequences of 500
10 sequences of 500
11 sequences of 500
12 sequences of 500
13 sequences of 500
14 sequences of 500
15 sequences of 500
16 sequences of 500
17 sequences of 500
18 sequences of 500
19 sequences of 500
20 sequences of 500
21 sequences of 500
22 sequences of 500
23 sequences of 500
24 sequences of 500
25 sequences of 500
26 sequences of 500
27 sequences of 500
28 sequences of 500
29 sequences of 500
30 sequences of 500
31 sequences of 500
32 sequences of 500
33 sequences of 500
34 sequences of 500
35 sequences of 500
36 sequences of 500
37 sequences of 500
38 sequences of 500
39 sequences of 500
40 sequences of 500
41 sequences of 500
42 sequences of 500
43 sequences of 500
44 sequences of 500
45 sequences of 500
46 sequences of 500
47 sequences of 500
48 sequences of 500
49 sequences of 500
50 sequences of 500
51 sequen

397 sequences of 500
398 sequences of 500
399 sequences of 500
400 sequences of 500
401 sequences of 500
402 sequences of 500
403 sequences of 500
404 sequences of 500
405 sequences of 500
406 sequences of 500
407 sequences of 500
408 sequences of 500
409 sequences of 500
410 sequences of 500
411 sequences of 500
412 sequences of 500
413 sequences of 500
414 sequences of 500
415 sequences of 500
416 sequences of 500
417 sequences of 500
418 sequences of 500
419 sequences of 500
420 sequences of 500
421 sequences of 500
422 sequences of 500
423 sequences of 500
424 sequences of 500
425 sequences of 500
426 sequences of 500
427 sequences of 500
428 sequences of 500
429 sequences of 500
430 sequences of 500
431 sequences of 500
432 sequences of 500
433 sequences of 500
434 sequences of 500
435 sequences of 500
436 sequences of 500
437 sequences of 500
438 sequences of 500
439 sequences of 500
440 sequences of 500
441 sequences of 500
442 sequences of 500
443 sequences of 500
444 sequences

In [22]:
res_25h=results_to_dict(sec_25h_raw)

In [27]:
%store res_25h

Stored 'res_25h' (dict)


In [28]:
%store sec_25h_raw

Stored 'sec_25h_raw' (dict)


In [69]:
len(res_25h)

173

## 2500-3000

In [12]:
sec_3k=sec_1[2500:3000]

In [14]:
res_3k_raw=multiple_sequence_search(sec_3k)

1 sequences of 500
2 sequences of 500
3 sequences of 500
4 sequences of 500
5 sequences of 500
6 sequences of 500
7 sequences of 500
8 sequences of 500
9 sequences of 500
10 sequences of 500
11 sequences of 500
12 sequences of 500
13 sequences of 500
14 sequences of 500
15 sequences of 500
16 sequences of 500
17 sequences of 500
18 sequences of 500
19 sequences of 500
20 sequences of 500
21 sequences of 500
22 sequences of 500
23 sequences of 500
24 sequences of 500
25 sequences of 500
26 sequences of 500
27 sequences of 500
28 sequences of 500
29 sequences of 500
30 sequences of 500
31 sequences of 500
32 sequences of 500
33 sequences of 500
34 sequences of 500
35 sequences of 500
36 sequences of 500
37 sequences of 500
38 sequences of 500
39 sequences of 500
40 sequences of 500
41 sequences of 500
42 sequences of 500
43 sequences of 500
44 sequences of 500
45 sequences of 500
46 sequences of 500
47 sequences of 500
48 sequences of 500
49 sequences of 500
50 sequences of 500
51 sequen

397 sequences of 500
398 sequences of 500
399 sequences of 500
400 sequences of 500
401 sequences of 500
402 sequences of 500
403 sequences of 500
404 sequences of 500
405 sequences of 500
406 sequences of 500
407 sequences of 500
408 sequences of 500
409 sequences of 500
410 sequences of 500
411 sequences of 500
412 sequences of 500
413 sequences of 500
414 sequences of 500
415 sequences of 500
416 sequences of 500
417 sequences of 500
418 sequences of 500
419 sequences of 500
420 sequences of 500
421 sequences of 500
422 sequences of 500
423 sequences of 500
424 sequences of 500
425 sequences of 500
426 sequences of 500
427 sequences of 500
428 sequences of 500
429 sequences of 500
430 sequences of 500
431 sequences of 500
432 sequences of 500
433 sequences of 500
434 sequences of 500
435 sequences of 500
436 sequences of 500
437 sequences of 500
438 sequences of 500
439 sequences of 500
440 sequences of 500
441 sequences of 500
442 sequences of 500
443 sequences of 500
444 sequences

In [15]:
res_3k=results_to_dict(res_3k_raw)

In [16]:
%store res_3k res_3k_raw

Stored 'res_3k' (dict)
Stored 'res_3k_raw' (dict)


In [17]:
len(res_3k)

169

## 3000-3500 (colab)

In [18]:
res_35h={'GIGKFLHSAKKPGKAFVGEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 8.7e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 8.9e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 2.1e-08, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 2.9e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 5.1e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 5.1e-07, 'percentage_identity': 82.6, 'result_sequence': None}], 'VDKPPYLPRPRPPRRIYN': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezn_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '4ezn', 'e_value': 0.0013, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.0022, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3dpp_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpp', 'e_value': 0.037, 'percentage_identity': 72.2, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '3dpq_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpq', 'e_value': 0.037, 'percentage_identity': 72.2, 'result_sequence': None}], 'GLMSVTKGVLKTAGKHIFKNVGGSLLDQAKCKISGQC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.041, 'percentage_identity': 43.2, 'result_sequence': None}], 'DTFDYKKFGYRYDSLELEGR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3j32_1', 'molecule_name': ['Hemocyanin isoform 1'], 'pdb_id': '3j32', 'e_value': 1.3e-07, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4bed_2', 'molecule_name': ['Hemocyanin 1'], 'pdb_id': '4bed', 'e_value': 0.00067, 'percentage_identity': 73.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1lnl_1', 'molecule_name': ['Hemocyanin type 2 unit e'], 'pdb_id': '1lnl', 'e_value': 0.019, 'percentage_identity': 70.0, 'result_sequence': None}], 'VTCDLLSFEAKGFAANHSICAAHCLAIGRKGGSCQNGVCVCRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 1e-15, 'percentage_identity': 79.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 9.8e-09, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.3e-08, 'percentage_identity': 62.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 5.3e-08, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 9.7e-08, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 2.1e-07, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 4.8e-07, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 6.8e-07, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 1.2e-06, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 1.5e-06, 'percentage_identity': 63.3, 'result_sequence': None}], 'IGKEFKRIVQRIKDFLRNLVPRTES': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 2.8e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 3.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 3.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 3.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 3.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 3.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'GTLPCGESCVWIPCISSVVGCSCKSKVCYKD': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 4.6e-13, 'percentage_identity': 82.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 4.6e-13, 'percentage_identity': 82.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 2.1e-12, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 6.8e-11, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 8.3e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 3.9e-10, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 9.4e-10, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 1.1e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 3.6e-06, 'percentage_identity': 57.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 4.5e-06, 'percentage_identity': 65.4, 'result_sequence': None}], 'GEFKRIVQRIKDFLRNLV': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 2.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 3e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'KWKLFKKIPKFLH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.012, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.012, 'percentage_identity': 80.0, 'result_sequence': None}], 'EIEKFDKSKLK': [{'chain_id': ['H'], 'entity_id': 2, 'entry_entity': '4pl8_2', 'molecule_name': ['Protein cordon-bleu', 'Thymosin beta-4'], 'pdb_id': '4pl8', 'e_value': 0.0059, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4pl7_1', 'molecule_name': ['Actin', 'Thymosin beta-4'], 'pdb_id': '4pl7', 'e_value': 0.048, 'percentage_identity': 100.0, 'result_sequence': None}], 'ATRSYGNGVYCNNSKCWVNWGEAKENIAGIVISGWASGLAGMGH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2a2b_1', 'molecule_name': ['Bacteriocin curvacin-A'], 'pdb_id': '2a2b', 'e_value': 1.3e-13, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw5_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1cw5', 'e_value': 4.4e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ry3_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1ry3', 'e_value': 5.5e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohn_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohn', 'e_value': 0.00016, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1og7_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1og7', 'e_value': 0.00016, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohm_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohm', 'e_value': 0.001, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ukz_1', 'molecule_name': ['Bacteriocin pediocin PA-1'], 'pdb_id': '5ukz', 'e_value': 0.005, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2leu', 'e_value': 0.0053, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw6_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '1cw6', 'e_value': 0.0053, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '3leu', 'e_value': 0.0053, 'percentage_identity': 66.7, 'result_sequence': None}], 'RIKRFWPVVIRTVV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 0.00087, 'percentage_identity': 71.4, 'result_sequence': None}], 'FLPILASLAAKFGPKLFSLVTKKS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4x_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4x', 'e_value': 3e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4v_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4v', 'e_value': 3e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r95_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r95', 'e_value': 3.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r96_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r96', 'e_value': 3.2e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 7.8e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 7.8e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 7.8e-10, 'percentage_identity': 95.7, 'result_sequence': None}], 'FIFHIIKGLFHAGKMIHGLVTRRRHGVEELQDLDQRAFEREKAFA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5h2s_1', 'molecule_name': ['Piscidin-4'], 'pdb_id': '5h2s', 'e_value': 0.00071, 'percentage_identity': 70.8, 'result_sequence': None}], 'VTCDLLSFKGQVNDSACAANCLSLGKAGGHCEKGVCICRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 3.4e-09, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 2.1e-08, 'percentage_identity': 64.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 1.1e-07, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 7.8e-07, 'percentage_identity': 53.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 8.8e-07, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 6e-06, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 3.1e-05, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 0.00013, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 0.0002, 'percentage_identity': 48.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 0.0002, 'percentage_identity': 48.7, 'result_sequence': None}], 'LSFPTTKTYFPHF': [{'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5jdo_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5jdo', 'e_value': 1.8e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'E'], 'entity_id': 1, 'entry_entity': '3kmf_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3kmf', 'e_value': 1.8e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5kdq_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5kdq', 'e_value': 1.8e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'WPKRLTKAHWFEIQHIQPSPLQCNRAMSGINNYTQHCKHQNTFLH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 1.1e-23, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 1.1e-23, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 1.1e-23, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 1.1e-23, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 1.9e-21, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 2.8e-11, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 9.5e-09, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 9.5e-09, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 9.5e-09, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 9.5e-09, 'percentage_identity': 55.8, 'result_sequence': None}], 'KTCENLSGTFKGPCIPDGNCNKHCRNNEHLLSGRCRDDFRCWCTNRC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 1.9e-24, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 1e-20, 'percentage_identity': 85.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 1.5e-17, 'percentage_identity': 70.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 8e-17, 'percentage_identity': 70.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 1.4e-16, 'percentage_identity': 68.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jkz_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '1jkz', 'e_value': 4e-09, 'percentage_identity': 44.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 6.2e-07, 'percentage_identity': 47.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 4.8e-06, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 4.8e-06, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 3.7e-05, 'percentage_identity': 41.7, 'result_sequence': None}], 'MGAVVKGGLKIIGGTAASWLGWEAGTRIWK': [{'chain_id': ['I', 'K', 'J', 'L'], 'entity_id': 3, 'entry_entity': '5c8j_3', 'molecule_name': ['Protein MJ0480'], 'pdb_id': '5c8j', 'e_value': 0.08, 'percentage_identity': 52.2, 'result_sequence': None}], 'VDKGRYLEAPTPPRPIYNRN': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezn_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '4ezn', 'e_value': 5.5e-07, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3dpp_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpp', 'e_value': 0.00018, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '3dpq_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpq', 'e_value': 0.00018, 'percentage_identity': 75.0, 'result_sequence': None}], 'FSLKDLFVPVIKDQVSDLWRTGDIDLVGHSCTYNVKPDIQGFELYFIGSVTCPGWTTLRGESNTRSKSGVVNSAVKDFIQKALKAGLVTEEEAKPHLV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2job_1', 'molecule_name': ['Anti-lipopolysaccharide factor isoform 3'], 'pdb_id': '2job', 'e_value': 1.8e-17, 'percentage_identity': 42.5, 'result_sequence': None}], 'KPKGMTSSQWFKIQHMQPSPQACNSAMKNINKHTKRCKDLNTFLHEPFSSVAATCQTPKIACKNGDKNCHQSHGAVSLTMCKLTSGKYPNCRYKEKRQNKSYVVACK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 1.4e-50, 'percentage_identity': 99.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 2.7e-26, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 2.7e-26, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 2.7e-26, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 2.7e-26, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 1.5e-25, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nui_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2nui', 'e_value': 6.9e-18, 'percentage_identity': 45.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '4srn', 'e_value': 1.1e-17, 'percentage_identity': 45.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3srn_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3srn', 'e_value': 1.1e-17, 'percentage_identity': 45.0, 'result_sequence': None}], 'GIGKFLKKAKKFGKAFVKMKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.00019, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.0002, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 0.0025, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 0.0025, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 0.0044, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.0044, 'percentage_identity': 76.5, 'result_sequence': None}], 'FQNRRMKWKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2o00_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2o00', 'e_value': 0.00074, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nzz_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2nzz', 'e_value': 0.00074, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlx_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlx', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlw_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlw', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4xic_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '4xic', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}], 'ATCDLLSGFGVGDSACAAHCIARRNRGGYCNAKKVCVCRN': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 1.8e-16, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 8.1e-15, 'percentage_identity': 77.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1e-14, 'percentage_identity': 77.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 2.3e-14, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 1.9e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 2.5e-12, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 2.6e-11, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 5e-11, 'percentage_identity': 68.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 8.4e-08, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 3.1e-06, 'percentage_identity': 51.1, 'result_sequence': None}], 'AQRCGDQARGAKCPNCLCCGKYGFCGSGDAYCGAGSCQSQCRGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lb7_1', 'molecule_name': ['Antimicrobial peptide 1a'], 'pdb_id': '2lb7', 'e_value': 3.1e-21, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3iwr_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '3iwr', 'e_value': 3.8e-11, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dkv_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '2dkv', 'e_value': 3.8e-11, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 3.9e-10, 'percentage_identity': 64.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uln_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uln', 'e_value': 6.5e-09, 'percentage_identity': 56.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uha_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uha', 'e_value': 6.5e-09, 'percentage_identity': 56.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulm_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1ulm', 'e_value': 6.5e-09, 'percentage_identity': 56.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mpi_1', 'molecule_name': ['Probable inactive chitinase-like protein LaCIC'], 'pdb_id': '4mpi', 'e_value': 1.6e-08, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 2.2e-08, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9wga_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '9wga', 'e_value': 2.2e-08, 'percentage_identity': 60.5, 'result_sequence': None}], 'FLLPPSTACCTQLYRKPLSDKLLRKVIQVELQEADGDCHLQAFVLHLAQRSICIHPQNPSLSQWFEHQERKLHGTLPKLNFGMLRKMG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kum_1', 'molecule_name': ['C-C motif chemokine 27'], 'pdb_id': '2kum', 'e_value': 5.6e-43, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6cws_1', 'molecule_name': ['C-C motif chemokine 28'], 'pdb_id': '6cws', 'e_value': 9.1e-11, 'percentage_identity': 39.7, 'result_sequence': None}], 'FFHHAFRGIVHVGKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 1.6e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 1.6e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 1.6e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 1.6e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 1.6e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 2.6e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 1.4e-08, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 8.8e-06, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 8.8e-06, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 8.8e-06, 'percentage_identity': 63.6, 'result_sequence': None}], 'IVPFLLGMVPKLVCLITKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 0.097, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 0.097, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 0.097, 'percentage_identity': 81.8, 'result_sequence': None}], 'KWKSFIKKAKTSFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.0058, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.0058, 'percentage_identity': 65.0, 'result_sequence': None}], 'VTCDLLSFEAKGFAANHSICAAHCLVIGRKGGACQNGVCVCRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 7.9e-15, 'percentage_identity': 74.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 4.6e-09, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 9.1e-09, 'percentage_identity': 62.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 3.5e-08, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 4.4e-08, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 1.4e-07, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 1.5e-06, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 2.1e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 3.8e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 5.5e-06, 'percentage_identity': 60.0, 'result_sequence': None}], 'WQRRMRKLGAPSIT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 0.0001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00027, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0075, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.0075, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.0075, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['X', 'S'], 'entity_id': 2, 'entry_entity': '2j4u_2', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2j4u', 'e_value': 0.014, 'percentage_identity': 64.3, 'result_sequence': None}], 'RWCVYAYRRVRGVLVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 1.4e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 2.2e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 2.9e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 2.9e-11, 'percentage_identity': 90.5, 'result_sequence': None}], 'PPPVIKFNRPFLMWIVERDTRSILFMGKIVNPKAP': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3ndf_2', 'molecule_name': ['Short peptide from AAT'], 'pdb_id': '3ndf', 'e_value': 5.4e-08, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3ndd_2', 'molecule_name': ['Short peptide from AAT'], 'pdb_id': '3ndd', 'e_value': 5.4e-08, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1ezx_2', 'molecule_name': ['Short peptide from AAT'], 'pdb_id': '1ezx', 'e_value': 5.4e-08, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7api_2', 'molecule_name': ['Short peptide from AAT'], 'pdb_id': '7api', 'e_value': 5.4e-08, 'percentage_identity': 58.1, 'result_sequence': None}], 'STIGKLVKWIIKTVNKFTKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dtb_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2dtb', 'e_value': 3.6e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kam_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2kam', 'e_value': 3.6e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dtc_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '1dtc', 'e_value': 3.6e-06, 'percentage_identity': 90.0, 'result_sequence': None}], 'LVQRGRFGRFLKKVRRFIPKVIIAAQIGSRFG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gdl_1', 'molecule_name': ['myeloid antimicrobial peptide 27'], 'pdb_id': '2gdl', 'e_value': 2.5e-09, 'percentage_identity': 71.0, 'result_sequence': None}], 'NGVQPWYKWWKWWKKWW': [{'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '1yst_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '1yst', 'e_value': 0.085, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['AL', 'BL'], 'entity_id': 5, 'entry_entity': '4v9g_5', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '4v9g', 'e_value': 0.087, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '5lri_1', 'molecule_name': ['Reaction center protein L chain'], 'pdb_id': '5lri', 'e_value': 0.087, 'percentage_identity': 57.1, 'result_sequence': None}], 'RQRDPQQQYEQCQERCQRHETEPRHMQTCQQRCERRYEKEKRKQQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6o3q_1', 'molecule_name': ['Vicilin'], 'pdb_id': '6o3q', 'e_value': 7.7e-06, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lqx_1', 'molecule_name': ['Trypsin inhibitor BWI-2c'], 'pdb_id': '2lqx', 'e_value': 0.00051, 'percentage_identity': 31.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l37_1', 'molecule_name': ['Ribosome-inactivating protein luffin P1'], 'pdb_id': '2l37', 'e_value': 0.00079, 'percentage_identity': 41.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6o3s_1', 'molecule_name': ['Ribosome-inactivating protein luffin P1'], 'pdb_id': '6o3s', 'e_value': 0.00085, 'percentage_identity': 41.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6wql_1', 'molecule_name': ['Cupin type-1 domain-containing protein'], 'pdb_id': '6wql', 'e_value': 0.014, 'percentage_identity': 40.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l2r_1', 'molecule_name': ['Antimicrobial peptide EcAMP1'], 'pdb_id': '2l2r', 'e_value': 0.098, 'percentage_identity': 46.2, 'result_sequence': None}], 'GLPVCGETCVGGTCNTPGCACSWPVCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 5.3e-14, 'percentage_identity': 96.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 5.3e-14, 'percentage_identity': 96.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 1.5e-13, 'percentage_identity': 93.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 2.2e-13, 'percentage_identity': 96.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 2.2e-13, 'percentage_identity': 96.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 2.7e-13, 'percentage_identity': 93.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 6.3e-13, 'percentage_identity': 93.3, 'result_sequence': None}], 'FKRLEKLFKKIWNWK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 6.4e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 0.00054, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 0.00054, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 0.0016, 'percentage_identity': 86.7, 'result_sequence': None}], 'GIGGALLSAGKSALKGLAKGLAEHFAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 1.2e-07, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.025, 'percentage_identity': 60.0, 'result_sequence': None}], 'FLSLIPHAINAVGVHAKHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq0_1', 'molecule_name': ['Phylloseptin-H1'], 'pdb_id': '2jq0', 'e_value': 3.9e-05, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpy_1', 'molecule_name': ['Phylloseptin-H2'], 'pdb_id': '2jpy', 'e_value': 7.3e-05, 'percentage_identity': 73.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq1_1', 'molecule_name': ['Phyllospetin-H3'], 'pdb_id': '2jq1', 'e_value': 0.00019, 'percentage_identity': 77.8, 'result_sequence': None}], 'GFMDTAKNAAKNVAVTLLDNLKCKITKAC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.013, 'percentage_identity': 53.6, 'result_sequence': None}], 'FLSFPTTKTYFPHF': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5woh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5woh', 'e_value': 2.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5wog_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5wog', 'e_value': 2.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5jdo_3', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5jdo', 'e_value': 3e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'GVVDILKGAAKDIAGHLASKVMAKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 4.6e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 4.6e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 4.6e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 6.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 6.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 6.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 7e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 7e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 7e-09, 'percentage_identity': 100.0, 'result_sequence': None}], 'STLHLVARLRGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mws_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mws', 'e_value': 0.0041, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5jtj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtj', 'e_value': 0.0041, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3w', 'e_value': 0.0041, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbe', 'e_value': 0.0041, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nr2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nr2', 'e_value': 0.0041, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbd', 'e_value': 0.0041, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5k9p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5k9p', 'e_value': 0.0041, 'percentage_identity': 91.7, 'result_sequence': None}], 'GFGSLFKFLAKKVAKTVAKQAAKQGAKYVANKHME': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k38_1', 'molecule_name': ['Cupiennin-1a'], 'pdb_id': '2k38', 'e_value': 1.9e-14, 'percentage_identity': 91.4, 'result_sequence': None}], 'VLSAADKGNVKAAWGKVGGHAAEYGAEALERM': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ii1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ii1', 'e_value': 1.3e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 1.4e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 1.4e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 1.4e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 1.4e-14, 'percentage_identity': 100.0, 'result_sequence': None}], 'GFGCNGPWSEKDMHCHNHCKSIKGYKGGYCAKGGFICKCY': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 1.6e-20, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 1.6e-20, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 1.6e-20, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 3.7e-20, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 4.5e-20, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 3.8e-05, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 0.00028, 'percentage_identity': 46.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 0.00075, 'percentage_identity': 47.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 0.0049, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 0.0065, 'percentage_identity': 41.0, 'result_sequence': None}], 'RTCESQSHRFKGPCARDSNCATVCLTEGFSGGDCRGFRRRCFCTRPC': [{'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 8.7e-19, 'percentage_identity': 80.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 1.9e-18, 'percentage_identity': 80.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 4.1e-18, 'percentage_identity': 78.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 1.6e-15, 'percentage_identity': 68.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6lcq_1', 'molecule_name': ['Defensin-like protein CAL1'], 'pdb_id': '6lcq', 'e_value': 3.1e-11, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gpt_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gpt', 'e_value': 1.6e-10, 'percentage_identity': 53.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gps_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gps', 'e_value': 7.3e-10, 'percentage_identity': 51.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4uj0_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '4uj0', 'e_value': 1.1e-08, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nce_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '5nce', 'e_value': 1.1e-08, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ksk_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2ksk', 'e_value': 5.4e-08, 'percentage_identity': 44.0, 'result_sequence': None}], 'GLNTLKKVIQGLHEVIKLVNNHA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncx_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncx', 'e_value': 3.5e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncy_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncy', 'e_value': 6.1e-06, 'percentage_identity': 72.7, 'result_sequence': None}], 'SRSEKAGLQFPVGRIGRMLKK': [{'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 5, 'entry_entity': '4x23_5', 'molecule_name': ['Histone H2A'], 'pdb_id': '4x23', 'e_value': 4e-06, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['M', 'Q', 'C', 'G'], 'entity_id': 3, 'entry_entity': '5wcu_3', 'molecule_name': ['Histone H2A'], 'pdb_id': '5wcu', 'e_value': 4e-06, 'percentage_identity': 76.2, 'result_sequence': None}], 'GIGTKILGGVKAALKGALKELASTYVN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 1.3e-06, 'percentage_identity': 74.1, 'result_sequence': None}], 'KFKSFIKKLTSKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.092, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.092, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.092, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.092, 'percentage_identity': 65.0, 'result_sequence': None}], 'PTTKTYFPHFDLSHGSAQVKGHGAKVAAALTKAVEHLDDLPGALSELSDLHAHKLRVDPVNF': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ii1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ii1', 'e_value': 1.1e-27, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 1.2e-27, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 1.2e-27, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 1.2e-27, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 1.2e-27, 'percentage_identity': 100.0, 'result_sequence': None}], 'DAEFRHDSGYEVHHQKLVFFAEDVGSNKGAIIGLMVGGVVIA': [{'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2nao_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '2nao', 'e_value': 5.4e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2mxu_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '2mxu', 'e_value': 5.4e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '5kk3_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '5kk3', 'e_value': 5.4e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '5oqv_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '5oqv', 'e_value': 5.4e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2wk3_2', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '2wk3', 'e_value': 5.4e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E'], 'entity_id': 1, 'entry_entity': '2beg_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '2beg', 'e_value': 5.4e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z0q_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '1z0q', 'e_value': 5.4e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1iyt_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '1iyt', 'e_value': 5.4e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6szf_1', 'molecule_name': ['Amyloid-beta protein 42'], 'pdb_id': '6szf', 'e_value': 5.4e-20, 'percentage_identity': 100.0, 'result_sequence': None}], 'SGGYCGGWHRLRCTSYRSG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 1.3e-08, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 0.095, 'percentage_identity': 56.2, 'result_sequence': None}], 'PLPFPRPGPRPIPRPLPFPRPGPRPIPRP': [{'chain_id': ['a0'], 'entity_id': 1, 'entry_entity': '6ncl_1', 'molecule_name': ['P14'], 'pdb_id': '6ncl', 'e_value': 0.00013, 'percentage_identity': 51.7, 'result_sequence': None}], 'RTCENLADKYRGPCFSGCDTHCTTKENAVSGRCRGFRRRCWCTKRC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 1.7e-12, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 2.1e-12, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 4.5e-12, 'percentage_identity': 60.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 1.4e-10, 'percentage_identity': 52.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 9.1e-10, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 1.6e-09, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 3.4e-09, 'percentage_identity': 54.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 4.1e-09, 'percentage_identity': 52.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jkz_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '1jkz', 'e_value': 1.3e-08, 'percentage_identity': 47.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 5.6e-06, 'percentage_identity': 41.7, 'result_sequence': None}], 'KWKLFKKIFKRIVQRIKDFLR': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.034, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.04, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.04, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.04, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.04, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.04, 'percentage_identity': 93.3, 'result_sequence': None}], 'GIGKFLHSAKKFGKAFVAEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 5.1e-11, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 5.3e-11, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 4.2e-10, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 7.8e-10, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 7.8e-10, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 1.4e-09, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.0066, 'percentage_identity': 100.0, 'result_sequence': None}], 'KKKKKKKKKKGIGKLFLHAAKKFAKAFVAEKMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 1.4e-06, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 6.3e-06, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 6.3e-06, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 4.7e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 4.9e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.00075, 'percentage_identity': 70.8, 'result_sequence': None}], 'TYMPVEEGEYIVNISYADQPKKNSPFTAKKQPGPKVDLSGVKAYGPG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgx_1', 'molecule_name': ['Filamin-C'], 'pdb_id': '4mgx', 'e_value': 0.0012, 'percentage_identity': 42.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3v8o_1', 'molecule_name': ['Filamin-C'], 'pdb_id': '3v8o', 'e_value': 0.0013, 'percentage_identity': 42.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7q_1', 'molecule_name': ['Filamin-A'], 'pdb_id': '2k7q', 'e_value': 0.01, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2j3s_1', 'molecule_name': ['Filamin-A'], 'pdb_id': '2j3s', 'e_value': 0.015, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dj4_1', 'molecule_name': ['Filamin-B'], 'pdb_id': '2dj4', 'e_value': 0.024, 'percentage_identity': 46.9, 'result_sequence': None}], 'RRRRRRRRRRGIGKFLHSAKKFGKAFVGEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 1.1e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 1.2e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 1.6e-07, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 1.4e-06, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 2.3e-06, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 2.3e-06, 'percentage_identity': 87.0, 'result_sequence': None}], 'GSKGAPCAKKPCCGPLGHYKVDCSTIPDYPCCSKYGNCGSGPQYCG': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aml_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '4aml', 'e_value': 0.00099, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2x3t_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2x3t', 'e_value': 0.00099, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2uvo_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2uvo', 'e_value': 0.00099, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgc_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '1wgc', 'e_value': 0.00099, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2cwg_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '2cwg', 'e_value': 0.00099, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7wga_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '7wga', 'e_value': 0.00099, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 0.0014, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 0.0014, 'percentage_identity': 39.1, 'result_sequence': None}], 'DKLIGSCVWGAVNYTSDCNGECLLRGYKGGHCGSFANVNCWCRT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 2.8e-23, 'percentage_identity': 97.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 7.3e-22, 'percentage_identity': 93.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 1.3e-20, 'percentage_identity': 88.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 5.4e-20, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 6.6e-20, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkb_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-5'], 'pdb_id': '2lkb', 'e_value': 0.0018, 'percentage_identity': 39.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kbk_1', 'molecule_name': ['Neurotoxin BmK-M10'], 'pdb_id': '2kbk', 'e_value': 0.02, 'percentage_identity': 37.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.023, 'percentage_identity': 42.1, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.026, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1kv0_1', 'molecule_name': ['Alpha-like toxin BmK-M7'], 'pdb_id': '1kv0', 'e_value': 0.031, 'percentage_identity': 37.5, 'result_sequence': None}], 'VKRFKKFFRKFKKSV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.0057, 'percentage_identity': 85.7, 'result_sequence': None}], 'FRIRVRVFKRIVQRIKDFLR': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.019, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.022, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.022, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.022, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.022, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.022, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLAANFLPKIFCKITRKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 0.00077, 'percentage_identity': 64.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 0.00077, 'percentage_identity': 64.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 0.00077, 'percentage_identity': 64.7, 'result_sequence': None}], 'KFFKKLKNSVKKRAKKFFKKPRVIGVSIPF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 2.8e-09, 'percentage_identity': 80.0, 'result_sequence': None}], 'QYEALVTSILGKLTGLWHNDSVDFMGHICYFRRRPKIRRFKLYHEGKFWCPGWAPFEGRCKYCVVF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2job_1', 'molecule_name': ['Anti-lipopolysaccharide factor isoform 3'], 'pdb_id': '2job', 'e_value': 5.8e-12, 'percentage_identity': 38.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mdl_1', 'molecule_name': ['Anti-lipopolysaccharide factor'], 'pdb_id': '2mdl', 'e_value': 1.6e-07, 'percentage_identity': 78.3, 'result_sequence': None}], 'GIGKFLHSAKKFAKAFVAEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 5.9e-11, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 1.1e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 3.7e-10, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 3.7e-10, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 3.8e-10, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 1.1e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.0053, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.0053, 'percentage_identity': 100.0, 'result_sequence': None}], 'HHQELCTKGDDALVTELECIRLRISPETNAAFDNAVQQLNCLNRACAYRKMCATNNLEQAMSVYFTNEQIKEIHDAATACDPEAHHEHDH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knj_1', 'molecule_name': ['Antimicrobial peptide microplusin'], 'pdb_id': '2knj', 'e_value': 4e-48, 'percentage_identity': 100.0, 'result_sequence': None}], 'KSCCRSTQARNIYNAPRFAGGSRPLCALGSGCKIVDDKKTPPND': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orl_1', 'molecule_name': ['Viscotoxin-C1'], 'pdb_id': '1orl', 'e_value': 2e-09, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jmn_1', 'molecule_name': ['Viscotoxin-A2'], 'pdb_id': '1jmn', 'e_value': 2.5e-09, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1okh_1', 'molecule_name': ['Viscotoxin-A3'], 'pdb_id': '1okh', 'e_value': 9.1e-09, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ed0_1', 'molecule_name': ['Viscotoxin-A3'], 'pdb_id': '1ed0', 'e_value': 9.1e-09, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3c8p_1', 'molecule_name': ['Viscotoxin A1'], 'pdb_id': '3c8p', 'e_value': 1.4e-08, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jmp_1', 'molecule_name': ['Viscotoxin-B'], 'pdb_id': '1jmp', 'e_value': 2.7e-08, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2v9b_1', 'molecule_name': ['Viscotoxin-B'], 'pdb_id': '2v9b', 'e_value': 8.2e-08, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '3szs_1', 'molecule_name': ['Hellethionin-D'], 'pdb_id': '3szs', 'e_value': 1.6e-07, 'percentage_identity': 53.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbl_1', 'molecule_name': ['Hellethionin-D'], 'pdb_id': '1nbl', 'e_value': 1.6e-07, 'percentage_identity': 53.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wuw_1', 'molecule_name': ['Beta-hordothionin'], 'pdb_id': '1wuw', 'e_value': 8.9e-05, 'percentage_identity': 48.8, 'result_sequence': None}], 'SFFKKAAHVGKHVGKAALTHYL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 8e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 8.2e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 1.5e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 2.7e-09, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 0.00085, 'percentage_identity': 54.5, 'result_sequence': None}], 'QYEALTAAILTKLSKMWHSDTLNFLGHTCHVSRTPTVKRFKLYWKGKFWCPGWAPFSGTSRTKSRSGSAREATKSFVDQALQRRLITQQEADLWLKG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2job_1', 'molecule_name': ['Anti-lipopolysaccharide factor isoform 3'], 'pdb_id': '2job', 'e_value': 2e-20, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mdl_1', 'molecule_name': ['Anti-lipopolysaccharide factor'], 'pdb_id': '2mdl', 'e_value': 3.7e-05, 'percentage_identity': 62.5, 'result_sequence': None}], 'ATCDLLSGTGINHSACAAHCLLRGNRGGYCNGKGVCVCRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 7.4e-21, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 4.1e-20, 'percentage_identity': 97.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 5.2e-19, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 6.6e-13, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 2.4e-12, 'percentage_identity': 67.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 6.9e-12, 'percentage_identity': 67.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 5.9e-11, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 1.5e-09, 'percentage_identity': 62.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 2e-08, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 8.9e-08, 'percentage_identity': 54.5, 'result_sequence': None}], 'GIGKFLHAALLFAKAFVAEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 1.7e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 1.1e-08, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 1.1e-08, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 1.3e-07, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 1.4e-07, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 4.2e-06, 'percentage_identity': 69.6, 'result_sequence': None}], 'ATCDLLSATGFSGTACAAHCLLIGHRGGYCNTKSVCVCRD': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 9.2e-15, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.1e-14, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 3.5e-14, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 3.3e-11, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 1.2e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 1.9e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 4.6e-10, 'percentage_identity': 57.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 9.8e-08, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 1e-07, 'percentage_identity': 52.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 7.3e-06, 'percentage_identity': 44.7, 'result_sequence': None}], 'SIGRRGGYCAGIIKQTCTCYR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 1.5e-06, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 0.00034, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 0.001, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 0.0037, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 0.046, 'percentage_identity': 60.0, 'result_sequence': None}], 'KWKSFIKKRTSKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.00097, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.00097, 'percentage_identity': 70.0, 'result_sequence': None}], 'VFQFLGRIIHHVGNFVHGFSHVF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6c41_1', 'molecule_name': ['Clavanin-A'], 'pdb_id': '6c41', 'e_value': 2.4e-12, 'percentage_identity': 95.7, 'result_sequence': None}], 'ATCDLLSGTGVKHSACAAHCLLRGNRGGYCNGRAICVCRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 1.2e-21, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 1.7e-20, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 5.2e-20, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 1.2e-13, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 1.4e-12, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 5.3e-12, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 2.6e-11, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 3.5e-09, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 1.3e-07, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 2e-07, 'percentage_identity': 52.3, 'result_sequence': None}], 'GLPVCGETCAGGTCNTPGCSCSWPICTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 1.5e-13, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 1.5e-13, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 4e-13, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 6.1e-13, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 6.1e-13, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 7.4e-13, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 1.7e-12, 'percentage_identity': 86.7, 'result_sequence': None}], 'KAEDLARYYSALRHYINLITRQRY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f8p_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1f8p', 'e_value': 4.5e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ron_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1ron', 'e_value': 7.8e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fvn_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1fvn', 'e_value': 9e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1icy_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1icy', 'e_value': 1.2e-08, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oon_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oon', 'e_value': 1.8e-07, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rlk_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2rlk', 'e_value': 7.1e-07, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz4_1', 'molecule_name': ['Pancreatic hormone', 'Neuropeptide Y'], 'pdb_id': '1tz4', 'e_value': 7.1e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qbf_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1qbf', 'e_value': 7.1e-07, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ru5_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ru5', 'e_value': 7.1e-07, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oop_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oop', 'e_value': 7.1e-07, 'percentage_identity': 72.7, 'result_sequence': None}], 'NDAACAAHCLFRGRSGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.00031, 'percentage_identity': 70.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.00031, 'percentage_identity': 70.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.0011, 'percentage_identity': 64.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.0017, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 0.031, 'percentage_identity': 58.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.052, 'percentage_identity': 69.2, 'result_sequence': None}], 'GGVCPKILQRCRRDSDCPGACICRGNGYCGSGSD': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5wov_1', 'molecule_name': ['Two inhibitor peptide topologies 2'], 'pdb_id': '5wov', 'e_value': 1.9e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5wow_1', 'molecule_name': ['Two inhibitor peptide topologies 2'], 'pdb_id': '5wow', 'e_value': 2.8e-15, 'percentage_identity': 91.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m86_1', 'molecule_name': ['MCo-PMI'], 'pdb_id': '2m86', 'e_value': 5e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2po8_1', 'molecule_name': ['Trypsin inhibitor 2'], 'pdb_id': '2po8', 'e_value': 8.7e-15, 'percentage_identity': 93.5, 'result_sequence': None}, {'chain_id': ['D', 'E', 'F'], 'entity_id': 2, 'entry_entity': '4gux_2', 'molecule_name': ['Trypsin inhibitor 2'], 'pdb_id': '4gux', 'e_value': 2.4e-14, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ib9_1', 'molecule_name': ['Trypsin inhibitor 2'], 'pdb_id': '1ib9', 'e_value': 2.4e-14, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ha9_1', 'molecule_name': ['Trypsin inhibitor 2'], 'pdb_id': '1ha9', 'e_value': 2.4e-14, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2it8_1', 'molecule_name': ['Trypsin inhibitor 2'], 'pdb_id': '2it8', 'e_value': 1.1e-13, 'percentage_identity': 93.1, 'result_sequence': None}], 'GIGKFLHSAKKAGKAFVGEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 2.5e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 2.6e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 6.7e-09, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 9.7e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 1.8e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 1.8e-07, 'percentage_identity': 82.6, 'result_sequence': None}], 'RPPQFTRAQWFAIQHISLNPPRSTIAMRAINNYRWRSKNQNTFLR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 5.6e-22, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 5.6e-22, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 5.6e-22, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 5.6e-22, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 5.6e-22, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 5.7e-22, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 5.7e-22, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 5.7e-22, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 5.7e-22, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 5.7e-22, 'percentage_identity': 95.6, 'result_sequence': None}], 'KWKSFIKKLTSKFLHSAKKN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.015, 'percentage_identity': 68.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.015, 'percentage_identity': 68.4, 'result_sequence': None}], 'GLKALKKVFKGIHKAIKLINNHVQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncx_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncx', 'e_value': 4.9e-09, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncy_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncy', 'e_value': 8e-08, 'percentage_identity': 83.3, 'result_sequence': None}], 'KWKSFIKKLTSKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.0018, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.0018, 'percentage_identity': 70.0, 'result_sequence': None}], 'STLHLVLALRGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ko3_1', 'molecule_name': ['NEDD8'], 'pdb_id': '2ko3', 'e_value': 0.0016, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4hcp_2', 'molecule_name': ['NEDD8'], 'pdb_id': '4hcp', 'e_value': 0.0016, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1xt9_2', 'molecule_name': ['NEDD8'], 'pdb_id': '1xt9', 'e_value': 0.0016, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2bkr_2', 'molecule_name': ['NEDD8'], 'pdb_id': '2bkr', 'e_value': 0.0016, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bt0_1', 'molecule_name': ['NEDD8-like protein RUB1'], 'pdb_id': '1bt0', 'e_value': 0.0016, 'percentage_identity': 91.7, 'result_sequence': None}], 'RWCVYAYVRGRGVLVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 2.2e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 2.2e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 2.2e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 2e-10, 'percentage_identity': 90.5, 'result_sequence': None}], 'GLPRKILCAIAKKKGKAKGPLKLVCKA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mbd_1', 'molecule_name': ['Lasiocepsin'], 'pdb_id': '2mbd', 'e_value': 2.2e-11, 'percentage_identity': 96.2, 'result_sequence': None}], 'RKRIHIGPGRAFYTT': [{'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '3uji_3', 'molecule_name': ['Surface protein gp120'], 'pdb_id': '3uji', 'e_value': 8.4e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P', 'Q'], 'entity_id': 3, 'entry_entity': '3mlx_3', 'molecule_name': ['Surface protein gp120'], 'pdb_id': '3mlx', 'e_value': 8.4e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P', 'Q'], 'entity_id': 1, 'entry_entity': '6mns_1', 'molecule_name': ['Surface protein gp120'], 'pdb_id': '6mns', 'e_value': 8.4e-07, 'percentage_identity': 100.0, 'result_sequence': None}], 'AKKVFKRLRKLFKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 0.0029, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0g_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0g', 'e_value': 0.0029, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 0.0029, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 0.0029, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 0.0029, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5k_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5k', 'e_value': 0.031, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5l', 'e_value': 0.077, 'percentage_identity': 80.0, 'result_sequence': None}], 'RKRLRKFRNKIKEKLKKIGQKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 2.7e-07, 'percentage_identity': 100.0, 'result_sequence': None}], 'SIGAKILGGVKTFFKGALKELAFTYLQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 9e-05, 'percentage_identity': 69.2, 'result_sequence': None}], 'VDRGWGNGCGLFGKGG': [{'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3j42_1', 'molecule_name': ['Core protein'], 'pdb_id': '3j42', 'e_value': 4.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3j35_1', 'molecule_name': ['Core protein'], 'pdb_id': '3j35', 'e_value': 4.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2r6p_1', 'molecule_name': ['Envelope protein E'], 'pdb_id': '2r6p', 'e_value': 4.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3ixy_1', 'molecule_name': ['Envelope protein E'], 'pdb_id': '3ixy', 'e_value': 4.8e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'FFAAIFRPIVHVGKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 4.8e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 1.5e-07, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 1.5e-07, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 1.6e-07, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 1.6e-07, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 1.6e-07, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 1.2e-06, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 0.012, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 0.012, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 0.012, 'percentage_identity': 54.5, 'result_sequence': None}], 'RWCVYAYVRVRGVLGRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 1.8e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 2.3e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 2.3e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 1.6e-10, 'percentage_identity': 90.5, 'result_sequence': None}], 'KRKLTLVFGVMAGVIGTKKR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2kpe_1', 'molecule_name': ['Glycophorin-A'], 'pdb_id': '2kpe', 'e_value': 0.06, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2kpf_1', 'molecule_name': ['Glycophorin-A'], 'pdb_id': '2kpf', 'e_value': 0.073, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1afo_1', 'molecule_name': ['Glycophorin-A'], 'pdb_id': '1afo', 'e_value': 0.076, 'percentage_identity': 83.3, 'result_sequence': None}], 'KKTWWKTWWTKW': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6vpd_1', 'molecule_name': ['Glutathione peroxidase'], 'pdb_id': '6vpd', 'e_value': 0.0032, 'percentage_identity': 75.0, 'result_sequence': None}], 'GLPALISSIKRKRQQ': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 9.7e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 9.7e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.0001, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.0001, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.0001, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.0001, 'percentage_identity': 93.3, 'result_sequence': None}], 'WKLLSKAQEKFGKNKSR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 1.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 1.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 0.00025, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b7u_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b7u', 'e_value': 0.0022, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i6b_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1i6b', 'e_value': 0.0022, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b7z_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b7z', 'e_value': 0.0022, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qjm_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1qjm', 'e_value': 0.0022, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f9b_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1f9b', 'e_value': 0.0022, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3cr9_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '3cr9', 'e_value': 0.0022, 'percentage_identity': 75.0, 'result_sequence': None}], 'GLPVCGETCVGGTCNTPGCSCSWPVCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 5.3e-14, 'percentage_identity': 96.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 5.3e-14, 'percentage_identity': 96.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 1.5e-13, 'percentage_identity': 93.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 2.2e-13, 'percentage_identity': 96.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 2.2e-13, 'percentage_identity': 96.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 2.7e-13, 'percentage_identity': 93.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 6.2e-13, 'percentage_identity': 93.3, 'result_sequence': None}], 'ELPKLPDDKVLIRSRSNCPKGKVWNGFDCKSPFAFS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1iyc_1', 'molecule_name': ['Scarabaecin'], 'pdb_id': '1iyc', 'e_value': 4.2e-19, 'percentage_identity': 100.0, 'result_sequence': None}], 'FFHHIKRGIVHVGKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 2.4e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 2.4e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 2.5e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 2.5e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 2.5e-09, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 3.7e-08, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 1.9e-07, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 0.0001, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 0.0001, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 0.0001, 'percentage_identity': 63.6, 'result_sequence': None}], 'GIGKFLHSAKKFGKAFVGEIMNSKKKKKKKKKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 3.2e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 3.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 5.1e-08, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 4.8e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 7.9e-07, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 7.9e-07, 'percentage_identity': 87.0, 'result_sequence': None}], 'FLPLLLAGLPKFLCLVFKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 0.071, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 0.071, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 0.071, 'percentage_identity': 58.3, 'result_sequence': None}], 'VDKPPYLPRPHPPRRIYNNR': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezn_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '4ezn', 'e_value': 0.0025, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.016, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3dpp_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpp', 'e_value': 0.068, 'percentage_identity': 72.2, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '3dpq_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpq', 'e_value': 0.068, 'percentage_identity': 72.2, 'result_sequence': None}], 'ARNFGKEFTPV': [{'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3pia_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3pia', 'e_value': 0.0015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2qss_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2qss', 'e_value': 0.0015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3pi9_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3pi9', 'e_value': 0.0015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2qsp_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2qsp', 'e_value': 0.0015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3ciu_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3ciu', 'e_value': 0.0015, 'percentage_identity': 100.0, 'result_sequence': None}], 'RLKKWMQKVIDRFGG': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5to3_2', 'molecule_name': ['Thrombin heavy chain', 'Thrombomodulin'], 'pdb_id': '5to3', 'e_value': 0.0009, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 3, 'entry_entity': '1bhx_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1bhx', 'e_value': 0.0022, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 4, 'entry_entity': '2hnt_4', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2hnt', 'e_value': 0.0022, 'percentage_identity': 85.7, 'result_sequence': None}], 'IKFEPPLPPKKAH': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e0v_1', 'molecule_name': ['L-amino-acid oxidase'], 'pdb_id': '4e0v', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5ts5_1', 'molecule_name': ['L-amino-acid oxidase'], 'pdb_id': '5ts5', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tdn_1', 'molecule_name': ['L-amino-acid oxidase'], 'pdb_id': '1tdn', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tdo_1', 'molecule_name': ['L-amino-acid oxidase'], 'pdb_id': '1tdo', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tdk_1', 'molecule_name': ['L-amino-acid oxidase'], 'pdb_id': '1tdk', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1reo_1', 'molecule_name': ['L-amino-acid oxidase'], 'pdb_id': '1reo', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5z2g_1', 'molecule_name': ['L-amino-acid oxidase'], 'pdb_id': '5z2g', 'e_value': 0.016, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '3kve_1', 'molecule_name': ['L-amino-acid oxidase'], 'pdb_id': '3kve', 'e_value': 0.026, 'percentage_identity': 84.6, 'result_sequence': None}], 'GLWSKIKEVGKEAAKAAAKAAGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k9b_1', 'molecule_name': ['Dermaseptin-DI1'], 'pdb_id': '2k9b', 'e_value': 4.1e-08, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jx6_1', 'molecule_name': ['Dermaseptin-DI1'], 'pdb_id': '2jx6', 'e_value': 4.1e-08, 'percentage_identity': 91.3, 'result_sequence': None}], 'GGLKKLGKKLEGAGKRVFKASEKALPVVVGIKAIGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmm_1', 'molecule_name': ['AAEL000598-PA'], 'pdb_id': '2mmm', 'e_value': 1.9e-16, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLSALISWIKRKRQQ': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 6.9e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 6.9e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 7.2e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 7.2e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 7.2e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 7.2e-05, 'percentage_identity': 93.3, 'result_sequence': None}], 'NLVSGLIEARKYLEQLHRKLKNRKV': [{'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '1xkm_2', 'molecule_name': ['Distinctin chain B'], 'pdb_id': '1xkm', 'e_value': 4.4e-09, 'percentage_identity': 96.0, 'result_sequence': None}], 'GIGKFLHSAKKFGKAWVGEIMNS': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 3.2e-11, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 5.3e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 5.5e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 1.5e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 2.5e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 2.5e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.024, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.024, 'percentage_identity': 100.0, 'result_sequence': None}], 'APKVNVNALRKGGRVIRKGLGVIGAAGTAHEVYNHVRNRNQG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jr8_1', 'molecule_name': ['Moricin'], 'pdb_id': '2jr8', 'e_value': 0.01, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1x22_1', 'molecule_name': ['Moricin'], 'pdb_id': '1x22', 'e_value': 0.019, 'percentage_identity': 41.0, 'result_sequence': None}], 'PRPPRLPRPRPRPLPFPRPG': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 0.0018, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x5v_1', 'molecule_name': ['GD'], 'pdb_id': '5x5v', 'e_value': 0.067, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5x5w_1', 'molecule_name': ['GD'], 'pdb_id': '5x5w', 'e_value': 0.067, 'percentage_identity': 75.0, 'result_sequence': None}], 'VNYGNGVSCSKTKCSVNWGQAFQERYTAGINSFVSGVASGAGSIGRRP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw5_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1cw5', 'e_value': 2.9e-22, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ry3_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1ry3', 'e_value': 1.2e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2leu', 'e_value': 2.8e-05, 'percentage_identity': 52.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw6_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '1cw6', 'e_value': 2.8e-05, 'percentage_identity': 52.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '3leu', 'e_value': 2.8e-05, 'percentage_identity': 52.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ukz_1', 'molecule_name': ['Bacteriocin pediocin PA-1'], 'pdb_id': '5ukz', 'e_value': 0.0019, 'percentage_identity': 42.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n4k_1', 'molecule_name': ['Enterocin-HF'], 'pdb_id': '2n4k', 'e_value': 0.0028, 'percentage_identity': 73.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2a2b_1', 'molecule_name': ['Bacteriocin curvacin-A'], 'pdb_id': '2a2b', 'e_value': 0.006, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ljq_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2ljq', 'e_value': 0.0083, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ljt_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2ljt', 'e_value': 0.012, 'percentage_identity': 47.1, 'result_sequence': None}], 'VLSAADKGNVKAAWGKVGGHAAEYGAEALERMF': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ii1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ii1', 'e_value': 5e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 5.1e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 5.1e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 5.1e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 5.1e-15, 'percentage_identity': 100.0, 'result_sequence': None}], 'KWKSFINKLTSKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.0075, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.0075, 'percentage_identity': 65.0, 'result_sequence': None}], 'RFWPVVIRTVVAGYNLYRAIKKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 3.5e-10, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hfr_1', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '2hfr', 'e_value': 2.5e-05, 'percentage_identity': 61.5, 'result_sequence': None}], 'FALLGDFFRKSKEKIGKEFKRIVQRIKDFFRNLVPRTES': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 4.6e-16, 'percentage_identity': 97.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 4.6e-16, 'percentage_identity': 97.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 4.6e-16, 'percentage_identity': 97.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 4.6e-16, 'percentage_identity': 97.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 4.6e-16, 'percentage_identity': 97.3, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 1.7e-08, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 7.8e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'RIKRFWPVVIRTVVAGY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 8.7e-06, 'percentage_identity': 76.5, 'result_sequence': None}], 'GVIPGQKQCIALKGVCRDKLCSTLDDTIGICNEGKKCCRRWWILEPYPTPVPKGKSP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lwl_1', 'molecule_name': ['Beta-defensin 106'], 'pdb_id': '2lwl', 'e_value': 0.031, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mjv_1', 'molecule_name': ['Human beta-defensin'], 'pdb_id': '6mjv', 'e_value': 0.073, 'percentage_identity': 46.9, 'result_sequence': None}], 'GFGCPFNKNECHAHCLSIGRKFGFCAGPLRATCTCGKQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 2.3e-20, 'percentage_identity': 97.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 1.3e-09, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 2.5e-07, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 7.5e-07, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 6.5e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 1.9e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 0.00082, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 0.0024, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 0.0024, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 0.0024, 'percentage_identity': 46.3, 'result_sequence': None}], 'RIKRFWPVVIRTVVAGYNLYRAIKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 8.8e-11, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hfr_1', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '2hfr', 'e_value': 7.9e-05, 'percentage_identity': 61.5, 'result_sequence': None}], 'PRLKVYLPRYKVYSTAAGRYQLLSRYWDAYR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3d3d_1', 'molecule_name': ['Endolysin'], 'pdb_id': '3d3d', 'e_value': 1.5e-05, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1d9u_1', 'molecule_name': ['Endolysin'], 'pdb_id': '1d9u', 'e_value': 1.5e-05, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1am7_1', 'molecule_name': ['Endolysin'], 'pdb_id': '1am7', 'e_value': 1.6e-05, 'percentage_identity': 68.8, 'result_sequence': None}], 'GKPRPYSPRPSSHPRPIRV': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.023, 'percentage_identity': 70.6, 'result_sequence': None}], 'GFARIVQRIKDFLRNLV': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.00013, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.00019, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.00019, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.00019, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.00019, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.00019, 'percentage_identity': 93.8, 'result_sequence': None}], 'MAFKKLEKVGRNIRDGIIKAGPAVAVIGQATSIARPTGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 4.2e-11, 'percentage_identity': 75.8, 'result_sequence': None}], 'AAAAGSCVWGAVNYTSDCAAECKRRGYKGGHCGSFANVNCWCET': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 7.7e-20, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 9.4e-20, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 1.1e-19, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 3.8e-19, 'percentage_identity': 92.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 3.4e-18, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ti5_1', 'molecule_name': ['Plant defensin'], 'pdb_id': '1ti5', 'e_value': 0.0053, 'percentage_identity': 41.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7k18_2', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '7k18', 'e_value': 0.0084, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bmr_1', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '1bmr', 'e_value': 0.0085, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fh3_1', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '1fh3', 'e_value': 0.0085, 'percentage_identity': 35.9, 'result_sequence': None}], 'VFRLKKWMQKVIDRFGG': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5to3_2', 'molecule_name': ['Thrombin heavy chain', 'Thrombomodulin'], 'pdb_id': '5to3', 'e_value': 4.1e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 3, 'entry_entity': '1bhx_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1bhx', 'e_value': 0.00011, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 4, 'entry_entity': '2hnt_4', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2hnt', 'e_value': 0.00011, 'percentage_identity': 87.5, 'result_sequence': None}], 'RIKRFWPVVIRTVVAGYNLYRA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 4.3e-09, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hfr_1', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '2hfr', 'e_value': 0.00099, 'percentage_identity': 65.2, 'result_sequence': None}], 'GIGGALLSVGKLALKGLANVLADKFAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 1.4e-07, 'percentage_identity': 77.8, 'result_sequence': None}], 'FFAAIFAPIVHVGKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 2.8e-08, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 9.6e-07, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 9.6e-07, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 9.9e-07, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 9.9e-07, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 9.9e-07, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 7.7e-06, 'percentage_identity': 77.3, 'result_sequence': None}], 'GIPCAESCVWIPCTVTALIGCGCSNKVCYN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 3.7e-13, 'percentage_identity': 92.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 1.4e-11, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 2.8e-10, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 2.8e-10, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 1.4e-09, 'percentage_identity': 65.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 3.5e-09, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 6.7e-09, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 6.9e-09, 'percentage_identity': 62.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 1.1e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 1.7e-07, 'percentage_identity': 65.4, 'result_sequence': None}], 'GLPVCGETCFGGTCNTPGCACDPWPVCTRD': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 1.1e-11, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 5.4e-11, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 5.4e-11, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 8.1e-11, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 8.1e-11, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 1.5e-10, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 2.6e-10, 'percentage_identity': 80.0, 'result_sequence': None}], 'KWKLFKKIEKVGQNIRDGIIKAGPAVAVVGQATQIAK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 3e-14, 'percentage_identity': 78.4, 'result_sequence': None}], 'KKTWWKTWWTKWKK': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6vpd_1', 'molecule_name': ['Glutathione peroxidase'], 'pdb_id': '6vpd', 'e_value': 0.0034, 'percentage_identity': 64.3, 'result_sequence': None}], 'CSGIIKQTCTCYRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 0.0011, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 0.052, 'percentage_identity': 66.7, 'result_sequence': None}], 'DTHFPICIFCCGCCHRSKCGMCCKT': [{'chain_id': ['R', 'U', 'P', 'Q', 'T', 'S'], 'entity_id': 1, 'entry_entity': '4qae_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '4qae', 'e_value': 1.6e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3h0t_3', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '3h0t', 'e_value': 1.6e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kef_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '2kef', 'e_value': 1.6e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1m4f_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '1m4f', 'e_value': 1.6e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6wbv_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '6wbv', 'e_value': 1.6e-13, 'percentage_identity': 100.0, 'result_sequence': None}], 'GIGTKFLGGVKTALKGALKELASTYVN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 5e-07, 'percentage_identity': 74.1, 'result_sequence': None}], 'RHRHCFSQSHKFVGACLRESNCENVCKTEGFPSGECKWHGIVSKCHCKRIC': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6lcq_1', 'molecule_name': ['Defensin-like protein CAL1'], 'pdb_id': '6lcq', 'e_value': 6.5e-15, 'percentage_identity': 61.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 2.8e-11, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 3.3e-11, 'percentage_identity': 59.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nce_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '5nce', 'e_value': 4.8e-10, 'percentage_identity': 49.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 9.7e-10, 'percentage_identity': 51.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 1.1e-07, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gps_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gps', 'e_value': 6.2e-05, 'percentage_identity': 38.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gpt_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gpt', 'e_value': 7.5e-05, 'percentage_identity': 38.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6ohx_1', 'molecule_name': ['Venom toxin Hj1a'], 'pdb_id': '6ohx', 'e_value': 0.0009, 'percentage_identity': 37.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ksk_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2ksk', 'e_value': 0.0011, 'percentage_identity': 28.6, 'result_sequence': None}], 'WCRRLCYKQRCVTYCRGR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 0.048, 'percentage_identity': 52.9, 'result_sequence': None}], 'RCPGHTRQIGTSFGPRVKSCRKW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 3.9e-05, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg5_1', 'molecule_name': ['Gallinacin-2'], 'pdb_id': '2lg5', 'e_value': 0.039, 'percentage_identity': 50.0, 'result_sequence': None}], 'TDTNVIGECFDEWSRCHRQTRWWTKILFQSCENRCKCKVQLMGNCIKVPFKCFLWKQKRFMCECYGPISGTKPWYCGWEL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k35_1', 'molecule_name': ['Hydramacin-1'], 'pdb_id': '2k35', 'e_value': 1.3e-06, 'percentage_identity': 36.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln8_1', 'molecule_name': ['Theromacin'], 'pdb_id': '2ln8', 'e_value': 0.00029, 'percentage_identity': 34.5, 'result_sequence': None}], 'PRPPHPPRPPHPPRPPHPRL': [{'chain_id': ['D', 'C', 'E'], 'entity_id': 2, 'entry_entity': '5ou9_2', 'molecule_name': ['Collagen alpha-1(I) chain'], 'pdb_id': '5ou9', 'e_value': 0.033, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3ipn_1', 'molecule_name': ['Non-natural Collagen'], 'pdb_id': '3ipn', 'e_value': 0.033, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4z1r_1', 'molecule_name': ['Collagen-like peptide'], 'pdb_id': '4z1r', 'e_value': 0.034, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1cag_1', 'molecule_name': ['COLLAGEN-LIKE PEPTIDE'], 'pdb_id': '1cag', 'e_value': 0.034, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1cgd_1', 'molecule_name': ['COLLAGEN-LIKE PEPTIDE'], 'pdb_id': '1cgd', 'e_value': 0.034, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2cuo_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '2cuo', 'e_value': 0.038, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3a08_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3a08', 'e_value': 0.038, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3ah9_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '3ah9', 'e_value': 0.038, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3a0a_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3a0a', 'e_value': 0.038, 'percentage_identity': 66.7, 'result_sequence': None}], 'GIGKFLKKAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.0035, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.012, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.012, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.012, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.012, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.012, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.012, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.013, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.014, 'percentage_identity': 83.3, 'result_sequence': None}], 'STLHAVLRLRGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2jwz', 'e_value': 0.0019, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kna_1', 'molecule_name': ['viral Ubiquitin'], 'pdb_id': '6kna', 'e_value': 0.0038, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mws_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mws', 'e_value': 0.0055, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5jtj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtj', 'e_value': 0.0055, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3w', 'e_value': 0.0055, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbe', 'e_value': 0.0055, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nr2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nr2', 'e_value': 0.0055, 'percentage_identity': 91.7, 'result_sequence': None}], 'GRLKKLGKKIEGAGKRVFKAAEKALPVVAGVKALG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmm_1', 'molecule_name': ['AAEL000598-PA'], 'pdb_id': '2mmm', 'e_value': 2.7e-12, 'percentage_identity': 82.9, 'result_sequence': None}], 'GRKKRRQRRRGGWMWVTNLRTD': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m51_1', 'molecule_name': ['Ras GTPase-activating protein 1'], 'pdb_id': '2m51', 'e_value': 0.084, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4fss_1', 'molecule_name': ['Ras GTPase-activating protein 1'], 'pdb_id': '4fss', 'e_value': 0.084, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2j06_1', 'molecule_name': ['Ras GTPase-activating protein 1'], 'pdb_id': '2j06', 'e_value': 0.085, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2j05_1', 'molecule_name': ['Ras GTPase-activating protein 1'], 'pdb_id': '2j05', 'e_value': 0.085, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gqi_1', 'molecule_name': ['Ras GTPase-activating protein 1'], 'pdb_id': '2gqi', 'e_value': 0.088, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLPCGESCVFIPCITTVVGCSCKNKVCYNN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 2.3e-12, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 2.3e-12, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 7e-12, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 9.1e-12, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 1.1e-11, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 1.4e-11, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 1.7e-10, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 2.5e-08, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 2.5e-08, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 7.9e-08, 'percentage_identity': 71.4, 'result_sequence': None}], 'FTPVLQADFQKVVAGVANALAHRYH': [{'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3pia_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3pia', 'e_value': 2.1e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2qss_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2qss', 'e_value': 2.1e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3pi9_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3pi9', 'e_value': 2.1e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2qsp_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '2qsp', 'e_value': 2.1e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3ciu_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '3ciu', 'e_value': 2.1e-11, 'percentage_identity': 100.0, 'result_sequence': None}], 'PIPRPLPFPRPGPRPIPRPLPFPRPGPRPIPRP': [{'chain_id': ['a0'], 'entity_id': 1, 'entry_entity': '6ncl_1', 'molecule_name': ['P14'], 'pdb_id': '6ncl', 'e_value': 6.3e-06, 'percentage_identity': 51.5, 'result_sequence': None}], 'GEYCGESCYLIPCFTPGCYCVSRQCVNKN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 0.00083, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dhr_1', 'molecule_name': ['Rivi3'], 'pdb_id': '6dhr', 'e_value': 0.0013, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 0.002, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 0.0025, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 0.0025, 'percentage_identity': 51.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1yp8_1', 'molecule_name': ['Tricyclon-A'], 'pdb_id': '1yp8', 'e_value': 0.0033, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 0.004, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.0076, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 0.0077, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 0.0095, 'percentage_identity': 50.0, 'result_sequence': None}], 'AIRCLAKRRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 0.019, 'percentage_identity': 80.0, 'result_sequence': None}], 'GIGAVLWVLTTGLPALISWIKRKRQQ': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 5.3e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 5.3e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 5.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 5.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 5.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 5.4e-10, 'percentage_identity': 96.2, 'result_sequence': None}], 'RTCESQSNTFPGICITKPPCRKACISEKFTDGHCSKILRRCLCTKPC': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aaz_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4aaz', 'e_value': 1.5e-21, 'percentage_identity': 91.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ab0_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4ab0', 'e_value': 1.5e-21, 'percentage_identity': 91.5, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4cqk_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4cqk', 'e_value': 1.5e-21, 'percentage_identity': 91.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mr4_1', 'molecule_name': ['Defensin-like protein'], 'pdb_id': '1mr4', 'e_value': 1.5e-21, 'percentage_identity': 91.5, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'T', 'N', 'O', 'E', 'F', 'G', 'L', 'S'], 'entity_id': 1, 'entry_entity': '6b55_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '6b55', 'e_value': 1.5e-21, 'percentage_identity': 91.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5kk4_1', 'molecule_name': ['Defensin NsD7'], 'pdb_id': '5kk4', 'e_value': 1.8e-20, 'percentage_identity': 87.2, 'result_sequence': None}, {'chain_id': ['V', 'W', 'H', 'R', 'U', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'T', 'X', 'N', 'O', 'E', 'F', 'G', 'L', 'S'], 'entity_id': 1, 'entry_entity': '5vyp_1', 'molecule_name': ['Defensin NsD7'], 'pdb_id': '5vyp', 'e_value': 1.8e-20, 'percentage_identity': 87.2, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '6mry_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6mry', 'e_value': 3.7e-18, 'percentage_identity': 78.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4uj0_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '4uj0', 'e_value': 1.1e-13, 'percentage_identity': 64.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1n4n_1', 'molecule_name': ['Floral defensin-like protein 1'], 'pdb_id': '1n4n', 'e_value': 7.5e-12, 'percentage_identity': 58.7, 'result_sequence': None}], 'GFGCPNNYQCHRHCKSIPGRCGGYCGGWHRLRCTCYRC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 2.7e-21, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 1e-08, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 4.8e-06, 'percentage_identity': 52.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 1.1e-05, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 1.2e-05, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 1.4e-05, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 2.6e-05, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 2.6e-05, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 2.6e-05, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 8.2e-05, 'percentage_identity': 51.2, 'result_sequence': None}], 'DKLIGSCVWGAVNYTSDCAAECLLRGYKGGHCGSFANVNCWCRT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 7.9e-22, 'percentage_identity': 93.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 2.1e-20, 'percentage_identity': 88.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 2.6e-20, 'percentage_identity': 88.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 3.2e-20, 'percentage_identity': 88.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 1.1e-19, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.0026, 'percentage_identity': 47.4, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.003, 'percentage_identity': 51.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.0036, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.0036, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.0082, 'percentage_identity': 53.1, 'result_sequence': None}], 'RLPRPVYIPQPRPPHPRL': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.089, 'percentage_identity': 71.4, 'result_sequence': None}], 'KWKLFKKIGAVLKVL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.038, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.038, 'percentage_identity': 100.0, 'result_sequence': None}]}


In [19]:
len(res_35h)

167

In [20]:
%store res_35h

Stored 'res_35h' (dict)


## 3500 - 3750

In [16]:
sec_3750=sec_1[3500:3750]

In [18]:
res_3750_raw=multiple_sequence_search(sec_3750)

1 sequences of 250
2 sequences of 250
3 sequences of 250
4 sequences of 250
5 sequences of 250
6 sequences of 250
7 sequences of 250
8 sequences of 250
9 sequences of 250
10 sequences of 250
11 sequences of 250
12 sequences of 250
13 sequences of 250
14 sequences of 250
15 sequences of 250
16 sequences of 250
17 sequences of 250
18 sequences of 250
19 sequences of 250
20 sequences of 250
21 sequences of 250
22 sequences of 250
23 sequences of 250
24 sequences of 250
25 sequences of 250
26 sequences of 250
27 sequences of 250
28 sequences of 250
29 sequences of 250
30 sequences of 250
31 sequences of 250
32 sequences of 250
33 sequences of 250
34 sequences of 250
35 sequences of 250
36 sequences of 250
37 sequences of 250
38 sequences of 250
39 sequences of 250
40 sequences of 250
41 sequences of 250
42 sequences of 250
43 sequences of 250
44 sequences of 250
45 sequences of 250
46 sequences of 250
47 sequences of 250
48 sequences of 250
49 sequences of 250
50 sequences of 250
51 sequen

In [20]:
res_3750=results_to_dict(res_3750_raw)

In [23]:
%store res_3750 res_3750_raw

Stored 'res_3750' (dict)
Stored 'res_3750_raw' (dict)


In [63]:
len(res_3750)

77

## 3750 - 4000

In [24]:
sec_4k=sec_1[3750:4000]

In [25]:
res_4k_raw=multiple_sequence_search(sec_4k)

1 sequences of 250
2 sequences of 250
3 sequences of 250
4 sequences of 250
5 sequences of 250
6 sequences of 250
7 sequences of 250
8 sequences of 250
9 sequences of 250
10 sequences of 250
11 sequences of 250
12 sequences of 250
13 sequences of 250
14 sequences of 250
15 sequences of 250
16 sequences of 250
17 sequences of 250
18 sequences of 250
19 sequences of 250
20 sequences of 250
21 sequences of 250
22 sequences of 250
23 sequences of 250
24 sequences of 250
25 sequences of 250
26 sequences of 250
27 sequences of 250
28 sequences of 250
29 sequences of 250
30 sequences of 250
31 sequences of 250
32 sequences of 250
33 sequences of 250
34 sequences of 250
35 sequences of 250
36 sequences of 250
37 sequences of 250
38 sequences of 250
39 sequences of 250
40 sequences of 250
41 sequences of 250
42 sequences of 250
43 sequences of 250
44 sequences of 250
45 sequences of 250
46 sequences of 250
47 sequences of 250
48 sequences of 250
49 sequences of 250
50 sequences of 250
51 sequen

In [26]:
res_4k=results_to_dict(res_4k_raw)

In [29]:
%store res_4k res_4k_raw

Stored 'res_4k' (dict)
Stored 'res_4k_raw' (dict)


In [30]:
len(res_4k)

94

## 4000 - 4250 (colab)

In [19]:
res_4250={'KVKVKVKVPPTKVKVKVK': [{'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '2n1e_1', 'molecule_name': ['MAX1 peptide'], 'pdb_id': '2n1e', 'e_value': 6.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'IEFEKSTESHDIQKRGVTITVKPPFPGCVFYECIANCRSRGYKNGGYCTINGCQCLR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pjv_1', 'molecule_name': ['Potassium channel toxin alpha-KTx 10.1'], 'pdb_id': '1pjv', 'e_value': 0.01, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bi5_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6bi5', 'e_value': 0.056, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6b9w_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '6b9w', 'e_value': 0.074, 'percentage_identity': 45.2, 'result_sequence': None}], 'GIGDILKNLAKAAGKAALHAVGESL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6wpo_1', 'molecule_name': ['Hylaseptin-4'], 'pdb_id': '6wpo', 'e_value': 1.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'VDKGSYLPRPTPPRPIYNRN': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezn_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '4ezn', 'e_value': 1.4e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3dpp_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpp', 'e_value': 1.4e-05, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '3dpq_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpq', 'e_value': 1.4e-05, 'percentage_identity': 85.0, 'result_sequence': None}], 'ALWKTMLKKAAHVGKHVGKAALGAAARRRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 2.5e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 2.6e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 0.00033, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 0.00055, 'percentage_identity': 60.0, 'result_sequence': None}], 'VIGSILGALASGLPTLISWIKNR': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.00092, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.00092, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.00095, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.00095, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.00095, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.00095, 'percentage_identity': 59.1, 'result_sequence': None}], 'GCPLNQGACHNHCRSIGRRGGYCAGIIKQTCTCYRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 1e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 1.7e-09, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 1.1e-07, 'percentage_identity': 61.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 9.8e-07, 'percentage_identity': 55.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 5.8e-06, 'percentage_identity': 55.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 3e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 0.00015, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 0.00034, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 0.00034, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 0.00034, 'percentage_identity': 48.8, 'result_sequence': None}], 'FKRLKKLISWIKRKRQQ': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.038, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.038, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.039, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.039, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.039, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.039, 'percentage_identity': 85.7, 'result_sequence': None}], 'MAKEFGIPAAVAKTVLNVVEAGGWVTTIVSILTAVGSGGKSLLAAAGRESIKAYLKKEIKKKGKRAVIAW': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4rgd_1', 'molecule_name': ['AS-48A'], 'pdb_id': '4rgd', 'e_value': 8.5e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o82_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o82', 'e_value': 2.2e-27, 'percentage_identity': 97.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o84_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o84', 'e_value': 2.2e-27, 'percentage_identity': 97.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o83_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o83', 'e_value': 2.2e-27, 'percentage_identity': 97.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e68_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1e68', 'e_value': 2.2e-27, 'percentage_identity': 97.1, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8j_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8j', 'e_value': 0.00022, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mp8_1', 'molecule_name': ['Nkr-5-3b'], 'pdb_id': '2mp8', 'e_value': 0.00054, 'percentage_identity': 37.1, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8p_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8p', 'e_value': 0.0022, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8r_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8r', 'e_value': 0.019, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8s_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8s', 'e_value': 0.019, 'percentage_identity': 64.0, 'result_sequence': None}], 'GGLKKLGKKLEGAGKRVFNAAEKALP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmm_1', 'molecule_name': ['AAEL000598-PA'], 'pdb_id': '2mmm', 'e_value': 2e-10, 'percentage_identity': 92.3, 'result_sequence': None}], 'GLNALKKVFQGIHEAIKLINNHVK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncx_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncx', 'e_value': 5.5e-11, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncy_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncy', 'e_value': 9.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}], 'RWKIFKKIERVGQNVRDGIIKAGKAIQVLGTAKALGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 9.4e-11, 'percentage_identity': 70.3, 'result_sequence': None}], 'GFGVLAKVAAHVVPAIAEHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmj_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mmj', 'e_value': 9.1e-06, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn8_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn8', 'e_value': 9.1e-06, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn9_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn9', 'e_value': 9.1e-06, 'percentage_identity': 89.5, 'result_sequence': None}], 'APKAMHSAKKFGKAFVGEIMLQKKGI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 4.9e-05, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 5e-05, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.00025, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 0.011, 'percentage_identity': 72.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 0.02, 'percentage_identity': 72.2, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 0.02, 'percentage_identity': 72.2, 'result_sequence': None}], 'CAIEHMRTPLKPFSFGPPYNL': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3j32_1', 'molecule_name': ['Hemocyanin isoform 1'], 'pdb_id': '3j32', 'e_value': 4.4e-07, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1lnl_1', 'molecule_name': ['Hemocyanin type 2 unit e'], 'pdb_id': '1lnl', 'e_value': 5.6e-07, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4bed_2', 'molecule_name': ['Hemocyanin 1'], 'pdb_id': '4bed', 'e_value': 5.4e-06, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['W', 'H', 'K', 'Q', 'c', 'B', 'T', 'N', 'Z', 'E'], 'entity_id': 2, 'entry_entity': '4yd9_2', 'molecule_name': ['hemocyanin'], 'pdb_id': '4yd9', 'e_value': 7.4e-06, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['1a', '2a', '3a', '4a', '8a', '5a', '7a', '10a', '6a', '9a'], 'entity_id': 1, 'entry_entity': '6r83_1', 'molecule_name': ['Hemocyanin subunit 1'], 'pdb_id': '6r83', 'e_value': 2.1e-05, 'percentage_identity': 81.0, 'result_sequence': None}], 'GKFLKKAKKFGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.014, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.015, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.054, 'percentage_identity': 75.0, 'result_sequence': None}], 'GGARSLGRKALRAWKKYGPIIVPIIRI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fry_1', 'molecule_name': ['Cathelin-related peptide SC5'], 'pdb_id': '1fry', 'e_value': 0.0059, 'percentage_identity': 57.7, 'result_sequence': None}], 'KWCFRVCYRGICYRKCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 2.1e-07, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7ald_1', 'molecule_name': ['BRICHOS domain-containing protein'], 'pdb_id': '7ald', 'e_value': 0.0022, 'percentage_identity': 64.3, 'result_sequence': None}], 'GIGKFIHSAKKFGKLFVGEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 1e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 1.1e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 2.3e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 2.9e-07, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 5.2e-07, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 5.2e-07, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.051, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.051, 'percentage_identity': 91.7, 'result_sequence': None}], 'QPEWFKARRWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 8.1e-08, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 2.6e-06, 'percentage_identity': 84.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 2.6e-06, 'percentage_identity': 84.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 9.3e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 0.042, 'percentage_identity': 66.7, 'result_sequence': None}], 'KWKSFIKKLTSKFLHSANKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.01, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.01, 'percentage_identity': 65.0, 'result_sequence': None}], 'KRFWPLVPVAINTVAAGINLYKAIRRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hfr_1', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '2hfr', 'e_value': 1.7e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 0.00013, 'percentage_identity': 63.0, 'result_sequence': None}], 'RWCRYAYVRVRGVLVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 9.4e-12, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 1.2e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 1.2e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 8.9e-11, 'percentage_identity': 90.5, 'result_sequence': None}], 'KWKKFIKSLTKSAAKTVVKTAKKPLIV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.078, 'percentage_identity': 40.7, 'result_sequence': None}], 'THVFRLKKWMQKVIDRFGG': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5to3_2', 'molecule_name': ['Thrombin heavy chain', 'Thrombomodulin'], 'pdb_id': '5to3', 'e_value': 8.2e-07, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 3, 'entry_entity': '1bhx_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1bhx', 'e_value': 2.6e-06, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 4, 'entry_entity': '2hnt_4', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2hnt', 'e_value': 2.6e-06, 'percentage_identity': 88.9, 'result_sequence': None}], 'RFGRFLRKIRRFRPKVTITIQGSARF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gdl_1', 'molecule_name': ['myeloid antimicrobial peptide 27'], 'pdb_id': '2gdl', 'e_value': 5.5e-12, 'percentage_identity': 100.0, 'result_sequence': None}], 'FLKKAKKFGKAF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.0068, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.0072, 'percentage_identity': 83.3, 'result_sequence': None}], 'SMATPHVAGAAALILSKHPTWTNAQVRDRLESTATYLGNSFYYGK': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5gl8_1', 'molecule_name': ['Subtilisin NAT'], 'pdb_id': '5gl8', 'e_value': 5.2e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3vyv_1', 'molecule_name': ['Subtilisin NAT'], 'pdb_id': '3vyv', 'e_value': 5.2e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4dww_1', 'molecule_name': ['Subtilisin NAT'], 'pdb_id': '4dww', 'e_value': 5.2e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1scj_1', 'molecule_name': ['Subtilisin E'], 'pdb_id': '1scj', 'e_value': 1.5e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3whi_1', 'molecule_name': ['Subtilisin E'], 'pdb_id': '3whi', 'e_value': 1.6e-19, 'percentage_identity': 97.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6o44_1', 'molecule_name': ['Subtilisin E'], 'pdb_id': '6o44', 'e_value': 1.6e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6pak_1', 'molecule_name': ['Subtilisin E'], 'pdb_id': '6pak', 'e_value': 1.6e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mee_1', 'molecule_name': ['Subtilisin'], 'pdb_id': '1mee', 'e_value': 1.9e-19, 'percentage_identity': 97.8, 'result_sequence': None}], 'GLPICGETCVGGSCNTPGCSCSWPVCTRN': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttm_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttm', 'e_value': 1.5e-13, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 1.5e-13, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 4e-13, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 6e-13, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 6e-13, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 7.4e-13, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 1.7e-12, 'percentage_identity': 86.7, 'result_sequence': None}], 'SAADKGNVKAAWGKVGGHAAEYGAEALERMFLSF': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 1.1e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 1.1e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 1.1e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 1.1e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ciu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ciu', 'e_value': 1.1e-15, 'percentage_identity': 100.0, 'result_sequence': None}], 'GVVDILKGAAKDIAGHLLSKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 5.4e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 5.4e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 5.4e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 1e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 1e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 1e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 6.9e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 6.9e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 6.9e-08, 'percentage_identity': 95.5, 'result_sequence': None}], 'QIAALEQEIAALEQEIAALQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1u0i_1', 'molecule_name': ['IAAL-E3'], 'pdb_id': '1u0i', 'e_value': 0.0011, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4etp_2', 'molecule_name': ['Spindle pole body-associated protein VIK1'], 'pdb_id': '4etp', 'e_value': 0.0034, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4dzn_1', 'molecule_name': ['COILED-COIL PEPTIDE CC-PIL'], 'pdb_id': '4dzn', 'e_value': 0.0049, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6f37_1', 'molecule_name': ['Fucose-binding lectin protein'], 'pdb_id': '6f37', 'e_value': 0.0083, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '6s99_1', 'molecule_name': ['Fucose-binding lectin protein'], 'pdb_id': '6s99', 'e_value': 0.0083, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4g1a_1', 'molecule_name': ['AQ-C16C19 peptide'], 'pdb_id': '4g1a', 'e_value': 0.014, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4dzm_1', 'molecule_name': ['COILED-COIL PEPTIDE CC-DI'], 'pdb_id': '4dzm', 'e_value': 0.043, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'C', 'B', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4dzl_1', 'molecule_name': ['COILED-COIL PEPTIDE CC-TRI'], 'pdb_id': '4dzl', 'e_value': 0.057, 'percentage_identity': 60.0, 'result_sequence': None}], 'RIKRFWPVVIR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 0.018, 'percentage_identity': 72.7, 'result_sequence': None}], 'KWKLFKKIGIGAVLKVLTTGLPALKLTLK': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.001, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.033, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.033, 'percentage_identity': 85.7, 'result_sequence': None}], 'QTPKIACKNGDKNCHQSHGAVSLTMCKLTSGKYPNCRYKEKRQNKSYVVACKPPQKKDSQQFHLVPVHLDRVL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 1.4e-33, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 1.1e-15, 'percentage_identity': 58.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 1.1e-15, 'percentage_identity': 58.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 1.1e-15, 'percentage_identity': 58.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 1.1e-15, 'percentage_identity': 58.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 1.1e-15, 'percentage_identity': 58.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2apq_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2apq', 'e_value': 3.3e-10, 'percentage_identity': 46.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nui_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2nui', 'e_value': 5.4e-09, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1fev_2', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '1fev', 'e_value': 7.6e-09, 'percentage_identity': 46.3, 'result_sequence': None}], 'VTLASHLPSDFTPAVHASLDKF': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 1.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 1.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 1.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 1.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ciu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ciu', 'e_value': 1.3e-09, 'percentage_identity': 100.0, 'result_sequence': None}], 'KWKLFKKIPKFLHL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.018, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.018, 'percentage_identity': 80.0, 'result_sequence': None}], 'VFQFLGKIIHHVGNFVHGFSAVF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6c41_1', 'molecule_name': ['Clavanin-A'], 'pdb_id': '6c41', 'e_value': 3e-11, 'percentage_identity': 95.7, 'result_sequence': None}], 'GSKKPVPIIYCQRRSGKCQRM': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xo9_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo9', 'e_value': 2.5e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo4_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo4', 'e_value': 2.5e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gd5_2', 'molecule_name': ['Thanatin'], 'pdb_id': '6gd5', 'e_value': 2.5e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6aab_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6aab', 'e_value': 2.5e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8tfv_1', 'molecule_name': ['Thanatin'], 'pdb_id': '8tfv', 'e_value': 2.5e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo3_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo3', 'e_value': 1.6e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xoa_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xoa', 'e_value': 1.6e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6afq_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6afq', 'e_value': 1.6e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xok_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xok', 'e_value': 2.2e-07, 'percentage_identity': 85.0, 'result_sequence': None}], 'GLPVCGETCFGGTCNTPGCTCDPWPVCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 1.2e-12, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 5.7e-12, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 5.7e-12, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 1.5e-11, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 2.3e-11, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 2.3e-11, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 2.8e-11, 'percentage_identity': 86.7, 'result_sequence': None}], 'VGPGGECGGRFGGCAGGQCCSRFGFCGSGPKYCAH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kus_1', 'molecule_name': ['Sm-AMP-1.1a'], 'pdb_id': '2kus', 'e_value': 2.5e-12, 'percentage_identity': 70.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 3.6e-07, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 1.2e-06, 'percentage_identity': 65.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 1.8e-06, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mmc_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1mmc', 'e_value': 2.7e-06, 'percentage_identity': 62.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 7.4e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zuv_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zuv', 'e_value': 1.1e-05, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zwu_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zwu', 'e_value': 2.4e-05, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mpi_1', 'molecule_name': ['Probable inactive chitinase-like protein LaCIC'], 'pdb_id': '4mpi', 'e_value': 7.2e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aml_1', 'molecule_name': ['Agglutinin isolectin 1'], 'pdb_id': '4aml', 'e_value': 0.00016, 'percentage_identity': 51.7, 'result_sequence': None}], 'GLNALKKVFQPIHKAIKKINNHVQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncy_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncy', 'e_value': 1.6e-09, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncx_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncx', 'e_value': 4.2e-08, 'percentage_identity': 87.5, 'result_sequence': None}], 'RWCVYARVRGVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 1.9e-06, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 1.9e-06, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 2.5e-06, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 2.5e-06, 'percentage_identity': 76.2, 'result_sequence': None}], 'RWCVYARVRGVRYRRCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 9.1e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 9.1e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 0.00012, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 0.00012, 'percentage_identity': 75.0, 'result_sequence': None}], 'RKLPDAPGMHTWGGGGRCAVLSCLPKEQI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.011, 'percentage_identity': 92.9, 'result_sequence': None}], 'RVKRFKKFFRKIKKGFRKIFKKTKIFIG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.011, 'percentage_identity': 56.7, 'result_sequence': None}], 'KAKKFGKAFVKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.074, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.078, 'percentage_identity': 100.0, 'result_sequence': None}], 'AISYGNGVYCNKEKCWVNKAENKQAITGIVIGGWASSLAGMGH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2a2b_1', 'molecule_name': ['Bacteriocin curvacin-A'], 'pdb_id': '2a2b', 'e_value': 2e-11, 'percentage_identity': 68.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw5_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1cw5', 'e_value': 0.046, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ry3_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1ry3', 'e_value': 0.058, 'percentage_identity': 40.0, 'result_sequence': None}], 'MAKEFGIPAAVAGTVLNVVEAGGWVTTIVSILTAVGSGGLSLLAAAGRESIKSYLKKEIKKKGKRAVIAW': [{'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o82_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o82', 'e_value': 2.6e-27, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o84_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o84', 'e_value': 2.6e-27, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o83_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o83', 'e_value': 2.6e-27, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e68_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1e68', 'e_value': 2.6e-27, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4rgd_1', 'molecule_name': ['AS-48A'], 'pdb_id': '4rgd', 'e_value': 7.2e-26, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8j_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8j', 'e_value': 0.00041, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8p_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8p', 'e_value': 0.0037, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mp8_1', 'molecule_name': ['Nkr-5-3b'], 'pdb_id': '2mp8', 'e_value': 0.01, 'percentage_identity': 34.3, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8r_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8r', 'e_value': 0.029, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8s_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8s', 'e_value': 0.029, 'percentage_identity': 64.0, 'result_sequence': None}], 'FKCWRWQWRWKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 8.3e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0023, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.013, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.013, 'percentage_identity': 80.0, 'result_sequence': None}], 'GFCWNVCVYRNGVRVCHRRCN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5y0i_1', 'molecule_name': ['NZ17074(N1)'], 'pdb_id': '5y0i', 'e_value': 4.2e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5y0j_1', 'molecule_name': ['N2'], 'pdb_id': '5y0j', 'e_value': 2.9e-11, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5v0y_1', 'molecule_name': ['Arenicin-3'], 'pdb_id': '5v0y', 'e_value': 3.7e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5y0h_1', 'molecule_name': ['N6'], 'pdb_id': '5y0h', 'e_value': 6.8e-10, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k4v_1', 'molecule_name': ['smart chimeric peptide G6'], 'pdb_id': '6k4v', 'e_value': 9.2e-10, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k4w_1', 'molecule_name': ['SCP-A6'], 'pdb_id': '6k4w', 'e_value': 9.8e-10, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5v11_1', 'molecule_name': ['AA139'], 'pdb_id': '5v11', 'e_value': 5.6e-09, 'percentage_identity': 76.2, 'result_sequence': None}], 'GVVDILKGAAKDIAAHLASKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 2.3e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 2.3e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 2.3e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 4.9e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 4.9e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 4.9e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 3.4e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 3.4e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 3.4e-08, 'percentage_identity': 95.5, 'result_sequence': None}], 'GFKEVLKADLGSLVKGIAAHVAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l5r_1', 'molecule_name': ['Antimicrobial peptide Alyteserin-1C'], 'pdb_id': '2l5r', 'e_value': 8.5e-07, 'percentage_identity': 78.3, 'result_sequence': None}], 'KTLTGKTITLE': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1gjz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '1gjz', 'e_value': 0.0073, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2msg_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2msg', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '5gvi_3', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5gvi', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F', 'E', 'G'], 'entity_id': 1, 'entry_entity': '3ldz_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ldz', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3ons_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3ons', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}], 'KFFRKLKKSVKKRAKKFFKKPRVIGVSIPF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 1.6e-09, 'percentage_identity': 80.0, 'result_sequence': None}], 'LCAIAKKKGKCKGPLKLVCKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mbd_1', 'molecule_name': ['Lasiocepsin'], 'pdb_id': '2mbd', 'e_value': 3.7e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'GFGCPFNANECHAHCLSIGRKFGFCAGPLRATCTCGKQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 8.1e-21, 'percentage_identity': 97.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 1e-09, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 3.2e-08, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 4.6e-07, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 9.3e-07, 'percentage_identity': 52.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 2e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 0.00074, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 0.0022, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 0.0022, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 0.0022, 'percentage_identity': 46.3, 'result_sequence': None}], 'LVKRFKKFFRKLKKSVKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.00086, 'percentage_identity': 87.5, 'result_sequence': None}], 'AQWFAIQHISLNPPRSTIAMRAINNYRWRSKNQNTFLR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 1.8e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 1.8e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 1.8e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 1.8e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 1.8e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 1.8e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 1.8e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 1.8e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 1.8e-19, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 1.8e-19, 'percentage_identity': 94.7, 'result_sequence': None}], 'ATCDLLSGTGVGHSACAAHCLLRGNRGGYCNGKGVCVCRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 3.4e-20, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 1.9e-19, 'percentage_identity': 92.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 5.4e-19, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 8.7e-14, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 7.7e-12, 'percentage_identity': 67.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 7.9e-12, 'percentage_identity': 67.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 2.8e-11, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 6.2e-09, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 6.6e-08, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 3.7e-07, 'percentage_identity': 52.3, 'result_sequence': None}], 'SPIHACRYQRGVCIPGPCRWPYYRVGSCGSGLKSCCVRNRWA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 6.4e-10, 'percentage_identity': 52.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.0015, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 0.046, 'percentage_identity': 40.5, 'result_sequence': None}], 'MTESLVLSPAPAKPKRVKASRRSASHPTYSEMIAAAIRAEKSRGGSSRQSIQKYIKSHYKVGHNADLQIKLSIRRLLAAGVLKQTKGVGASGSFRLAKSDKAKRSPGKKKKAVRRSTSPKKAARPRKARSPAKKPKATARKARKKSRASPKKAKKPKTVKAKSRKASKAKKVKRSKPRAKSGARKSPKKK': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6n89_2', 'molecule_name': ['Histone H1.0'], 'pdb_id': '6n89', 'e_value': 1e-40, 'percentage_identity': 66.7, 'result_sequence': None}], 'MEKKSFAGLCFLFLVLFVAQECVLQTEAKTCENLADTFRGPCFATGNCDDHCKNKEHLLRGRCRDDFRCWCTRNC': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 2.2e-22, 'percentage_identity': 89.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 2.5e-21, 'percentage_identity': 87.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 3.6e-21, 'percentage_identity': 85.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 2.1e-19, 'percentage_identity': 78.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 1.8e-17, 'percentage_identity': 72.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jkz_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '1jkz', 'e_value': 2.7e-13, 'percentage_identity': 61.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 3.4e-06, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 8.7e-06, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 1.5e-05, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2q_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '2n2q', 'e_value': 0.00033, 'percentage_identity': 33.3, 'result_sequence': None}], 'KWFKRLEKLFSKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 0.0085, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0g_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0g', 'e_value': 0.0085, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 0.0085, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 0.0085, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 0.0085, 'percentage_identity': 92.3, 'result_sequence': None}], 'GIGGALLSAGKSALKGLAKGLAEHL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 6e-06, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.028, 'percentage_identity': 68.2, 'result_sequence': None}], 'GIGGALLSAGKSALKGLAKGLAEHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 2e-06, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.021, 'percentage_identity': 60.0, 'result_sequence': None}], 'MPKWKVFKKIEKVGRNIRNGIVKAGPAIAVLGEAKALG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 8.3e-12, 'percentage_identity': 75.0, 'result_sequence': None}], 'GFIATLCTKVLDFGIDKLIQLIEDK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1of9_1', 'molecule_name': ['Pore-forming peptide ameobapore A'], 'pdb_id': '1of9', 'e_value': 2.5e-11, 'percentage_identity': 100.0, 'result_sequence': None}], 'AKKVFKRLEKLFSKIFNFK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 6e-06, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 2.6e-05, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 2.6e-05, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 3.5e-05, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0g_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0g', 'e_value': 0.00015, 'percentage_identity': 89.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5k_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5k', 'e_value': 0.0016, 'percentage_identity': 84.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5l', 'e_value': 0.0038, 'percentage_identity': 84.2, 'result_sequence': None}], 'GLPRKILAAIAKKKGKCKGPLKLVAKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mbd_1', 'molecule_name': ['Lasiocepsin'], 'pdb_id': '2mbd', 'e_value': 3.3e-11, 'percentage_identity': 92.6, 'result_sequence': None}], 'TKPKGTKPKGTKPKGTKPKG': [{'chain_id': ['U'], 'entity_id': 8, 'entry_entity': '7k5y_8', 'molecule_name': ['Histone H1.4'], 'pdb_id': '7k5y', 'e_value': 0.059, 'percentage_identity': 66.7, 'result_sequence': None}], 'GIMRVFKGVLKTAGKSVAKNVAGSFLDRLKCKISGGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 8e-05, 'percentage_identity': 54.1, 'result_sequence': None}], 'PPYNLNSHTQEYSKPEDTFD': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3j32_1', 'molecule_name': ['Hemocyanin isoform 1'], 'pdb_id': '3j32', 'e_value': 2.7e-07, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4bed_2', 'molecule_name': ['Hemocyanin 1'], 'pdb_id': '4bed', 'e_value': 1.2e-05, 'percentage_identity': 78.9, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1lnl_1', 'molecule_name': ['Hemocyanin type 2 unit e'], 'pdb_id': '1lnl', 'e_value': 6.2e-05, 'percentage_identity': 78.9, 'result_sequence': None}, {'chain_id': ['W', 'H', 'K', 'Q', 'c', 'B', 'T', 'N', 'Z', 'E'], 'entity_id': 2, 'entry_entity': '4yd9_2', 'molecule_name': ['hemocyanin'], 'pdb_id': '4yd9', 'e_value': 0.072, 'percentage_identity': 63.2, 'result_sequence': None}], 'RWKIFKKIEKVGRNVRDGIIKAGPAVAVVGQAATVVKG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 1.1e-16, 'percentage_identity': 100.0, 'result_sequence': None}], 'GIGRKFLGGVKTTFRGGVKDFASKHLY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.01, 'percentage_identity': 48.1, 'result_sequence': None}], 'LLGDFFRKVKEKIGKEFKRIVQR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 2.2e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 2.8e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 2.8e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 2.8e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 2.8e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 2.8e-08, 'percentage_identity': 95.7, 'result_sequence': None}], 'RRIRPRPPRLPRPRPRP': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 3.9e-05, 'percentage_identity': 88.2, 'result_sequence': None}], 'KRFKKFFRKLKKSVKKRKKEFKKKPRVIGVSIPF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 1.6e-09, 'percentage_identity': 76.5, 'result_sequence': None}], 'GIMDTIKGAAKDLAGQLLDKLKCKITKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 4.3e-06, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 0.003, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 0.003, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 0.003, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 0.008, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 0.008, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 0.008, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 0.0083, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 0.0083, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 0.0083, 'percentage_identity': 57.1, 'result_sequence': None}], 'FKRIVQRIKDFLRGIGKFLHSAKKF': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.017, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.022, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.022, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.022, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.022, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.022, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.033, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.033, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.074, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.074, 'percentage_identity': 100.0, 'result_sequence': None}], 'LVKRFKKFFRKLKKS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.011, 'percentage_identity': 84.6, 'result_sequence': None}], 'RRIRPRPPRLPRPRPRPL': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 5.5e-05, 'percentage_identity': 88.2, 'result_sequence': None}], 'DRCSQQCQHHRDPDRKQQCMRECRRH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m6a_1', 'molecule_name': ['Predicted protein'], 'pdb_id': '2m6a', 'e_value': 0.00067, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lm0_1', 'molecule_name': ['L-2'], 'pdb_id': '5lm0', 'e_value': 0.015, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l2r_1', 'molecule_name': ['Antimicrobial peptide EcAMP1'], 'pdb_id': '2l2r', 'e_value': 0.069, 'percentage_identity': 44.0, 'result_sequence': None}], 'KRGKQGGKVRAKAKTRSS': [{'chain_id': ['C', 'G'], 'entity_id': 4, 'entry_entity': '6kvd_4', 'molecule_name': ['Histone H2A.J'], 'pdb_id': '6kvd', 'e_value': 2.3e-05, 'percentage_identity': 94.1, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3kuy_3', 'molecule_name': ['Histone H2A type 1'], 'pdb_id': '3kuy', 'e_value': 4e-05, 'percentage_identity': 94.1, 'result_sequence': None}], 'TAEDLARYYSALRHYINLITRQRY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f8p_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1f8p', 'e_value': 2.8e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ron_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1ron', 'e_value': 4.8e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fvn_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1fvn', 'e_value': 5.8e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1icy_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1icy', 'e_value': 7.6e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oon_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oon', 'e_value': 6.9e-08, 'percentage_identity': 70.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rlk_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2rlk', 'e_value': 3.6e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qbf_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1qbf', 'e_value': 3.6e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ru5_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ru5', 'e_value': 3.6e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oop_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oop', 'e_value': 3.6e-07, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ruu_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ruu', 'e_value': 3.6e-07, 'percentage_identity': 66.7, 'result_sequence': None}], 'GSKKPVPIIYCQRRTGKCQRM': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xo9_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo9', 'e_value': 1e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo4_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo4', 'e_value': 1e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gd5_2', 'molecule_name': ['Thanatin'], 'pdb_id': '6gd5', 'e_value': 1e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6aab_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6aab', 'e_value': 1e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8tfv_1', 'molecule_name': ['Thanatin'], 'pdb_id': '8tfv', 'e_value': 1e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo3_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo3', 'e_value': 6.4e-09, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xoa_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xoa', 'e_value': 6.4e-09, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6afq_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6afq', 'e_value': 6.4e-09, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xok_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xok', 'e_value': 9.1e-08, 'percentage_identity': 90.0, 'result_sequence': None}], 'GLNALKKVFQGIHEAIKLINKHVQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncx_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncx', 'e_value': 1.8e-10, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ncy_1', 'molecule_name': ['Pseudin-2'], 'pdb_id': '2ncy', 'e_value': 3e-09, 'percentage_identity': 91.7, 'result_sequence': None}], 'GIGGALLSVGKSALKGLTKGLAEHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 1.9e-05, 'percentage_identity': 72.0, 'result_sequence': None}], 'YRQSMNNFQGLRSFGCRFGTCTVQKLAHQIYQFTDKDKDNVAPRSKISPQGY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l7s_1', 'molecule_name': ['Adrenomedullin'], 'pdb_id': '2l7s', 'e_value': 2.2e-26, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 6, 'entry_entity': '6uun_6', 'molecule_name': ['Adrenomedullin'], 'pdb_id': '6uun', 'e_value': 2.2e-26, 'percentage_identity': 100.0, 'result_sequence': None}], 'SGSLSTFFRLFNRSFTQALGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2i2j_1', 'molecule_name': ['Competence stimulating protein'], 'pdb_id': '2i2j', 'e_value': 4.3e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'GMWSKIKNAGKAAKAAAKAAGKAALGAVSEAM': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k9b_1', 'molecule_name': ['Dermaseptin-DI1'], 'pdb_id': '2k9b', 'e_value': 8.4e-09, 'percentage_identity': 84.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jx6_1', 'molecule_name': ['Dermaseptin-DI1'], 'pdb_id': '2jx6', 'e_value': 8.4e-09, 'percentage_identity': 84.8, 'result_sequence': None}], 'AQCGAQGGGQTCPGGLCCSQWGWCGSTPKYCGAGCQSNCR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mpi_1', 'molecule_name': ['Probable inactive chitinase-like protein LaCIC'], 'pdb_id': '4mpi', 'e_value': 1.6e-13, 'percentage_identity': 73.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4wp4_1', 'molecule_name': ['Hevein'], 'pdb_id': '4wp4', 'e_value': 6.8e-12, 'percentage_identity': 65.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q9b_1', 'molecule_name': ['Hevein'], 'pdb_id': '1q9b', 'e_value': 6.8e-12, 'percentage_identity': 65.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hev_1', 'molecule_name': ['Hevein'], 'pdb_id': '1hev', 'e_value': 6.8e-12, 'percentage_identity': 65.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wkx_1', 'molecule_name': ['Hevein'], 'pdb_id': '1wkx', 'e_value': 8.1e-12, 'percentage_identity': 65.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3iwr_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '3iwr', 'e_value': 9.5e-12, 'percentage_identity': 68.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dkv_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '2dkv', 'e_value': 9.5e-12, 'percentage_identity': 68.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 3.9e-11, 'percentage_identity': 71.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uln_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uln', 'e_value': 1.5e-10, 'percentage_identity': 60.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uha_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uha', 'e_value': 1.5e-10, 'percentage_identity': 60.5, 'result_sequence': None}], 'KKWGWLAWVDPAYEFIKGFGKGAIKEGNKDKWKNI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpk_1', 'molecule_name': ['Bacteriocin lactococcin-G subunit beta'], 'pdb_id': '2jpk', 'e_value': 8.8e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpm_1', 'molecule_name': ['Bacteriocin lactococcin-G subunit beta'], 'pdb_id': '2jpm', 'e_value': 8.8e-19, 'percentage_identity': 100.0, 'result_sequence': None}], 'VFHLLGKIIHHVGNFVYGFSHVF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6c41_1', 'molecule_name': ['Clavanin-A'], 'pdb_id': '6c41', 'e_value': 4.5e-10, 'percentage_identity': 87.0, 'result_sequence': None}], 'DAEFRHDSGYEVHHQKLVFFAEDVGSNKGAIIGLMVGGVV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9r_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2m9r', 'e_value': 1.7e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9s_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2m9s', 'e_value': 1.7e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'G'], 'entity_id': 4, 'entry_entity': '4m1c_4', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '4m1c', 'e_value': 1.7e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'A', 'B', 'C', 'J', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2lmo_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2lmo', 'e_value': 1.7e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '2lmp_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2lmp', 'e_value': 1.7e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'I', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '2m4j_1', 'molecule_name': ['Amyloid-beta protein 40'], 'pdb_id': '2m4j', 'e_value': 1.7e-20, 'percentage_identity': 100.0, 'result_sequence': None}]}

In [22]:
%store res_4250

Stored 'res_4250' (dict)


In [62]:
len(res_4250)

95

## 4250 - 4500 (colab)

In [31]:
res_45h={'GIMNTVKDVATGVATHLLNMVKCKITGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.00028, 'percentage_identity': 53.1, 'result_sequence': None}], 'KKLKLAPAKLALLWKALALKLKKA': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l96_1', 'molecule_name': ['LAK160-P7'], 'pdb_id': '2l96', 'e_value': 5.9e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l99_1', 'molecule_name': ['LAK160-P10'], 'pdb_id': '2l99', 'e_value': 5.5e-08, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l9a_1', 'molecule_name': ['LAK160-P12'], 'pdb_id': '2l9a', 'e_value': 5.5e-08, 'percentage_identity': 91.7, 'result_sequence': None}], 'KKLFKKILKYLGIGKFLHSAK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.0037, 'percentage_identity': 80.0, 'result_sequence': None}], 'FLPVIAGIAANVLPKLFCKLTKRC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 1.3e-05, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 1.3e-05, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 1.3e-05, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4x_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4x', 'e_value': 0.023, 'percentage_identity': 52.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4v_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4v', 'e_value': 0.023, 'percentage_identity': 52.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r95_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r95', 'e_value': 0.023, 'percentage_identity': 52.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r96_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r96', 'e_value': 0.023, 'percentage_identity': 52.2, 'result_sequence': None}], 'AVLSWLPKEEQIGKWSTRGRKWWRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 8.5e-06, 'percentage_identity': 85.2, 'result_sequence': None}], 'VTCDILSVEAKGVKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 0.0019, 'percentage_identity': 76.9, 'result_sequence': None}], 'GIGKFLHSAKKFGKAFVGEAMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 6.9e-11, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 7.1e-11, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 1.8e-09, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 2.5e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 4.5e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 4.5e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}], 'KGGPCAKKPCCGPLGHYKVDCSTIPDYPCCSKYGFCGSGPQYCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 0.0006, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 0.00087, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 0.0013, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 0.0022, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 0.0022, 'percentage_identity': 40.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9wga_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '9wga', 'e_value': 0.0022, 'percentage_identity': 43.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7u_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7u', 'e_value': 0.0023, 'percentage_identity': 40.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7v_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7v', 'e_value': 0.0023, 'percentage_identity': 40.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7t_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7t', 'e_value': 0.0023, 'percentage_identity': 40.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgt_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1wgt', 'e_value': 0.0023, 'percentage_identity': 40.9, 'result_sequence': None}], 'GFGCPFNQYECHAHCSGVPGYKGGYCKGLFKQTCNCY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 1e-10, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 1.6e-10, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 2.4e-10, 'percentage_identity': 59.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 1.1e-09, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 5.6e-08, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 8.6e-08, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 8.6e-08, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 8.6e-08, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k51_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k51', 'e_value': 2e-07, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 4.3e-06, 'percentage_identity': 47.5, 'result_sequence': None}], 'ATCDLLSGIGVQHSACALHCVFRGNRGGYCTGKGICVCRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 3.9e-18, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 1.8e-17, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 2.3e-17, 'percentage_identity': 77.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 3.3e-12, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 3.7e-11, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 2.2e-10, 'percentage_identity': 57.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 2.8e-10, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 2.1e-09, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 4.5e-08, 'percentage_identity': 53.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 2.5e-06, 'percentage_identity': 50.0, 'result_sequence': None}], 'CGGYSGGWKRKRSTSYRCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 6.5e-06, 'percentage_identity': 73.7, 'result_sequence': None}], 'RWKIFKKIERVGQNVRDGIIKAGPAIQVLGTAKALGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 1e-11, 'percentage_identity': 73.0, 'result_sequence': None}], 'LPRDTSRCVGYHGYCIRSKVCPKPFAAFGTCSWRQKTCCVDTTSDFHTCQDKGGHCVSPKIRCLEEQLGLCPLKRWTCCKEI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qeu_1', 'molecule_name': ['Gallinacin-11'], 'pdb_id': '6qeu', 'e_value': 8.3e-42, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qet_1', 'molecule_name': ['Gallinacin-11'], 'pdb_id': '6qet', 'e_value': 3.6e-18, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qes_1', 'molecule_name': ['Gallinacin-11'], 'pdb_id': '6qes', 'e_value': 1.7e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3osl_2', 'molecule_name': ['Carboxypeptidase inhibitor'], 'pdb_id': '3osl', 'e_value': 0.094, 'percentage_identity': 31.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3d4u_2', 'molecule_name': ['Carboxypeptidase inhibitor'], 'pdb_id': '3d4u', 'e_value': 0.094, 'percentage_identity': 31.2, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3lms_2', 'molecule_name': ['Carboxypeptidase inhibitor'], 'pdb_id': '3lms', 'e_value': 0.094, 'percentage_identity': 31.2, 'result_sequence': None}], 'ILGPVLGLVSNALGGLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 0.00012, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 0.00012, 'percentage_identity': 82.4, 'result_sequence': None}], 'GIPCGESCVFIPCISSVIGCSCSSKVCYRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 5.2e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 5.2e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 2.5e-13, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 2e-12, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 3.8e-12, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 1.6e-11, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 1.2e-10, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 1.7e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 1.7e-07, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 1.4e-06, 'percentage_identity': 64.3, 'result_sequence': None}], 'RWCFRVCYRGICYRKCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 2.8e-08, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7ald_1', 'molecule_name': ['BRICHOS domain-containing protein'], 'pdb_id': '7ald', 'e_value': 0.0004, 'percentage_identity': 62.5, 'result_sequence': None}], 'KYYGNGVTCGKHSCSVDWGKATTCIINNGAMAWATGGHQGNHKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ukz_1', 'molecule_name': ['Bacteriocin pediocin PA-1'], 'pdb_id': '5ukz', 'e_value': 9.5e-23, 'percentage_identity': 97.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohm_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohm', 'e_value': 2.2e-14, 'percentage_identity': 70.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohn_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohn', 'e_value': 6.5e-12, 'percentage_identity': 67.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1og7_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1og7', 'e_value': 6.5e-12, 'percentage_identity': 67.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n4k_1', 'molecule_name': ['Enterocin-HF'], 'pdb_id': '2n4k', 'e_value': 8.4e-09, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2leu', 'e_value': 0.00021, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw6_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '1cw6', 'e_value': 0.00021, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '3leu', 'e_value': 0.00021, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw5_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1cw5', 'e_value': 0.0013, 'percentage_identity': 42.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ry3_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1ry3', 'e_value': 0.0016, 'percentage_identity': 42.1, 'result_sequence': None}], 'GVVDILKGAAKDIAGHLAAKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 2.3e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 2.3e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 2.3e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 4.5e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 4.5e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 4.5e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 3e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 3e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 3e-08, 'percentage_identity': 95.5, 'result_sequence': None}], 'LFCKGGSCHFGGCPSHLIKVGSCFGFRSCCAWPWNA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg6_1', 'molecule_name': ['Gallinacin-2'], 'pdb_id': '2lg6', 'e_value': 4e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg5_1', 'molecule_name': ['Gallinacin-2'], 'pdb_id': '2lg5', 'e_value': 1.4e-19, 'percentage_identity': 97.2, 'result_sequence': None}], 'SFLNFFKGAAKNLLAAGLDKLKCKISGTQC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.063, 'percentage_identity': 51.9, 'result_sequence': None}], 'LTCDILGSTPACAAHCIARGYRGGWCDGQSVCNCRR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 6.7e-12, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 1.4e-08, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.7e-08, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 2.1e-08, 'percentage_identity': 55.3, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 2.2e-07, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 3.5e-07, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 6.3e-07, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 7.7e-07, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 4.3e-06, 'percentage_identity': 48.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 2.5e-05, 'percentage_identity': 45.2, 'result_sequence': None}], 'TFFRLFNRSFTQALGKGGGKNLRIIRKGIHIIKKY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2i2j_1', 'molecule_name': ['Competence stimulating protein'], 'pdb_id': '2i2j', 'e_value': 0.00036, 'percentage_identity': 100.0, 'result_sequence': None}], 'KKLALAPKKLALLWKKLALALKKA': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l96_1', 'molecule_name': ['LAK160-P7'], 'pdb_id': '2l96', 'e_value': 1.1e-07, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l99_1', 'molecule_name': ['LAK160-P10'], 'pdb_id': '2l99', 'e_value': 0.00012, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l9a_1', 'molecule_name': ['LAK160-P12'], 'pdb_id': '2l9a', 'e_value': 0.00012, 'percentage_identity': 75.0, 'result_sequence': None}], 'NAPAEDLARYYSALRHYINLITRQRY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f8p_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1f8p', 'e_value': 5.4e-12, 'percentage_identity': 96.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ron_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1ron', 'e_value': 9.3e-12, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fvn_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1fvn', 'e_value': 1.1e-10, 'percentage_identity': 88.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1icy_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1icy', 'e_value': 1.4e-10, 'percentage_identity': 88.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz4_1', 'molecule_name': ['Pancreatic hormone', 'Neuropeptide Y'], 'pdb_id': '1tz4', 'e_value': 8e-09, 'percentage_identity': 80.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oon_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oon', 'e_value': 4e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rlk_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2rlk', 'e_value': 2e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qbf_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1qbf', 'e_value': 2e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ru5_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ru5', 'e_value': 2e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oop_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oop', 'e_value': 2e-07, 'percentage_identity': 65.4, 'result_sequence': None}], 'KWKLFKKIGIGKFL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.0011, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.0011, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.0014, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.0014, 'percentage_identity': 92.9, 'result_sequence': None}], 'GLPALSSWIKRKRQQ': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 4.8e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 4.8e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 5e-05, 'percentage_identity': 93.3, 'result_sequence': None}], 'RGLRRLGRKIAHGVKKYGATVLRIIRIA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fry_1', 'molecule_name': ['Cathelin-related peptide SC5'], 'pdb_id': '1fry', 'e_value': 2.2e-11, 'percentage_identity': 96.4, 'result_sequence': None}], 'GKYGFYTHVFRLKKWMQKVIDRFGG': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5to3_2', 'molecule_name': ['Thrombin heavy chain', 'Thrombomodulin'], 'pdb_id': '5to3', 'e_value': 1.6e-10, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 3, 'entry_entity': '1bhx_3', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '1bhx', 'e_value': 5.1e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 4, 'entry_entity': '2hnt_4', 'molecule_name': ['Thrombin heavy chain'], 'pdb_id': '2hnt', 'e_value': 5.1e-10, 'percentage_identity': 91.7, 'result_sequence': None}], 'GVVDILAGAAKDIAGHLASKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 3e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 3e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 3e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 6.1e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 6.1e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 6.1e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 4.1e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 4.1e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 4.1e-08, 'percentage_identity': 95.5, 'result_sequence': None}], 'RWCVYRYVRVRGVLVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 2.7e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 3.6e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 3.6e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 2.4e-10, 'percentage_identity': 90.5, 'result_sequence': None}], 'RIPCQYEDATEDTICQQHCLPKGYSYGICVSYRCSCV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.02, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kp0_1', 'molecule_name': ['nasonin-1M'], 'pdb_id': '2kp0', 'e_value': 0.03, 'percentage_identity': 42.9, 'result_sequence': None}], 'KQIKIWFQNKKMKWKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2o00_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2o00', 'e_value': 1.2e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nzz_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2nzz', 'e_value': 1.2e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlx_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlx', 'e_value': 1.8e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlw_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlw', 'e_value': 1.8e-05, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4xic_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '4xic', 'e_value': 1.8e-05, 'percentage_identity': 81.2, 'result_sequence': None}], 'RWCVYAYSRVRGVLVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 6.6e-12, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 8.8e-12, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 1.2e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 1.2e-11, 'percentage_identity': 90.5, 'result_sequence': None}], 'KFFARLLASVRAAVKKFRKKPRLIGLSTLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.081, 'percentage_identity': 44.4, 'result_sequence': None}], 'INLKAIAAMAKKLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6duu_1', 'molecule_name': ['Mastoparan-L'], 'pdb_id': '6duu', 'e_value': 0.0015, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1smz_1', 'molecule_name': ['Transportan in bicellar solution with [DMPC]/[DHPC]=0.33'], 'pdb_id': '1smz', 'e_value': 0.0021, 'percentage_identity': 78.6, 'result_sequence': None}], 'KWKLFKKIEKVGQGIGAVLKVLTTGL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l36_1', 'molecule_name': ['MSI594'], 'pdb_id': '2l36', 'e_value': 0.0052, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.012, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 0.061, 'percentage_identity': 66.7, 'result_sequence': None}], 'GVLDTFKDVAIGVAKGAGTGVLKALLCKLDKSC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 7.9e-05, 'percentage_identity': 51.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.028, 'percentage_identity': 48.3, 'result_sequence': None}], 'GGLRSLGRKILRAWKKYGPIIVPIIRI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fry_1', 'molecule_name': ['Cathelin-related peptide SC5'], 'pdb_id': '1fry', 'e_value': 0.00029, 'percentage_identity': 65.4, 'result_sequence': None}], 'GFGCPFNQGACHRHCRSIGRRGGYCAGLFKQTCTCYSR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 9.3e-16, 'percentage_identity': 76.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 2.7e-11, 'percentage_identity': 65.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 9.5e-10, 'percentage_identity': 64.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 7.3e-09, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 4.3e-07, 'percentage_identity': 55.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 1e-06, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 5.2e-06, 'percentage_identity': 54.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 1.2e-05, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 1.2e-05, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 1.2e-05, 'percentage_identity': 52.4, 'result_sequence': None}], 'RIKRFWPVVIRTVVAGYNLYR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 2.5e-08, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hfr_1', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '2hfr', 'e_value': 0.0049, 'percentage_identity': 63.6, 'result_sequence': None}], 'ATCDLLSGFGVGDSACAAHCIARGNRGGYCNSKKVCVCRN': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 9.8e-16, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 1.2e-15, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.5e-15, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 3.4e-15, 'percentage_identity': 77.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 1e-12, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 2.9e-12, 'percentage_identity': 67.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 1.3e-11, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 5.9e-10, 'percentage_identity': 65.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 3.3e-07, 'percentage_identity': 53.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 4.4e-07, 'percentage_identity': 50.0, 'result_sequence': None}], 'GFFALIPKIIS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xc0_1', 'molecule_name': ['Pardaxin P-4'], 'pdb_id': '1xc0', 'e_value': 0.00014, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kns_1', 'molecule_name': ['Pardaxin P-4'], 'pdb_id': '2kns', 'e_value': 0.00014, 'percentage_identity': 100.0, 'result_sequence': None}], 'FFHHIFRAIVHVAKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 3.5e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 6.5e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 6.5e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 6.6e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 6.6e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 6.6e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 8.7e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 5.9e-05, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 5.9e-05, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 5.9e-05, 'percentage_identity': 59.1, 'result_sequence': None}], 'RWKIFKKIEKMGGSYCNRRTGKCQRM': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xo9_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo9', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo4_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo4', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gd5_2', 'molecule_name': ['Thanatin'], 'pdb_id': '6gd5', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6aab_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6aab', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8tfv_1', 'molecule_name': ['Thanatin'], 'pdb_id': '8tfv', 'e_value': 0.0012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo3_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo3', 'e_value': 0.0075, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xoa_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xoa', 'e_value': 0.0075, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6afq_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6afq', 'e_value': 0.0075, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xok_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xok', 'e_value': 0.064, 'percentage_identity': 100.0, 'result_sequence': None}], 'KKFGGGKTERARKAMAVE': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rd7_1', 'molecule_name': ['Complement component C8 alpha chain'], 'pdb_id': '2rd7', 'e_value': 2.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3ojy_1', 'molecule_name': ['Complement component C8 alpha chain'], 'pdb_id': '3ojy', 'e_value': 3.3e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'PTTKTYFPHF': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5woh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5woh', 'e_value': 0.006, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5wog_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '5wog', 'e_value': 0.006, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mqh_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '4mqh', 'e_value': 0.0061, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6bb5_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6bb5', 'e_value': 0.0061, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6hal_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6hal', 'e_value': 0.0061, 'percentage_identity': 100.0, 'result_sequence': None}], 'FFHHIFRAIVHVPKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 3.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 1.9e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 1.9e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 1.9e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 1.9e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 1.9e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 2.5e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 0.00014, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 0.00014, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 0.00014, 'percentage_identity': 59.1, 'result_sequence': None}], 'KWKVFKKIEKKWKVFKKIEKAGPKWKVFKKIEK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2igr_1', 'molecule_name': ['Anticancer peptide CB1a'], 'pdb_id': '2igr', 'e_value': 3.6e-16, 'percentage_identity': 100.0, 'result_sequence': None}], 'EAEEDGDLQCLCVKTTSQVRPRHITSLEVIKAGPHCPTAQLIATLKNGRKICLDLQAPLYKKIIKKLLES': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4r9w_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4r9w', 'e_value': 1.1e-33, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C', 'B', 'A'], 'entity_id': 1, 'entry_entity': '4r9y_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4r9y', 'e_value': 1.1e-33, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'U', 'I', 'C', 'X', 'O', 'F', 'L'], 'entity_id': 3, 'entry_entity': '4rau_3', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4rau', 'e_value': 1.1e-33, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1f9q_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1f9q', 'e_value': 1.1e-33, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1rhp_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1rhp', 'e_value': 1.1e-33, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1f9s_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1f9s', 'e_value': 5.3e-33, 'percentage_identity': 98.6, 'result_sequence': None}], 'GVAKFGKAAAHFGKGWIKEMLNS': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.013, 'percentage_identity': 43.5, 'result_sequence': None}], 'FQWQRNPRKVR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 0.019, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 0.019, 'percentage_identity': 90.9, 'result_sequence': None}], 'ILGPVLGLVGDTLGDLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 0.0012, 'percentage_identity': 76.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 0.0012, 'percentage_identity': 76.5, 'result_sequence': None}], 'AIGSILGRLAKGLPTLISWIKNR': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.0036, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.0036, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.0037, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.0037, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.0037, 'percentage_identity': 59.1, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.0037, 'percentage_identity': 59.1, 'result_sequence': None}], 'KRGFGKKLRKRLKKFRNSIKKRLKNFNVVIPIPLPG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 0.048, 'percentage_identity': 57.1, 'result_sequence': None}], 'GLPICGETCVGGTCNTPGCFCTWPVCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 1.3e-13, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 1.3e-13, 'percentage_identity': 89.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 3.7e-13, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 5.6e-13, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 5.6e-13, 'percentage_identity': 89.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 6.9e-13, 'percentage_identity': 86.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 1.6e-12, 'percentage_identity': 86.7, 'result_sequence': None}], 'KKLFKKILKYLGIGAVLKVLTTGLKDEL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.018, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l36_1', 'molecule_name': ['MSI594'], 'pdb_id': '2l36', 'e_value': 0.041, 'percentage_identity': 73.9, 'result_sequence': None}], 'GLPVCGETCTLGTCYTQGCTCSWPICKRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 5.6e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 9.5e-13, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 2.2e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 2.9e-11, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 2.9e-11, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 4.5e-11, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 8.5e-11, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 1.3e-10, 'percentage_identity': 78.6, 'result_sequence': None}], 'FKCRRWQWRMKKLGKRSKNKGFKEQAKSLLKWILD': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 5.3e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.001, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.027, 'percentage_identity': 76.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.027, 'percentage_identity': 76.5, 'result_sequence': None}], 'RLGNFFRKVKEKIGGGLKKVGQKIKDFLGNLVPRTAS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 1.7e-09, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 1.7e-09, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 1.7e-09, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 1.7e-09, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 1.7e-09, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.0021, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 0.034, 'percentage_identity': 59.1, 'result_sequence': None}], 'AMDPTKYYGNGVYCNSKKCWVDWGQASGCIGQTVVGGWLGGAIVGKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohm_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohm', 'e_value': 4e-08, 'percentage_identity': 52.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2a2b_1', 'molecule_name': ['Bacteriocin curvacin-A'], 'pdb_id': '2a2b', 'e_value': 1.8e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohn_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohn', 'e_value': 1.9e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1og7_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1og7', 'e_value': 1.9e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n4k_1', 'molecule_name': ['Enterocin-HF'], 'pdb_id': '2n4k', 'e_value': 7.5e-06, 'percentage_identity': 52.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ukz_1', 'molecule_name': ['Bacteriocin pediocin PA-1'], 'pdb_id': '5ukz', 'e_value': 1.7e-05, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2leu', 'e_value': 0.0048, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw6_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '1cw6', 'e_value': 0.0048, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '3leu', 'e_value': 0.0048, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw5_1', 'molecule_name': ['Bacteriocin carnobacteriocin B2'], 'pdb_id': '1cw5', 'e_value': 0.023, 'percentage_identity': 40.9, 'result_sequence': None}], 'IIGPVLGLIGKALGGLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 3.5e-05, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 3.5e-05, 'percentage_identity': 88.2, 'result_sequence': None}], 'KHVGKAALTHYL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 9.7e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 6.5e-05, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 9.5e-05, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 0.0042, 'percentage_identity': 66.7, 'result_sequence': None}], 'GILDKLKEFGISAARGVAQSLLNTASCKLAKTC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 6.5e-07, 'percentage_identity': 56.8, 'result_sequence': None}], 'KNIGNSVSCLRNKGVCMPGKCAPKMKQIGTCGMPQVKCCKRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 1.5e-08, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 1.5e-08, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 1.5e-08, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 1.5e-08, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 2.3e-07, 'percentage_identity': 51.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 1.4e-06, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 2.6e-05, 'percentage_identity': 52.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.00013, 'percentage_identity': 52.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.0058, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 0.016, 'percentage_identity': 45.7, 'result_sequence': None}], 'EPFSSVAATCQTPKIACKNGDKNCHQSHGAVSLTMCKLTSGKYPNCRYKEKR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 8.8e-24, 'percentage_identity': 98.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 9e-10, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 9.1e-10, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 9.1e-10, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 9.1e-10, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 9.1e-10, 'percentage_identity': 56.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nui_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '2nui', 'e_value': 1.3e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3ql1_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ql1', 'e_value': 1.6e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ql2_1', 'molecule_name': ['Ribonuclease pancreatic'], 'pdb_id': '3ql2', 'e_value': 1.6e-08, 'percentage_identity': 50.0, 'result_sequence': None}], 'RGRGRGRGRG': [{'chain_id': ['F', 'f'], 'entity_id': 6, 'entry_entity': '3jb9_6', 'molecule_name': ['Small nuclear ribonucleoprotein Sm D1'], 'pdb_id': '3jb9', 'e_value': 0.023, 'percentage_identity': 100.0, 'result_sequence': None}], 'GVVDILKGAAKDIAGHLASKVANKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 1e-09, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 1e-09, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 1e-09, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 1.9e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 1.9e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 1.9e-08, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 9.3e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 9.3e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 9.3e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'GVPICGETCTLGTCYTAGCSCSWPVCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 1.8e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 1.5e-12, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 6.7e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4ttm_2', 'molecule_name': ['D-kalata B1'], 'pdb_id': '4ttm', 'e_value': 5.6e-11, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 5.6e-11, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lam_1', 'molecule_name': ['Cyclotide cter-M'], 'pdb_id': '2lam', 'e_value': 1.1e-10, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 1.6e-10, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 2e-10, 'percentage_identity': 84.0, 'result_sequence': None}], 'GIGKFLKSKGKFGKA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.012, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.013, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.04, 'percentage_identity': 73.3, 'result_sequence': None}], 'KFHHIFRGIVHVGKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 1.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 1.9e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 2e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 3.3e-09, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 1.8e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 1.2e-05, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 1.2e-05, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 1.2e-05, 'percentage_identity': 70.0, 'result_sequence': None}], 'KTCENLADTY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 0.0018, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 0.0079, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jkz_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '1jkz', 'e_value': 0.016, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 0.016, 'percentage_identity': 90.0, 'result_sequence': None}], 'KSSAYSLQMGATAIKQVKKLFKKWGW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ytr_1', 'molecule_name': ['Bacteriocin plantaricin-A'], 'pdb_id': '1ytr', 'e_value': 5.1e-13, 'percentage_identity': 100.0, 'result_sequence': None}], 'RIWFQNRRMRWRR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2o00_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2o00', 'e_value': 0.0004, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nzz_1', 'molecule_name': ['Penetratin conjugated Gas (374-394) peptide'], 'pdb_id': '2nzz', 'e_value': 0.0004, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlx_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlx', 'e_value': 0.00064, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '5jlw_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '5jlw', 'e_value': 0.00064, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['D', 'A'], 'entity_id': 1, 'entry_entity': '4xic_1', 'molecule_name': ['Homeotic protein antennapedia'], 'pdb_id': '4xic', 'e_value': 0.00064, 'percentage_identity': 69.2, 'result_sequence': None}], 'FRKSKEKIGKEFKRI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 2.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 3.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 3.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 3.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 3.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 3.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'MLAKIKAMIKKFPNPYTLAAKLTTYEINWYKQQYGRYPWERPVA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5l82_1', 'molecule_name': ['Enterococcin K1'], 'pdb_id': '5l82', 'e_value': 0.00084, 'percentage_identity': 53.1, 'result_sequence': None}], 'GFKEVLKAGLGSLVKGIPAHVAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2l5r_1', 'molecule_name': ['Antimicrobial peptide Alyteserin-1C'], 'pdb_id': '2l5r', 'e_value': 3.7e-07, 'percentage_identity': 78.3, 'result_sequence': None}], 'GLFGKLIKKFLRKAISYAVKKARGKH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9p_1', 'molecule_name': ['M-zodatoxin-Lt2a'], 'pdb_id': '2g9p', 'e_value': 2.3e-10, 'percentage_identity': 96.2, 'result_sequence': None}], 'KKIMRTFLRRISKKILTGKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nkl_1', 'molecule_name': ['Antimicrobial peptide NK-lysin'], 'pdb_id': '1nkl', 'e_value': 1.5e-05, 'percentage_identity': 85.0, 'result_sequence': None}], 'RPKPRPRPRPE': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 0.008, 'percentage_identity': 90.0, 'result_sequence': None}], 'GIFSKLAGKKLKNLLISG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n6m_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '2n6m', 'e_value': 6.7e-07, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdj_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '5xdj', 'e_value': 6.7e-07, 'percentage_identity': 94.4, 'result_sequence': None}], 'VCRTGRSRWRDVCRNFMRRYQSR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l9l_1', 'molecule_name': ['Granulysin'], 'pdb_id': '1l9l', 'e_value': 3.1e-12, 'percentage_identity': 100.0, 'result_sequence': None}], 'AGLQFPVGRVHRLLRK': [{'chain_id': ['I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2xql_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2xql', 'e_value': 1.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4kha_2', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kha', 'e_value': 1.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hio', 'e_value': 1.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6k01_1', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k01', 'e_value': 1.6e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'GFKRIVQAIKDFLRNLV': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.00017, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.00024, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.00024, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.00024, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.00024, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.00024, 'percentage_identity': 93.8, 'result_sequence': None}], 'VTCDLLSFKGQVNDSACAANCLSLGKAGGHCEKGVCICRKTSFKDLWDKRF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 3.8e-08, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.9e-07, 'percentage_identity': 64.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 8.8e-07, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 5.3e-06, 'percentage_identity': 53.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 5.9e-06, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 3.4e-05, 'percentage_identity': 52.4, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.00015, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 0.00056, 'percentage_identity': 51.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 0.00082, 'percentage_identity': 48.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 0.00082, 'percentage_identity': 48.7, 'result_sequence': None}], 'GILSSFKDVAKGVAKNVAAQLLDKLKCKITGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 5.9e-11, 'percentage_identity': 78.1, 'result_sequence': None}], 'GKFHQIKKMFLSVGVKVTSLKRIQFGDF': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1vio_1', 'molecule_name': ['Ribosomal small subunit pseudouridine synthase A'], 'pdb_id': '1vio', 'e_value': 0.0015, 'percentage_identity': 48.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ksk_1', 'molecule_name': ['Ribosomal small subunit pseudouridine synthase A'], 'pdb_id': '1ksk', 'e_value': 0.04, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ksv_1', 'molecule_name': ['Ribosomal small subunit pseudouridine synthase A'], 'pdb_id': '1ksv', 'e_value': 0.04, 'percentage_identity': 46.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ksl_1', 'molecule_name': ['Ribosomal small subunit pseudouridine synthase A'], 'pdb_id': '1ksl', 'e_value': 0.04, 'percentage_identity': 46.2, 'result_sequence': None}], 'KWWCFRVCYRGICYRRCRWK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 5.2e-06, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7ald_1', 'molecule_name': ['BRICHOS domain-containing protein'], 'pdb_id': '7ald', 'e_value': 0.0018, 'percentage_identity': 68.8, 'result_sequence': None}], 'GAVDILKGAAKDIAGHLASKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 1.8e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 1.8e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 1.8e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 3.6e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 3.6e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 3.6e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 2.4e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 2.4e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 2.4e-08, 'percentage_identity': 95.5, 'result_sequence': None}], 'GIGKFLHAAKKFAKAFVAEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 1.8e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 1.1e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 1.1e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 1.3e-09, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 1.4e-09, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 3.8e-08, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.018, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.018, 'percentage_identity': 91.7, 'result_sequence': None}]}

In [32]:
len(res_45h)

89

In [53]:
%store res_45h

Stored 'res_45h' (dict)


## 4500- 5000 (colab)

In [50]:
res_5k={'AARCLSQRRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 0.03, 'percentage_identity': 80.0, 'result_sequence': None}], 'RSGGYCNGKRVCVCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 2.2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.00022, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.00029, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.00064, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 0.0018, 'percentage_identity': 84.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 0.0018, 'percentage_identity': 84.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 0.0018, 'percentage_identity': 84.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ygp_1', 'molecule_name': ['Wnt inhibitory factor 1'], 'pdb_id': '2ygp', 'e_value': 0.064, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ygo_1', 'molecule_name': ['Wnt inhibitory factor 1'], 'pdb_id': '2ygo', 'e_value': 0.064, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ygq_1', 'molecule_name': ['Wnt inhibitory factor 1'], 'pdb_id': '2ygq', 'e_value': 0.097, 'percentage_identity': 57.1, 'result_sequence': None}], 'RIKRFWPVVIRTVVAGYNL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2amn_1', 'molecule_name': ['Cathelicidin-1'], 'pdb_id': '2amn', 'e_value': 8.5e-07, 'percentage_identity': 78.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hfr_1', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '2hfr', 'e_value': 0.06, 'percentage_identity': 65.0, 'result_sequence': None}], 'CGNFLKRTCICVKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 0.054, 'percentage_identity': 50.0, 'result_sequence': None}], 'GFKRIVQRIADFLRNLV': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.00014, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.00019, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.00019, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.00019, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.00019, 'percentage_identity': 93.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.00019, 'percentage_identity': 93.8, 'result_sequence': None}], 'GGVWNLNVNPGTTGARVWARTK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4jru_1', 'molecule_name': ['thaumatin-like protein'], 'pdb_id': '4jru', 'e_value': 3.1e-05, 'percentage_identity': 76.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4l2j_1', 'molecule_name': ['Osmotin: antifungal laticifer protein'], 'pdb_id': '4l2j', 'e_value': 0.00016, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mbt_1', 'molecule_name': ['VVTL1'], 'pdb_id': '4mbt', 'e_value': 0.00034, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4l5h_1', 'molecule_name': ['VVTL1'], 'pdb_id': '4l5h', 'e_value': 0.00034, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z3q_1', 'molecule_name': ['Ripening-associated protein'], 'pdb_id': '1z3q', 'e_value': 0.00045, 'percentage_identity': 61.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kw7_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kw7', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kvx_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kvx', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5kw4_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5kw4', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lh6_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5lh6', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lh1_1', 'molecule_name': ['Thaumatin I'], 'pdb_id': '5lh1', 'e_value': 0.0006, 'percentage_identity': 57.1, 'result_sequence': None}], 'ENFFKEIERAGQRIRDAIISAAPAVETLAQAQKIIKGGD': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 7.4e-05, 'percentage_identity': 48.5, 'result_sequence': None}], 'RRWCFRVCYKGRCRYKCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 3.1e-07, 'percentage_identity': 78.9, 'result_sequence': None}], 'IGKLVKWIIKTVNKFTKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dtb_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2dtb', 'e_value': 2.2e-05, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kam_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2kam', 'e_value': 2.2e-05, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dtc_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '1dtc', 'e_value': 2.2e-05, 'percentage_identity': 88.9, 'result_sequence': None}], 'KGWGSFFKKAAHVGKHVGKAALTHYL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 8.3e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 8.5e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 1.2e-11, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 2e-11, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 1.7e-06, 'percentage_identity': 60.0, 'result_sequence': None}], 'GLFGVLAKVAAHVVPAIAEHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmj_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mmj', 'e_value': 3.7e-07, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn8_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn8', 'e_value': 3.7e-07, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn9_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn9', 'e_value': 3.7e-07, 'percentage_identity': 90.5, 'result_sequence': None}], 'GSKGAPCAKKPCCGPLGHYKVDCSTIPDYPCCSKVGFCGSGPQYCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 0.001, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 0.0022, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 0.0032, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zwu_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zwu', 'e_value': 0.0039, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mmc_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1mmc', 'e_value': 0.0046, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zuv_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1zuv', 'e_value': 0.0068, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 0.011, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 0.011, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulk_1', 'molecule_name': ['Lectin-C'], 'pdb_id': '1ulk', 'e_value': 0.011, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '9wga_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '9wga', 'e_value': 0.011, 'percentage_identity': 41.3, 'result_sequence': None}], 'GLGKFLHSAKRFGKAFVGEAMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 6.6e-10, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 6.8e-10, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 1.6e-08, 'percentage_identity': 78.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 2.2e-07, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 3.9e-07, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 3.9e-07, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.07, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.07, 'percentage_identity': 83.3, 'result_sequence': None}], 'QYEALVTSILGKLTGLWHNDSVDFMGHICYFRRRPKIRRFKLYHEGKFWCPGWAPFEGRSRTKSRSGSSREATKDFVRKALQNGLVTQQDASLWLNN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2job_1', 'molecule_name': ['Anti-lipopolysaccharide factor isoform 3'], 'pdb_id': '2job', 'e_value': 9.4e-24, 'percentage_identity': 42.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mdl_1', 'molecule_name': ['Anti-lipopolysaccharide factor'], 'pdb_id': '2mdl', 'e_value': 1.8e-07, 'percentage_identity': 78.3, 'result_sequence': None}], 'GIGAVLKVLALISWIKRKR': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 4.4e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 4.4e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 4.5e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 4.5e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 4.5e-05, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 4.5e-05, 'percentage_identity': 79.2, 'result_sequence': None}], 'ATYNGKCYKKDNICKYKAQSGKTAICKCYVKKCPRDGAKCEFDSYKGKCYC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1afp_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '1afp', 'e_value': 3e-23, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oqs_1', 'molecule_name': ['NFAP'], 'pdb_id': '5oqs', 'e_value': 2.6e-07, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbf_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '2nbf', 'e_value': 5.2e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhv_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '2mhv', 'e_value': 5.2e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nb0_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '2nb0', 'e_value': 5.2e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcn_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '2kcn', 'e_value': 5.2e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6haj_1', 'molecule_name': ['Pc24g00380 protein'], 'pdb_id': '6haj', 'e_value': 5.2e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hah_1', 'molecule_name': ['Pc24g00380 protein'], 'pdb_id': '6hah', 'e_value': 5.2e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6ha4_1', 'molecule_name': ['Pc24g00380 protein'], 'pdb_id': '6ha4', 'e_value': 5.2e-06, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nc2_1', 'molecule_name': ['Antifungal protein'], 'pdb_id': '2nc2', 'e_value': 0.026, 'percentage_identity': 32.7, 'result_sequence': None}], 'KEFKRIVQRIKDFLRNLV': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 4.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 5.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 5.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 5.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 5.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 5.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLLPCAESCVYIPCLTTVIGCSCKSKVCYKN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 8.4e-12, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 8.4e-12, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 2.6e-11, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 4.2e-11, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 1.7e-10, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 1.7e-10, 'percentage_identity': 70.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 1.1e-09, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 5.4e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 1.1e-07, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 5.5e-07, 'percentage_identity': 67.9, 'result_sequence': None}], 'TRRRLFNRSFTQALGKSGGGFKKFWKWFRRF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2i2j_1', 'molecule_name': ['Competence stimulating protein'], 'pdb_id': '2i2j', 'e_value': 0.032, 'percentage_identity': 100.0, 'result_sequence': None}], 'IGKFLKKAKKFG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.013, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.014, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.052, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.054, 'percentage_identity': 100.0, 'result_sequence': None}], 'KSCCRSTTARTARVPCAKKSNIYNGCRVPGGCKIQEAKKCEPPYD': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3c8p_1', 'molecule_name': ['Viscotoxin A1'], 'pdb_id': '3c8p', 'e_value': 0.013, 'percentage_identity': 38.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jmp_1', 'molecule_name': ['Viscotoxin-B'], 'pdb_id': '1jmp', 'e_value': 0.036, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2v9b_1', 'molecule_name': ['Viscotoxin-B'], 'pdb_id': '2v9b', 'e_value': 0.044, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1orl_1', 'molecule_name': ['Viscotoxin-C1'], 'pdb_id': '1orl', 'e_value': 0.044, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jmn_1', 'molecule_name': ['Viscotoxin-A2'], 'pdb_id': '1jmn', 'e_value': 0.054, 'percentage_identity': 36.4, 'result_sequence': None}], 'KTCENLADTYKGPCFTTGSCDDHCKNKEHLRSGRCRDDFRCWCTKNC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 3.5e-23, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 3.1e-22, 'percentage_identity': 93.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 1.2e-20, 'percentage_identity': 87.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 1.1e-17, 'percentage_identity': 74.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 3.4e-16, 'percentage_identity': 70.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jkz_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '1jkz', 'e_value': 1.6e-13, 'percentage_identity': 68.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 4.6e-06, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 4.1e-05, 'percentage_identity': 39.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 0.0001, 'percentage_identity': 35.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2q_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '2n2q', 'e_value': 0.00056, 'percentage_identity': 31.4, 'result_sequence': None}], 'KISKKIMRTFLRRILTGKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nkl_1', 'molecule_name': ['Antimicrobial peptide NK-lysin'], 'pdb_id': '1nkl', 'e_value': 0.012, 'percentage_identity': 68.2, 'result_sequence': None}], 'FFHHIFRGIVHVGKTAHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 8.6e-11, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 8.6e-11, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 8.8e-11, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 8.8e-11, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 8.8e-11, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 1.5e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 8e-09, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 5.7e-06, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 5.7e-06, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 5.7e-06, 'percentage_identity': 63.6, 'result_sequence': None}], 'SNDIYFNFQR': [{'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1g8w_1', 'molecule_name': ['Leucoagglutinating phytohemagglutinin'], 'pdb_id': '1g8w', 'e_value': 0.0084, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fat_1', 'molecule_name': ['Leucoagglutinating phytohemagglutinin'], 'pdb_id': '1fat', 'e_value': 0.009, 'percentage_identity': 100.0, 'result_sequence': None}], 'RPPQFTRAQWFAIQHISLN': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 3.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 3.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 3.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 3.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvz_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2lvz', 'e_value': 3.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxf_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxf', 'e_value': 3.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dyt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1dyt', 'e_value': 3.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qmt_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1qmt', 'e_value': 3.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kb5_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '2kb5', 'e_value': 3.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h1h_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '1h1h', 'e_value': 3.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}], 'GWGSFFKKAAHVGKHVGKAALT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 2.3e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 2.4e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 4e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 7.1e-10, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 0.00015, 'percentage_identity': 57.1, 'result_sequence': None}], 'SSMKLSFRARAYGFRGPGLQL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lv4_1', 'molecule_name': ['Catestatin'], 'pdb_id': '1lv4', 'e_value': 3.1e-09, 'percentage_identity': 95.2, 'result_sequence': None}], 'FFHHIWRGIVHVGKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 3.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 3.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 3.5e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 3.5e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 3.5e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 5.3e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 2.8e-08, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 1.6e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 1.6e-05, 'percentage_identity': 63.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 1.6e-05, 'percentage_identity': 63.6, 'result_sequence': None}], 'KRIVQRIKDWLR': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.045, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.068, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.068, 'percentage_identity': 91.7, 'result_sequence': None}], 'GKIPVKAIKKGGQIIGKALRGINIASTAHDIISQFKPKKKKNH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jr8_1', 'molecule_name': ['Moricin'], 'pdb_id': '2jr8', 'e_value': 1.1e-11, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1x22_1', 'molecule_name': ['Moricin'], 'pdb_id': '1x22', 'e_value': 2.2e-11, 'percentage_identity': 78.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kv4_1', 'molecule_name': ['Moricin-1'], 'pdb_id': '1kv4', 'e_value': 2.6e-08, 'percentage_identity': 62.5, 'result_sequence': None}], 'KRFKKFFKKLKNSVKKRAKKFFKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 1.1e-06, 'percentage_identity': 83.3, 'result_sequence': None}], 'EVGGGGGSQQCPQERPKLSSCKDYVMERCFTMKDFPVTWPTKWWKGGCEHEVREKCCKQLSQIAPQCRCDSIRRVIQGRLGGFLGIWRGEVFKQLQRAQSLPSKCNMGADCKFPSGYYW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1psy_1', 'molecule_name': ['Allergen Ric c 3 large chain'], 'pdb_id': '1psy', 'e_value': 0.0001, 'percentage_identity': 28.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vj0_1', 'molecule_name': ['AAI domain-containing protein'], 'pdb_id': '6vj0', 'e_value': 0.00034, 'percentage_identity': 34.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lvf_1', 'molecule_name': ['2S sulfur-rich seed storage protein large chain 1B'], 'pdb_id': '2lvf', 'e_value': 0.00054, 'percentage_identity': 28.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6s3f_1', 'molecule_name': ['AAI domain-containing protein'], 'pdb_id': '6s3f', 'e_value': 0.0018, 'percentage_identity': 31.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u87_1', 'molecule_name': ['AAI domain-containing protein'], 'pdb_id': '5u87', 'e_value': 0.0019, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sm7_1', 'molecule_name': ['AAI domain-containing protein'], 'pdb_id': '1sm7', 'e_value': 0.0025, 'percentage_identity': 24.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5dom_1', 'molecule_name': ['AAI domain-containing protein'], 'pdb_id': '5dom', 'e_value': 0.0034, 'percentage_identity': 31.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1w2q_1', 'molecule_name': ['Conglutin'], 'pdb_id': '1w2q', 'e_value': 0.012, 'percentage_identity': 20.2, 'result_sequence': None}], 'YSRCQLQGFNCVVRSYGLPTIPCCRGLTCRSYFPGSTYGRCQRF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cix_1', 'molecule_name': ['Tachystatin-A2'], 'pdb_id': '1cix', 'e_value': 2.2e-21, 'percentage_identity': 97.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dcv_1', 'molecule_name': ['Tachystatin-B1'], 'pdb_id': '2dcv', 'e_value': 0.0014, 'percentage_identity': 44.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dcw_1', 'molecule_name': ['Tachystatin-B2'], 'pdb_id': '2dcw', 'e_value': 0.0018, 'percentage_identity': 44.2, 'result_sequence': None}], 'KSKEKIGKEFKRIVQRIKDFLRNLVPRTES': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 2.5e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 2.5e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 2.5e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 2.5e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 2.5e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 1.9e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 0.0029, 'percentage_identity': 100.0, 'result_sequence': None}], 'AKKVFKRLEKLFS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0g_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0g', 'e_value': 9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 9e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5k_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5k', 'e_value': 0.0014, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5l', 'e_value': 0.0041, 'percentage_identity': 92.3, 'result_sequence': None}], 'LRKLRKRLVRLASHLRKLRKRLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kny_1', 'molecule_name': ['Low-density lipoprotein receptor-related protein 1 515 kDa subunit', 'Apolipoprotein E'], 'pdb_id': '2kny', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6v7m_2', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '6v7m', 'e_value': 0.013, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1le4_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1le4', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1le2_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1le2', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bz4_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1bz4', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lpe_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1lpe', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1or2_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1or2', 'e_value': 0.017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1or3_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1or3', 'e_value': 0.017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gs9_1', 'molecule_name': ['Apolipoprotein E'], 'pdb_id': '1gs9', 'e_value': 0.017, 'percentage_identity': 100.0, 'result_sequence': None}], 'GSKGAPCAKKPCCGPLGHYKVDCSTIPDYPCCSKYGFCGSGPQVCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5zv6_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5zv6', 'e_value': 0.0029, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1s_1', 'molecule_name': ['Antimicrobial peptide 2.2a'], 'pdb_id': '2n1s', 'e_value': 0.0042, 'percentage_identity': 81.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1znt_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1znt', 'e_value': 0.0062, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1mmc_1', 'molecule_name': ['Antimicrobial peptide 2'], 'pdb_id': '1mmc', 'e_value': 0.013, 'percentage_identity': 55.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2wgc_1', 'molecule_name': ['Agglutinin isolectin 2'], 'pdb_id': '2wgc', 'e_value': 0.014, 'percentage_identity': 41.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2x52_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '2x52', 'e_value': 0.014, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7u_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7u', 'e_value': 0.014, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7v_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7v', 'e_value': 0.014, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1k7t_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1k7t', 'e_value': 0.014, 'percentage_identity': 39.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1wgt_1', 'molecule_name': ['Agglutinin isolectin 3'], 'pdb_id': '1wgt', 'e_value': 0.014, 'percentage_identity': 39.1, 'result_sequence': None}], 'AIGSILGRLAKGLPTLKSWIKNR': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.038, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.038, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.039, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.039, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.039, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.039, 'percentage_identity': 54.5, 'result_sequence': None}], 'KWKLFKKIPHLAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.072, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.072, 'percentage_identity': 65.0, 'result_sequence': None}], 'WPKRLTKAHWFEIQHIQPSPLQSNRAMSGINNYTQHSKHQNTFLH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 1.2e-20, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 1.2e-20, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 1.2e-20, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 1.2e-20, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 1.9e-18, 'percentage_identity': 95.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 1.7e-08, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4x08_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4x08', 'e_value': 5.1e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2y_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2y', 'e_value': 5.1e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4a2o_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4a2o', 'e_value': 5.1e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4oxb_1', 'molecule_name': ['Eosinophil cationic protein'], 'pdb_id': '4oxb', 'e_value': 5.1e-06, 'percentage_identity': 51.2, 'result_sequence': None}], 'DFGCARGMIFVCMRRCARMYPGSTGYCQGFRCMCDTHIPIRRPPFIMG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jzb_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jzb', 'e_value': 0.064, 'percentage_identity': 34.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jza_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jza', 'e_value': 0.064, 'percentage_identity': 34.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vxw_1', 'molecule_name': ['Beta-toxin Cl13'], 'pdb_id': '6vxw', 'e_value': 0.065, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['C', 'F'], 'entity_id': 3, 'entry_entity': '2yc1_3', 'molecule_name': ['Beta-mammal toxin Cn2'], 'pdb_id': '2yc1', 'e_value': 0.078, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '4v1d_3', 'molecule_name': ['Beta-mammal toxin Cn2'], 'pdb_id': '4v1d', 'e_value': 0.078, 'percentage_identity': 46.4, 'result_sequence': None}], 'VKRFKKFFRKLKKS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.0068, 'percentage_identity': 84.6, 'result_sequence': None}], 'GSIPCGESCVFIPCISAVIGCSCSNKVCYKN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 2.6e-12, 'percentage_identity': 82.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 1e-11, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 1e-11, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 1.3e-11, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 3.2e-11, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 4.9e-11, 'percentage_identity': 75.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 6.1e-11, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 3.2e-08, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 3.2e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 6e-07, 'percentage_identity': 67.9, 'result_sequence': None}], 'FLPIVAGLAANFLPKIVCKITKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 2e-06, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 2e-06, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 2e-06, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4x_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4x', 'e_value': 0.0036, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4v_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4v', 'e_value': 0.0036, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r95_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r95', 'e_value': 0.0036, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r96_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r96', 'e_value': 0.0036, 'percentage_identity': 60.9, 'result_sequence': None}], 'GASCGETCFTGICFTAGCSCNPWPTCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kuk_1', 'molecule_name': ['Leaf cyclotide 2'], 'pdb_id': '2kuk', 'e_value': 1.8e-10, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3e4h_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '3e4h', 'e_value': 9.4e-08, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k7g_1', 'molecule_name': ['Varv peptide F'], 'pdb_id': '2k7g', 'e_value': 9.4e-08, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 6.3e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mh1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mh1', 'e_value': 6.5e-07, 'percentage_identity': 62.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pt4_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '1pt4', 'e_value': 1.2e-06, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kch_1', 'molecule_name': ['Kalata-B2'], 'pdb_id': '2kch', 'e_value': 1.2e-06, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gj0_1', 'molecule_name': ['Cycloviolacin-O14'], 'pdb_id': '2gj0', 'e_value': 1.2e-06, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m9o_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2m9o', 'e_value': 2.7e-06, 'percentage_identity': 61.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jwm_1', 'molecule_name': ['Kalata-B7'], 'pdb_id': '2jwm', 'e_value': 2.7e-06, 'percentage_identity': 61.5, 'result_sequence': None}], 'YGRKKRRQRRR': [{'chain_id': ['D'], 'entity_id': 4, 'entry_entity': '5l1z_4', 'molecule_name': ['Protein Tat'], 'pdb_id': '5l1z', 'e_value': 0.0054, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'M'], 'entity_id': 4, 'entry_entity': '4ogr_4', 'molecule_name': ['Protein Tat'], 'pdb_id': '4ogr', 'e_value': 0.0054, 'percentage_identity': 100.0, 'result_sequence': None}], 'RRIRPRPPRLPRPRPRPLPFPRPG': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 2.3e-05, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x5v_1', 'molecule_name': ['GD'], 'pdb_id': '5x5v', 'e_value': 0.091, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5x5w_1', 'molecule_name': ['GD'], 'pdb_id': '5x5w', 'e_value': 0.091, 'percentage_identity': 75.0, 'result_sequence': None}], 'MAKEFGIPAAVAGTVLNVVEAGGAVTTIVSILTAVGSGGLSLLAAAGRESIKAYLKKEIKKKGKRAVIAW': [{'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o82_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o82', 'e_value': 6.3e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o84_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o84', 'e_value': 6.3e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o83_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o83', 'e_value': 6.3e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e68_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1e68', 'e_value': 6.3e-26, 'percentage_identity': 98.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4rgd_1', 'molecule_name': ['AS-48A'], 'pdb_id': '4rgd', 'e_value': 1.7e-24, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8j_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8j', 'e_value': 0.00057, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mp8_1', 'molecule_name': ['Nkr-5-3b'], 'pdb_id': '2mp8', 'e_value': 0.0013, 'percentage_identity': 37.1, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8p_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8p', 'e_value': 0.0051, 'percentage_identity': 68.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8r_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8r', 'e_value': 0.038, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8s_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8s', 'e_value': 0.038, 'percentage_identity': 64.0, 'result_sequence': None}], 'LQFPVGRVHRLLRK': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4kha_2', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kha', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2xql_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2xql', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6k01_1', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k01', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hio', 'e_value': 3.4e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'FLSFPTTKTYFPHFDLSHGSAQVKGHGAKVAAALTKAVEHLDDLPGALSELSDLHAHKLRVDPVN': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ii1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ii1', 'e_value': 5.4e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 5.5e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 5.5e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 5.5e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 5.5e-29, 'percentage_identity': 100.0, 'result_sequence': None}], 'RNPRPVYIPQPRPPHPRL': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.096, 'percentage_identity': 71.4, 'result_sequence': None}], 'FLPVVAGLAAKVLPSIICAVTKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 1e-05, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 1e-05, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 1e-05, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4x_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4x', 'e_value': 0.02, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4v_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4v', 'e_value': 0.02, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r95_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r95', 'e_value': 0.02, 'percentage_identity': 56.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r96_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6r96', 'e_value': 0.02, 'percentage_identity': 56.5, 'result_sequence': None}], 'EWEPVQNGGSSYYMVPRIWA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tie_1', 'molecule_name': ['Trypsin inhibitor DE-3'], 'pdb_id': '1tie', 'e_value': 0.083, 'percentage_identity': 61.1, 'result_sequence': None}], 'VDKPPYLPRPTPPRRIYNNR': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezn_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '4ezn', 'e_value': 0.00039, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3dpp_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpp', 'e_value': 0.011, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '3dpq_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpq', 'e_value': 0.011, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.018, 'percentage_identity': 75.0, 'result_sequence': None}], 'QQCGRQAGNRRCPNNLCCSQFGYCGRTNEYCCTGFGCQSNCRRCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5wuz_1', 'molecule_name': ['Morintide mO1'], 'pdb_id': '5wuz', 'e_value': 6e-13, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4wp4_1', 'molecule_name': ['Hevein'], 'pdb_id': '4wp4', 'e_value': 1.8e-12, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q9b_1', 'molecule_name': ['Hevein'], 'pdb_id': '1q9b', 'e_value': 1.8e-12, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hev_1', 'molecule_name': ['Hevein'], 'pdb_id': '1hev', 'e_value': 1.8e-12, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wkx_1', 'molecule_name': ['Hevein'], 'pdb_id': '1wkx', 'e_value': 4.3e-12, 'percentage_identity': 64.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3iwr_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '3iwr', 'e_value': 1.5e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dkv_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '2dkv', 'e_value': 1.5e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 2.5e-10, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1uha_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1uha', 'e_value': 4.1e-10, 'percentage_identity': 58.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ulm_1', 'molecule_name': ['Lectin-D2'], 'pdb_id': '1ulm', 'e_value': 4.1e-10, 'percentage_identity': 58.7, 'result_sequence': None}], 'KWKSFIKKKTSKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.00071, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.00071, 'percentage_identity': 70.0, 'result_sequence': None}], 'GEILCNLCTGLINTLENLLTTKGADKVKDYISSLCNKASGFIATLCTKVLDFGIDKLIQLIEDKVDANAICAKIHAC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1of9_1', 'molecule_name': ['Pore-forming peptide ameobapore A'], 'pdb_id': '1of9', 'e_value': 4.6e-36, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G'], 'entity_id': 1, 'entry_entity': '6w1b_1', 'molecule_name': ['Pulmonary surfactant-associated protein B'], 'pdb_id': '6w1b', 'e_value': 0.077, 'percentage_identity': 20.8, 'result_sequence': None}], 'GRCRDDFRCWCTKRC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 1.6e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 2.7e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 5.7e-07, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 5.7e-07, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 5.7e-07, 'percentage_identity': 86.7, 'result_sequence': None}], 'KWKLFKKIGAVLKKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.058, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.058, 'percentage_identity': 100.0, 'result_sequence': None}], 'KPKGMTSSQWFKIQHMQPSPQACNSAMKNINKHTKRCKDLNTFLHEPFSSVAATCQTPKIACKNGDKNCHQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 8.5e-34, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 7.8e-17, 'percentage_identity': 55.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 7.8e-17, 'percentage_identity': 55.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 7.8e-17, 'percentage_identity': 55.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 7.8e-17, 'percentage_identity': 55.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 4.7e-16, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 4e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 4e-11, 'percentage_identity': 41.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 4e-11, 'percentage_identity': 41.1, 'result_sequence': None}], 'KKKKGIGKFLHSAKKFGKAFVGEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 9.2e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 9.5e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 2e-09, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 2.5e-08, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 4.3e-08, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 4.3e-08, 'percentage_identity': 87.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.0022, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.0022, 'percentage_identity': 93.3, 'result_sequence': None}], 'TLRGDERILSILRHQNLLKELQDLALQGAK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r2x_1', 'molecule_name': ['Vasostatin-1'], 'pdb_id': '6r2x', 'e_value': 4.2e-09, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLRKRLRKPRNKPKEKLKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 0.00019, 'percentage_identity': 90.0, 'result_sequence': None}], 'FKCWRWQWRWKKLGAKVFKRLEKLFSKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00044, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0049, 'percentage_identity': 53.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.033, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.054, 'percentage_identity': 80.0, 'result_sequence': None}], 'KWASLWNWFNITNWLWYIK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jau_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '1jau', 'e_value': 2.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jav_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '1jav', 'e_value': 2.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2pv6_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2pv6', 'e_value': 2.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m8m_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2m8m', 'e_value': 3.1e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2m8o_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '2m8o', 'e_value': 3.1e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P'], 'entity_id': 3, 'entry_entity': '5ghw_3', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '5ghw', 'e_value': 3.7e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '6dln_1', 'molecule_name': ['Transmembrane protein gp41'], 'pdb_id': '6dln', 'e_value': 4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3g9r_1', 'molecule_name': ['General control transcription factor GCN4', 'Transmembrane protein gp41'], 'pdb_id': '3g9r', 'e_value': 4.2e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'RWCVYAYVRVRGVRVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 2.4e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 3.2e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 3.2e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 2.2e-10, 'percentage_identity': 90.5, 'result_sequence': None}], 'GWKKFFKKAKKVGK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 0.044, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 0.045, 'percentage_identity': 71.4, 'result_sequence': None}], 'RTCQSQSHKFKGACFSDTNCDSVCRTENFPRGQCNQHHVERKCYCERDC': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6lcq_1', 'molecule_name': ['Defensin-like protein CAL1'], 'pdb_id': '6lcq', 'e_value': 7.2e-14, 'percentage_identity': 59.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nce_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '5nce', 'e_value': 6.3e-12, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 2.7e-11, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 6.9e-11, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 1.8e-10, 'percentage_identity': 51.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 3.1e-10, 'percentage_identity': 51.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ksk_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2ksk', 'e_value': 1.4e-06, 'percentage_identity': 34.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4uj0_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '4uj0', 'e_value': 2.7e-06, 'percentage_identity': 38.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 4.7e-06, 'percentage_identity': 34.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gps_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gps', 'e_value': 1.2e-05, 'percentage_identity': 38.8, 'result_sequence': None}], 'QLGELIQQGGQKIVEKIQKIGQRIRDFFSNLRPRQEA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.0023, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.0023, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.0023, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.0023, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.0023, 'percentage_identity': 41.7, 'result_sequence': None}], 'ATCDLASGFGVGSSLCAAHCIARRYRGGYCNSKAVCVCRN': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 8.9e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 2.6e-16, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 3.2e-15, 'percentage_identity': 82.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 7.7e-14, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 5e-13, 'percentage_identity': 72.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 1.4e-12, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.4e-12, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 8.1e-12, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 1.3e-07, 'percentage_identity': 55.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 1.3e-06, 'percentage_identity': 53.5, 'result_sequence': None}], 'RLGDILQKAREKIEGGLKKLVQKIKDFFGKFAPRTES': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 9.4e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 9.4e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 9.4e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 9.4e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 9.4e-06, 'percentage_identity': 52.8, 'result_sequence': None}], 'KKWGWLAWVEPAGEFLKGFGKGAIKEGNKDKWKNI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpk_1', 'molecule_name': ['Bacteriocin lactococcin-G subunit beta'], 'pdb_id': '2jpk', 'e_value': 4e-17, 'percentage_identity': 91.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpm_1', 'molecule_name': ['Bacteriocin lactococcin-G subunit beta'], 'pdb_id': '2jpm', 'e_value': 4e-17, 'percentage_identity': 91.4, 'result_sequence': None}], 'AMDPTKYYGNGVYCNSKKCWVDWGSASGCIGQTVVGGWLGGAIPGKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohm_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohm', 'e_value': 2.2e-08, 'percentage_identity': 52.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohn_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohn', 'e_value': 1.1e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1og7_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1og7', 'e_value': 1.1e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2a2b_1', 'molecule_name': ['Bacteriocin curvacin-A'], 'pdb_id': '2a2b', 'e_value': 2.8e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n4k_1', 'molecule_name': ['Enterocin-HF'], 'pdb_id': '2n4k', 'e_value': 1.1e-05, 'percentage_identity': 52.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ukz_1', 'molecule_name': ['Bacteriocin pediocin PA-1'], 'pdb_id': '5ukz', 'e_value': 1.4e-05, 'percentage_identity': 47.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2leu', 'e_value': 0.0078, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw6_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '1cw6', 'e_value': 0.0078, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '3leu', 'e_value': 0.0078, 'percentage_identity': 48.5, 'result_sequence': None}], 'KAVAAKKSPKKAKKPATPKKAAKSPKKVKKPAAAAKKAAKSPKKATKAAKPKAAKPKAAKAKKAAPKKK': [{'chain_id': ['U'], 'entity_id': 8, 'entry_entity': '7k5y_8', 'molecule_name': ['Histone H1.4'], 'pdb_id': '7k5y', 'e_value': 2.1e-08, 'percentage_identity': 68.1, 'result_sequence': None}], 'KWKSFIKKLASKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.0017, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.0017, 'percentage_identity': 70.0, 'result_sequence': None}], 'GVLDILKGAAKDLAGHVATKVI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 5.1e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 5.1e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 5.1e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 5.2e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 5.2e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 5.2e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 5.5e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 5.5e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 5.5e-07, 'percentage_identity': 77.3, 'result_sequence': None}], 'LLGDFFRKSKEKIGKEFKRIVQR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 5.7e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 7.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 7.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 7.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 7.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 7.6e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'AMDPTKYYGNGVYCNSKKCWVDFGQASGCIGQTVVGGWLGGAIPGKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohm_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohm', 'e_value': 6.9e-07, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2a2b_1', 'molecule_name': ['Bacteriocin curvacin-A'], 'pdb_id': '2a2b', 'e_value': 1.7e-06, 'percentage_identity': 52.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ohn_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1ohn', 'e_value': 3.3e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1og7_1', 'molecule_name': ['Bacteriocin sakacin-P'], 'pdb_id': '1og7', 'e_value': 3.3e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n4k_1', 'molecule_name': ['Enterocin-HF'], 'pdb_id': '2n4k', 'e_value': 0.00013, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ukz_1', 'molecule_name': ['Bacteriocin pediocin PA-1'], 'pdb_id': '5ukz', 'e_value': 0.00016, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '2leu', 'e_value': 0.07, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1cw6_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '1cw6', 'e_value': 0.07, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '3leu_1', 'molecule_name': ['Bacteriocin leucocin-A'], 'pdb_id': '3leu', 'e_value': 0.07, 'percentage_identity': 45.5, 'result_sequence': None}], 'AANFGPSVFTPEVHETWQKFLNVVVAALGKQYH': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '1spg_2', 'molecule_name': ['Hemoglobin subunit beta'], 'pdb_id': '1spg', 'e_value': 1.9e-11, 'percentage_identity': 78.1, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3bcq_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '3bcq', 'e_value': 8.5e-08, 'percentage_identity': 69.7, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '2r1h_2', 'molecule_name': ['Hemoglobin subunit beta-4'], 'pdb_id': '2r1h', 'e_value': 8.9e-07, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '3bom_2', 'molecule_name': ['Hemoglobin subunit beta-4'], 'pdb_id': '3bom', 'e_value': 8.9e-07, 'percentage_identity': 60.6, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6iyi_2', 'molecule_name': ['GLOBIN domain-containing protein'], 'pdb_id': '6iyi', 'e_value': 1.2e-05, 'percentage_identity': 51.5, 'result_sequence': None}], 'HTPTPTPICKSRSHEYKGRCIQDMDCNAACVKESESYTGGFCNGRPPFKQCFCTKPCKRERAAATLRWPGL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ksk_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2ksk', 'e_value': 5.8e-35, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 3.2e-06, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 3.8e-06, 'percentage_identity': 42.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4uj0_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '4uj0', 'e_value': 6.6e-06, 'percentage_identity': 42.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 1.3e-05, 'percentage_identity': 42.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gps_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gps', 'e_value': 1.9e-05, 'percentage_identity': 38.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6lcq_1', 'molecule_name': ['Defensin-like protein CAL1'], 'pdb_id': '6lcq', 'e_value': 2.9e-05, 'percentage_identity': 34.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4aaz_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4aaz', 'e_value': 3.8e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4ab0_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4ab0', 'e_value': 3.8e-05, 'percentage_identity': 40.8, 'result_sequence': None}, {'chain_id': ['H', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4cqk_1', 'molecule_name': ['Flower-specific defensin'], 'pdb_id': '4cqk', 'e_value': 3.8e-05, 'percentage_identity': 40.8, 'result_sequence': None}], 'LSFPTTKTYFPHFDLSHGSAQVKGHGAKVAAA': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '6ii1_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '6ii1', 'e_value': 1.5e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 1.6e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 1.6e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 1.6e-16, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 1.6e-16, 'percentage_identity': 100.0, 'result_sequence': None}], 'VTCDVLSFEAKGIAVNHSACALHAIALRKKGGSCQNGVCVARN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 6.3e-20, 'percentage_identity': 95.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.8e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 7e-06, 'percentage_identity': 51.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 8.2e-06, 'percentage_identity': 52.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 1.6e-05, 'percentage_identity': 59.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 3.8e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 3.8e-05, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 4.2e-05, 'percentage_identity': 38.1, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 7.4e-05, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 0.00035, 'percentage_identity': 46.5, 'result_sequence': None}], 'AWKKTIRQYLKNEIKKKWRKAVIAW': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8t_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8t', 'e_value': 2.7e-09, 'percentage_identity': 84.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8j_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8j', 'e_value': 1.3e-07, 'percentage_identity': 88.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8r_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8r', 'e_value': 2.9e-07, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8s_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8s', 'e_value': 2.9e-07, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8p_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8p', 'e_value': 3e-05, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4rgd_1', 'molecule_name': ['AS-48A'], 'pdb_id': '4rgd', 'e_value': 0.001, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o82_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o82', 'e_value': 0.001, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o84_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o84', 'e_value': 0.001, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o83_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o83', 'e_value': 0.001, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e68_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1e68', 'e_value': 0.001, 'percentage_identity': 60.0, 'result_sequence': None}], 'PRPPHPPRPPHPPRPPHPPRPPHPRL': [{'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4z1r_1', 'molecule_name': ['Collagen-like peptide'], 'pdb_id': '4z1r', 'e_value': 0.00049, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1cag_1', 'molecule_name': ['COLLAGEN-LIKE PEPTIDE'], 'pdb_id': '1cag', 'e_value': 0.00049, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1cgd_1', 'molecule_name': ['COLLAGEN-LIKE PEPTIDE'], 'pdb_id': '1cgd', 'e_value': 0.00049, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2cuo_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '2cuo', 'e_value': 0.00054, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3a08_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3a08', 'e_value': 0.00054, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3ah9_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '3ah9', 'e_value': 0.00054, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3a0a_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3a0a', 'e_value': 0.00054, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '2d3h_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '2d3h', 'e_value': 0.00054, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3b0s_1', 'molecule_name': ['Saimiri transformation-associated protein'], 'pdb_id': '3b0s', 'e_value': 0.00054, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '3a19_1', 'molecule_name': ['collagen-like peptide'], 'pdb_id': '3a19', 'e_value': 0.00054, 'percentage_identity': 66.7, 'result_sequence': None}], 'FLSTLLNVASNVVPTLICKITKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 0.044, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 0.044, 'percentage_identity': 45.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 0.044, 'percentage_identity': 45.8, 'result_sequence': None}], 'KAKAKAVSRSARAGLQFPVGRIHRHLK': [{'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6m4d_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '6m4d', 'e_value': 7.3e-12, 'percentage_identity': 96.3, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3waa_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '3waa', 'e_value': 8.2e-12, 'percentage_identity': 96.3, 'result_sequence': None}], 'RWCVYAYVRVRGVLSRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 1e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 1.4e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 1.4e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 9.5e-11, 'percentage_identity': 90.5, 'result_sequence': None}], 'STLHLVLRARGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mws_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2mws', 'e_value': 0.0052, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5jtj_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5jtj', 'e_value': 0.0052, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['B', 'C'], 'entity_id': 2, 'entry_entity': '2n3w_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2n3w', 'e_value': 0.0052, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbe_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbe', 'e_value': 0.0052, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nr2_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nr2', 'e_value': 0.0052, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nbd_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '2nbd', 'e_value': 0.0052, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5k9p_1', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5k9p', 'e_value': 0.0052, 'percentage_identity': 91.7, 'result_sequence': None}], 'GMWSKIKETAMAAAKEAAKAAGKTISDMIKQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k9b_1', 'molecule_name': ['Dermaseptin-DI1'], 'pdb_id': '2k9b', 'e_value': 0.00062, 'percentage_identity': 51.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jx6_1', 'molecule_name': ['Dermaseptin-DI1'], 'pdb_id': '2jx6', 'e_value': 0.00062, 'percentage_identity': 51.6, 'result_sequence': None}], 'VDKPPYLPRPKPPRRIYNNR': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezn_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '4ezn', 'e_value': 0.0029, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.012, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3dpp_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpp', 'e_value': 0.075, 'percentage_identity': 72.2, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '3dpq_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpq', 'e_value': 0.075, 'percentage_identity': 72.2, 'result_sequence': None}], 'FEVLPNFKHIQVKVFNHGEHIHHH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n1c_1', 'molecule_name': ['Tyrosinase_Cu-bd domain-containing protein'], 'pdb_id': '2n1c', 'e_value': 5.9e-06, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n30_1', 'molecule_name': ['Tyrosinase_Cu-bd domain-containing protein'], 'pdb_id': '2n30', 'e_value': 6.2e-06, 'percentage_identity': 91.3, 'result_sequence': None}], 'KILRGVSKKIMRTFLRRISKDILTGKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nkl_1', 'molecule_name': ['Antimicrobial peptide NK-lysin'], 'pdb_id': '1nkl', 'e_value': 4.1e-09, 'percentage_identity': 85.2, 'result_sequence': None}], 'LYKKIIKKLLES': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4r9w_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4r9w', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'C', 'B', 'A'], 'entity_id': 1, 'entry_entity': '4r9y_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4r9y', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['R', 'U', 'I', 'C', 'X', 'O', 'F', 'L'], 'entity_id': 3, 'entry_entity': '4rau_3', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '4rau', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1pfn_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1pfn', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1f9r_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1f9r', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1f9s_1', 'molecule_name': ['Platelet factor 4'], 'pdb_id': '1f9s', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}], 'RCLPAGKTCVRGPMRVPCCGSCSQNKCT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1y29_1', 'molecule_name': ['Omega-theraphotoxin-Hs1a'], 'pdb_id': '1y29', 'e_value': 2.6e-07, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v5a_1', 'molecule_name': ['Covalitoxin-I'], 'pdb_id': '1v5a', 'e_value': 9.7e-07, 'percentage_identity': 53.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6saa_1', 'molecule_name': ['U1-theraphotoxin-Pf3'], 'pdb_id': '6saa', 'e_value': 3.7e-06, 'percentage_identity': 58.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2f2j_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2f2j', 'e_value': 0.063, 'percentage_identity': 56.5, 'result_sequence': None}], 'KWKSFKKKLTSKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 9.5e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 9.5e-05, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.015, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.015, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.02, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.02, 'percentage_identity': 70.0, 'result_sequence': None}], 'YRGGYTGPIPRPPPIGRPPFRPVCNACYRLSVSDARNCCIKFGSCCHLVK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ueo_1', 'molecule_name': ['Penaeidin-3a'], 'pdb_id': '1ueo', 'e_value': 2.5e-08, 'percentage_identity': 49.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 2.7e-06, 'percentage_identity': 46.0, 'result_sequence': None}], 'GLFGVLAKVAAHVVPAIAKHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mmj_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mmj', 'e_value': 1.7e-06, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn8_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn8', 'e_value': 1.7e-06, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn9_1', 'molecule_name': ['Maculatin-1.1'], 'pdb_id': '2mn9', 'e_value': 1.7e-06, 'percentage_identity': 85.7, 'result_sequence': None}], 'KWCVYAYVKVKGVLVKYKKCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 6.1e-10, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 8.1e-10, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 8.1e-10, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 5.9e-09, 'percentage_identity': 66.7, 'result_sequence': None}], 'FTCDVLSAEGGFRGVSIKLNHAACAAHCLYLKKRGGYCNDKAVCVCRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 6.8e-10, 'percentage_identity': 60.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 1.8e-09, 'percentage_identity': 58.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 7.2e-09, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 2.1e-08, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 2.9e-08, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 1.7e-07, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 1.7e-07, 'percentage_identity': 58.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 7.2e-07, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.0001, 'percentage_identity': 45.7, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.00012, 'percentage_identity': 52.2, 'result_sequence': None}], 'KWKKFIKKIGIGAVLKVLTTGLPALKLTKK': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.0017, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.0018, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.0018, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.0018, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.0018, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.0096, 'percentage_identity': 81.0, 'result_sequence': None}], 'FLSLIPHAIKAVGVHAKHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq0_1', 'molecule_name': ['Phylloseptin-H1'], 'pdb_id': '2jq0', 'e_value': 0.00042, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpy_1', 'molecule_name': ['Phylloseptin-H2'], 'pdb_id': '2jpy', 'e_value': 0.00077, 'percentage_identity': 68.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq1_1', 'molecule_name': ['Phyllospetin-H3'], 'pdb_id': '2jq1', 'e_value': 0.0019, 'percentage_identity': 72.2, 'result_sequence': None}], 'FLSMIPHIVSGVAALAKHL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq0_1', 'molecule_name': ['Phylloseptin-H1'], 'pdb_id': '2jq0', 'e_value': 0.00065, 'percentage_identity': 61.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq1_1', 'molecule_name': ['Phyllospetin-H3'], 'pdb_id': '2jq1', 'e_value': 0.0016, 'percentage_identity': 61.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpy_1', 'molecule_name': ['Phylloseptin-H2'], 'pdb_id': '2jpy', 'e_value': 0.024, 'percentage_identity': 47.4, 'result_sequence': None}], 'FLGALFKVASKVLPSVFCAITKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 0.0049, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 0.0049, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 0.0049, 'percentage_identity': 50.0, 'result_sequence': None}], 'STIGKLPKWIIKTVNKFTKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dtb_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2dtb', 'e_value': 9e-05, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kam_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '2kam', 'e_value': 9e-05, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dtc_1', 'molecule_name': ['Delta-hemolysin'], 'pdb_id': '1dtc', 'e_value': 9e-05, 'percentage_identity': 85.0, 'result_sequence': None}], 'FLPFLLSALPKVFCFFSKKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 0.056, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 0.056, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 0.056, 'percentage_identity': 50.0, 'result_sequence': None}], 'GIGKFLHSAKKFGKAAVGEIMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 1.4e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 1.4e-10, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 4.7e-10, 'percentage_identity': 91.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 5.8e-08, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 1.1e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 1.1e-07, 'percentage_identity': 82.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.0071, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.0071, 'percentage_identity': 100.0, 'result_sequence': None}], 'GWGSFFKKAAAVGKAVGKAALTAYL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 8.9e-09, 'percentage_identity': 88.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 9.2e-09, 'percentage_identity': 88.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 1.3e-07, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 2.2e-07, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 0.021, 'percentage_identity': 48.0, 'result_sequence': None}], 'GLQFPVGRVHRLLRK': [{'chain_id': ['I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2xql_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2xql', 'e_value': 4.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4kha_2', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kha', 'e_value': 5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6k01_1', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k01', 'e_value': 5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hio_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '1hio', 'e_value': 5.1e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'GVGKFLHSAKKFGQALASEIMKS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 8.6e-08, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 8.9e-08, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 7e-07, 'percentage_identity': 69.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 2.3e-06, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 4.2e-06, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 4.2e-06, 'percentage_identity': 65.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.0096, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.0096, 'percentage_identity': 91.7, 'result_sequence': None}], 'GIINTLQKYYARVRGGRAAVLSALPKEEQIGKASTRGRKCARRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 3.1e-17, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.00038, 'percentage_identity': 46.5, 'result_sequence': None}], 'GRFRRLRKKTRKRLKKIGKVLKLI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ket_1', 'molecule_name': ['Cathelicidin-6'], 'pdb_id': '2ket', 'e_value': 0.053, 'percentage_identity': 50.0, 'result_sequence': None}], 'GIGGVLLSAGKAALKGLTKVLAEKYAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 3.7e-10, 'percentage_identity': 96.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.067, 'percentage_identity': 60.0, 'result_sequence': None}], 'AAHHIARPIVHVGKTIHRLVTG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 4.4e-08, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 1.2e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 1.2e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 1.2e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 1.2e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 1.2e-06, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 8.4e-06, 'percentage_identity': 85.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 0.0065, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 0.0065, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 0.0065, 'percentage_identity': 60.0, 'result_sequence': None}], 'GIPCGESCVWIPCLTSAIGCSCKSKVCYRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 1.8e-15, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 1.8e-15, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 8.6e-15, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 3.6e-12, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 6.4e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 1.6e-10, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 3.2e-10, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 6.7e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 9.9e-07, 'percentage_identity': 67.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 2.4e-06, 'percentage_identity': 61.5, 'result_sequence': None}], 'FINKAGKLQSQLRTTVVAAAAFLDAFQKVA': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2d1l_1', 'molecule_name': ['Protein MTSS 1'], 'pdb_id': '2d1l', 'e_value': 4.3e-13, 'percentage_identity': 100.0, 'result_sequence': None}], 'AWKEKIRKKLKNEIKKKWRKAVIAW': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8t_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8t', 'e_value': 3e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8r_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8r', 'e_value': 2.6e-09, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8s_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8s', 'e_value': 2.6e-09, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8p_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8p', 'e_value': 2.2e-07, 'percentage_identity': 92.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '6o8j_1', 'molecule_name': ['Circular bacteriocin, circularin A/uberolysin family'], 'pdb_id': '6o8j', 'e_value': 2.5e-05, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4rgd_1', 'molecule_name': ['AS-48A'], 'pdb_id': '4rgd', 'e_value': 0.014, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o82_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o82', 'e_value': 0.014, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1o84_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o84', 'e_value': 0.014, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1o83_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1o83', 'e_value': 0.014, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e68_1', 'molecule_name': ['AS-48A'], 'pdb_id': '1e68', 'e_value': 0.014, 'percentage_identity': 60.0, 'result_sequence': None}], 'GFLNTAMNTVTNLAGTLMDKAKCKIRGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.0087, 'percentage_identity': 44.4, 'result_sequence': None}], 'GSKKPVPIIYCQRRTAKCQRM': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xo9_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo9', 'e_value': 3.8e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo4_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo4', 'e_value': 3.8e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gd5_2', 'molecule_name': ['Thanatin'], 'pdb_id': '6gd5', 'e_value': 3.8e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6aab_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6aab', 'e_value': 3.8e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8tfv_1', 'molecule_name': ['Thanatin'], 'pdb_id': '8tfv', 'e_value': 3.8e-09, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo3_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo3', 'e_value': 2.5e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xoa_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xoa', 'e_value': 2.5e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6afq_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6afq', 'e_value': 2.5e-08, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xok_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xok', 'e_value': 3.8e-07, 'percentage_identity': 85.0, 'result_sequence': None}], 'GAGALAKFLAKKVAKTVAKQAAKQGAKYVVNKQME': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k38_1', 'molecule_name': ['Cupiennin-1a'], 'pdb_id': '2k38', 'e_value': 1e-12, 'percentage_identity': 94.3, 'result_sequence': None}], 'MAGGKAGKDSGKAKAKAVSRSARAGLQFPVGRIHRHLK': [{'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6m4d_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '6m4d', 'e_value': 3.3e-16, 'percentage_identity': 97.4, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '3waa_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '3waa', 'e_value': 3.8e-16, 'percentage_identity': 97.4, 'result_sequence': None}], 'GIPCGESCVFIPCITAAIGCSCKSKVCYRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 3.8e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 3.8e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 1.8e-13, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 1e-11, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 1.3e-11, 'percentage_identity': 85.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 4.1e-11, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 4.1e-11, 'percentage_identity': 81.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 3.3e-07, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 3.3e-07, 'percentage_identity': 65.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 8.3e-07, 'percentage_identity': 67.9, 'result_sequence': None}], 'AGYLLGKINLKPLAALAKKIL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1smz_1', 'molecule_name': ['Transportan in bicellar solution with [DMPC]/[DHPC]=0.33'], 'pdb_id': '1smz', 'e_value': 1.4e-07, 'percentage_identity': 95.2, 'result_sequence': None}], 'GKPRPYPPRPPPHPRPIRV': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.061, 'percentage_identity': 76.5, 'result_sequence': None}], 'VLSAADKGNVKAAWGKVGGHAAEYGAEA': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 1.1e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 1.1e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 1.1e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 1.1e-12, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ciu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ciu', 'e_value': 1.1e-12, 'percentage_identity': 100.0, 'result_sequence': None}], 'RRIRPRPPRLPRPRPRPLPYPRP': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 1.9e-05, 'percentage_identity': 81.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x5v_1', 'molecule_name': ['GD'], 'pdb_id': '5x5v', 'e_value': 0.073, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '5x5w_1', 'molecule_name': ['GD'], 'pdb_id': '5x5w', 'e_value': 0.073, 'percentage_identity': 75.0, 'result_sequence': None}], 'ASHLPSDFTPAVHASLDKFLANVSTVLTSKYR': [{'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pia_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pia', 'e_value': 8.6e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qss_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qss', 'e_value': 8.6e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3pi9_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3pi9', 'e_value': 8.6e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '2qsp_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '2qsp', 'e_value': 8.6e-15, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'C'], 'entity_id': 1, 'entry_entity': '3ciu_1', 'molecule_name': ['Hemoglobin subunit alpha'], 'pdb_id': '3ciu', 'e_value': 8.6e-15, 'percentage_identity': 100.0, 'result_sequence': None}], 'GKPRPYSPKPTSHPRPIRV': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.07, 'percentage_identity': 64.7, 'result_sequence': None}], 'FKRIVQRIKDFLR': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 0.0004, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.0006, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.0006, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.0006, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.0006, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.0006, 'percentage_identity': 100.0, 'result_sequence': None}], 'GRRPRPRPRP': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4jwe_2', 'molecule_name': ['Cathelicidin-3'], 'pdb_id': '4jwe', 'e_value': 0.052, 'percentage_identity': 100.0, 'result_sequence': None}], 'NPLIPAIYIGATVGPSVWAYLVALVGAAAVTAANIRRASSDNHSCAGNRGWCRSKCFRHEYVDTYYSAVCGRYFCCRSR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rng_1', 'molecule_name': ['Big defensin'], 'pdb_id': '2rng', 'e_value': 1.4e-40, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qbk_1', 'molecule_name': ['Defensin_big domain-containing protein'], 'pdb_id': '6qbk', 'e_value': 1.3e-11, 'percentage_identity': 61.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qbl_1', 'molecule_name': ['Defensin_big domain-containing protein'], 'pdb_id': '6qbl', 'e_value': 2.2e-11, 'percentage_identity': 49.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rq2_1', 'molecule_name': ['Big defensin'], 'pdb_id': '2rq2', 'e_value': 2.8e-09, 'percentage_identity': 100.0, 'result_sequence': None}], 'AVVDILKGAAKDIAGHLASKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 3.1e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 3.1e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 3.1e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 6.6e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 6.6e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 6.6e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 4.6e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 4.6e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 4.6e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'AAAAGSCVWGAVNYTSDCNGECKRRGYKGGHCGSFANVNCWCET': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 2.2e-21, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 3.6e-20, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 9.8e-20, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 1.5e-19, 'percentage_identity': 92.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 1.8e-19, 'percentage_identity': 92.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lkb_1', 'molecule_name': ['Sodium channel neurotoxin MeuNaTxalpha-5'], 'pdb_id': '2lkb', 'e_value': 0.00066, 'percentage_identity': 37.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bmr_1', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '1bmr', 'e_value': 0.005, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fh3_1', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '1fh3', 'e_value': 0.005, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '7k18_2', 'molecule_name': ['Alpha-like toxin Lqh3'], 'pdb_id': '7k18', 'e_value': 0.005, 'percentage_identity': 35.9, 'result_sequence': None}], 'FIFHIIKGLFHAGKMIHGLVTRRRH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5h2s_1', 'molecule_name': ['Piscidin-4'], 'pdb_id': '5h2s', 'e_value': 1.3e-05, 'percentage_identity': 70.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 0.011, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 0.011, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 0.012, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 0.012, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 0.012, 'percentage_identity': 57.1, 'result_sequence': None}], 'KKLKLALAKLALLWKALALKLKKA': [{'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l96_1', 'molecule_name': ['LAK160-P7'], 'pdb_id': '2l96', 'e_value': 1.7e-09, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l99_1', 'molecule_name': ['LAK160-P10'], 'pdb_id': '2l99', 'e_value': 1.7e-09, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2l9a_1', 'molecule_name': ['LAK160-P12'], 'pdb_id': '2l9a', 'e_value': 1.7e-09, 'percentage_identity': 95.8, 'result_sequence': None}], 'LLPALISWIKRKRQQ': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 5.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 5.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 5.7e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'GIGKLFLHAAKKFAKAFVAEKMNS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 1.2e-08, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 7.1e-08, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['H', 'E', 'F', 'G'], 'entity_id': 2, 'entry_entity': '5cgn_2', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgn', 'e_value': 7.1e-08, 'percentage_identity': 87.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 7.9e-07, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 8.1e-07, 'percentage_identity': 79.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 2.2e-05, 'percentage_identity': 70.8, 'result_sequence': None}], 'KILRGVSKKIMRRISKDILTGKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nkl_1', 'molecule_name': ['Antimicrobial peptide NK-lysin'], 'pdb_id': '1nkl', 'e_value': 0.00019, 'percentage_identity': 74.1, 'result_sequence': None}], 'EPAEDLARYYSALRHYINLITRQRY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f8p_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1f8p', 'e_value': 3.4e-11, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ron_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1ron', 'e_value': 5.9e-11, 'percentage_identity': 95.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fvn_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1fvn', 'e_value': 6.7e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1icy_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1icy', 'e_value': 8.8e-10, 'percentage_identity': 91.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tz4_1', 'molecule_name': ['Pancreatic hormone', 'Neuropeptide Y'], 'pdb_id': '1tz4', 'e_value': 5.1e-08, 'percentage_identity': 83.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oon_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oon', 'e_value': 2e-07, 'percentage_identity': 73.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rlk_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2rlk', 'e_value': 7.7e-07, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qbf_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1qbf', 'e_value': 7.7e-07, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ru5_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ru5', 'e_value': 7.7e-07, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oop_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oop', 'e_value': 7.7e-07, 'percentage_identity': 72.7, 'result_sequence': None}], 'NLCERASLTWTGNCGNTGHCDTQCRNWESAKHGACHKRGNWKCFCYFNC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bk8_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1bk8', 'e_value': 6.8e-17, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kpy_1', 'molecule_name': ['Major pollen allergen Art v 1'], 'pdb_id': '2kpy', 'e_value': 2.9e-10, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ayj_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1ayj', 'e_value': 1.6e-08, 'percentage_identity': 53.1, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '2m8b_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2m8b', 'e_value': 3.8e-08, 'percentage_identity': 51.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2r_1', 'molecule_name': ['Defensin-like protein 2'], 'pdb_id': '2n2r', 'e_value': 1.3e-07, 'percentage_identity': 49.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2q_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '2n2q', 'e_value': 4.8e-06, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jkz_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '1jkz', 'e_value': 2.4e-05, 'percentage_identity': 42.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 6.9e-05, 'percentage_identity': 40.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 8.2e-05, 'percentage_identity': 40.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 0.00054, 'percentage_identity': 38.3, 'result_sequence': None}], 'INLKILARLAKKIL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6duu_1', 'molecule_name': ['Mastoparan-L'], 'pdb_id': '6duu', 'e_value': 0.012, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1smz_1', 'molecule_name': ['Transportan in bicellar solution with [DMPC]/[DHPC]=0.33'], 'pdb_id': '1smz', 'e_value': 0.017, 'percentage_identity': 85.7, 'result_sequence': None}], 'KKAAHVGKHVGKAAKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 9.1e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 9.5e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 0.0019, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 0.0035, 'percentage_identity': 71.4, 'result_sequence': None}], 'GVVDILKGAAKAIAGHLASKVMNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9y_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9y', 'e_value': 4.2e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua8_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5ua8', 'e_value': 4.2e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9s_1', 'molecule_name': ['Ocellatin-F1'], 'pdb_id': '5u9s', 'e_value': 4.2e-10, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua7_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5ua7', 'e_value': 8.9e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9x_1', 'molecule_name': ['Ocellatin-K1'], 'pdb_id': '5u9x', 'e_value': 8.9e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9r_1', 'molecule_name': ['Ocellatin-LB2'], 'pdb_id': '5u9r', 'e_value': 8.9e-09, 'percentage_identity': 95.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9q_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9q', 'e_value': 6.3e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ua6_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5ua6', 'e_value': 6.3e-08, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5u9v_1', 'molecule_name': ['Ocellatin-LB1'], 'pdb_id': '5u9v', 'e_value': 6.3e-08, 'percentage_identity': 95.5, 'result_sequence': None}], 'AQCGAQGGGATCPGGLCCSQWGWCGSTPKYCGAGCQSNCK': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4mpi_1', 'molecule_name': ['Probable inactive chitinase-like protein LaCIC'], 'pdb_id': '4mpi', 'e_value': 9.2e-14, 'percentage_identity': 76.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3iwr_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '3iwr', 'e_value': 5.2e-12, 'percentage_identity': 71.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dkv_1', 'molecule_name': ['Chitinase 2'], 'pdb_id': '2dkv', 'e_value': 5.2e-12, 'percentage_identity': 71.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4wp4_1', 'molecule_name': ['Hevein'], 'pdb_id': '4wp4', 'e_value': 1.1e-11, 'percentage_identity': 68.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q9b_1', 'molecule_name': ['Hevein'], 'pdb_id': '1q9b', 'e_value': 1.1e-11, 'percentage_identity': 68.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hev_1', 'molecule_name': ['Hevein'], 'pdb_id': '1hev', 'e_value': 1.1e-11, 'percentage_identity': 68.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1wkx_1', 'molecule_name': ['Hevein'], 'pdb_id': '1wkx', 'e_value': 1.3e-11, 'percentage_identity': 68.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdi_1', 'molecule_name': ['Chitin-binding type-1 domain-containing protein'], 'pdb_id': '5xdi', 'e_value': 2.1e-11, 'percentage_identity': 74.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ehh_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehh', 'e_value': 9.8e-11, 'percentage_identity': 63.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ehd_1', 'molecule_name': ['Agglutinin isolectin V'], 'pdb_id': '1ehd', 'e_value': 9.8e-11, 'percentage_identity': 63.2, 'result_sequence': None}], 'GVPCAESCVWIPCTVTALLGCSCKDKVCYLN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 1.7e-11, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 1.5e-10, 'percentage_identity': 67.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 1.5e-10, 'percentage_identity': 67.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 5.6e-10, 'percentage_identity': 67.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 5.6e-10, 'percentage_identity': 84.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 7e-10, 'percentage_identity': 64.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 2.6e-09, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 2.7e-08, 'percentage_identity': 60.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 3.6e-08, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 1e-07, 'percentage_identity': 82.6, 'result_sequence': None}], 'GFGCPFNLNECHAHCLSIGRKFGFCAGPLRATCTCGKQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 1.1e-20, 'percentage_identity': 97.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xa6_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '5xa6', 'e_value': 1.2e-09, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ru0_1', 'molecule_name': ['INVERT_DEFENSINS domain-containing protein'], 'pdb_id': '2ru0', 'e_value': 1.4e-07, 'percentage_identity': 54.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 6.7e-07, 'percentage_identity': 58.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lt8_1', 'molecule_name': ['Fungal defensin eurocin'], 'pdb_id': '2lt8', 'e_value': 4e-06, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b68_1', 'molecule_name': ['Defensin Cg-Defm'], 'pdb_id': '2b68', 'e_value': 2.3e-05, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6k50_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '6k50', 'e_value': 0.0011, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '3e7u_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7u', 'e_value': 0.0033, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['L'], 'entity_id': 1, 'entry_entity': '3e7r_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '3e7r', 'e_value': 0.0033, 'percentage_identity': 46.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zfu_1', 'molecule_name': ['Fungal defensin plectasin'], 'pdb_id': '1zfu', 'e_value': 0.0033, 'percentage_identity': 46.3, 'result_sequence': None}], 'TLRGDERILSILRHQNLLKE': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r2x_1', 'molecule_name': ['Vasostatin-1'], 'pdb_id': '6r2x', 'e_value': 2e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'LTCEIDRSLCLLHCRLKGYLRAYCSQQKVCRCVQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6px8_1', 'molecule_name': ['U-Asilidin(12)-Dg3b'], 'pdb_id': '6px8', 'e_value': 0.00032, 'percentage_identity': 38.2, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.0021, 'percentage_identity': 46.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 0.0036, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.017, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.021, 'percentage_identity': 42.9, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 0.052, 'percentage_identity': 39.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 0.052, 'percentage_identity': 35.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 0.055, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.066, 'percentage_identity': 35.7, 'result_sequence': None}], 'FSCDVLSFQSKWVSPNHSACAVRCLAQRRKGGKCKNGDCVCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 3.8e-12, 'percentage_identity': 63.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rty_1', 'molecule_name': ['navitoxin'], 'pdb_id': '2rty', 'e_value': 1.8e-11, 'percentage_identity': 93.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 1.6e-09, 'percentage_identity': 55.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 2.1e-09, 'percentage_identity': 53.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 4.8e-05, 'percentage_identity': 45.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.00013, 'percentage_identity': 47.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.0003, 'percentage_identity': 48.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 0.00067, 'percentage_identity': 40.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.0015, 'percentage_identity': 46.5, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 0.017, 'percentage_identity': 39.0, 'result_sequence': None}], 'KWKSFIKKLTLKFLHSKKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.026, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.026, 'percentage_identity': 65.0, 'result_sequence': None}], 'RMCKTPSGKFKGYCVNNTNCKNVCRTEGFPTGSCDFHVAGRKCYCYKPCP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nce_1', 'molecule_name': ['Defensin-1'], 'pdb_id': '5nce', 'e_value': 9.8e-24, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6lcq_1', 'molecule_name': ['Defensin-like protein CAL1'], 'pdb_id': '6lcq', 'e_value': 4.6e-10, 'percentage_identity': 55.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dmz_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6dmz', 'e_value': 4.7e-08, 'percentage_identity': 48.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '7c2p_1', 'molecule_name': ['Plant defensing Egk'], 'pdb_id': '7c2p', 'e_value': 7.9e-08, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n2r_1', 'molecule_name': ['Defensin-like protein 2'], 'pdb_id': '2n2r', 'e_value': 2.4e-07, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '2m8b_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2m8b', 'e_value': 3.4e-07, 'percentage_identity': 46.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ayj_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1ayj', 'e_value': 1.1e-06, 'percentage_identity': 44.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gpt_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gpt', 'e_value': 1.5e-06, 'percentage_identity': 44.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '7c31_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '7c31', 'e_value': 1.5e-06, 'percentage_identity': 44.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gps_1', 'molecule_name': ['Defensin-like protein 1'], 'pdb_id': '1gps', 'e_value': 4.4e-06, 'percentage_identity': 40.8, 'result_sequence': None}], 'PRPPRLPRPRPRPLPFPRPGPRPIPRPLPFP': [{'chain_id': ['a0'], 'entity_id': 1, 'entry_entity': '6ncl_1', 'molecule_name': ['P14'], 'pdb_id': '6ncl', 'e_value': 8.4e-05, 'percentage_identity': 48.4, 'result_sequence': None}], 'GGAGHVPEYFVGIGTPISFYG': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4cu4_2', 'molecule_name': ['Microcin J25'], 'pdb_id': '4cu4', 'e_value': 7.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q71_1', 'molecule_name': ['Microcin J25'], 'pdb_id': '1q71', 'e_value': 7.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1pp5_1', 'molecule_name': ['Microcin J25'], 'pdb_id': '1pp5', 'e_value': 7.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['M'], 'entity_id': 6, 'entry_entity': '6n60_6', 'molecule_name': ['Microcin J25'], 'pdb_id': '6n60', 'e_value': 7.4e-10, 'percentage_identity': 100.0, 'result_sequence': None}], 'KVPVGAIKKGGKAIKTGLGVVGAAGTAHEVYSHIRNRH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1x22_1', 'molecule_name': ['Moricin'], 'pdb_id': '1x22', 'e_value': 1.8e-05, 'percentage_identity': 52.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jr8_1', 'molecule_name': ['Moricin'], 'pdb_id': '2jr8', 'e_value': 0.0004, 'percentage_identity': 48.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kv4_1', 'molecule_name': ['Moricin-1'], 'pdb_id': '1kv4', 'e_value': 0.017, 'percentage_identity': 37.8, 'result_sequence': None}], 'KPPQFTWAQWFETQHINMTSQQSTNAMQVINNYQRRSKNQNTFLL': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 5.1e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 5.1e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi4_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi4', 'e_value': 5.1e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi3_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi3', 'e_value': 5.1e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi5_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi5', 'e_value': 5.1e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2c05_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c05', 'e_value': 5.1e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1gqv_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1gqv', 'e_value': 5.1e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hi2_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1hi2', 'e_value': 5.1e-21, 'percentage_identity': 95.6, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2c01_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2c01', 'e_value': 5.1e-21, 'percentage_identity': 95.6, 'result_sequence': None}], 'CVLIGQRCDNDRGPRCCSGQGNCVPLPFLGGVCAV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mfs_1', 'molecule_name': ['Ep-AMP1'], 'pdb_id': '2mfs', 'e_value': 5.7e-18, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v91_1', 'molecule_name': ['Delta-amaurobitoxin-Pl1b'], 'pdb_id': '1v91', 'e_value': 0.045, 'percentage_identity': 44.8, 'result_sequence': None}], 'KRIVQRIKDFLRNLVPRTES': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 4.2e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 5.5e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 5.5e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 5.5e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 5.5e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 5.5e-08, 'percentage_identity': 100.0, 'result_sequence': None}], 'RRICRCRIGRCLGLEVYFGVCFLHGRLARRCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kra_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '6kra', 'e_value': 5e-08, 'percentage_identity': 54.5, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C', 'E', 'F'], 'entity_id': 1, 'entry_entity': '5cuj_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cuj', 'e_value': 3.3e-07, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5cum_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '5cum', 'e_value': 3.3e-07, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4e86_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e86', 'e_value': 1.4e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e82_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e82', 'e_value': 1.4e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rbx_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbx', 'e_value': 1.7e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mit_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2mit', 'e_value': 1.7e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e83_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e83', 'e_value': 1.7e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '1zmp_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '1zmp', 'e_value': 1.7e-06, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3i5w_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '3i5w', 'e_value': 1.7e-06, 'percentage_identity': 56.7, 'result_sequence': None}], 'HSSGYTRPLRKPSRPIFIRPIGCDVCYGIPSSTARLCCFRYGDCCHL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 6.4e-25, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ueo_1', 'molecule_name': ['Penaeidin-3a'], 'pdb_id': '1ueo', 'e_value': 1.5e-06, 'percentage_identity': 43.9, 'result_sequence': None}], 'LVQRGRFGRFLRKIR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gdl_1', 'molecule_name': ['myeloid antimicrobial peptide 27'], 'pdb_id': '2gdl', 'e_value': 1.9e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'AKKVFKRLEKLFSKIQNDK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0g_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0g', 'e_value': 3.7e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5k_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5k', 'e_value': 3.6e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 3.6e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5l', 'e_value': 8.4e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 1.5e-05, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 1.5e-05, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 0.00014, 'percentage_identity': 89.5, 'result_sequence': None}], 'AKKVFKRLEKLFSKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ot0_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1ot0', 'e_value': 3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0g_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0g', 'e_value': 3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0o_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0o', 'e_value': 3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0j_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0j', 'e_value': 3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p0l', 'e_value': 3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5k_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5k', 'e_value': 0.00038, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p5l_1', 'molecule_name': ['50S ribosomal protein L1'], 'pdb_id': '1p5l', 'e_value': 0.00098, 'percentage_identity': 93.3, 'result_sequence': None}], 'RKLPDAPGMHTWGGGGKCSTRGRKCCRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.00032, 'percentage_identity': 100.0, 'result_sequence': None}], 'FLKLLKKLAAKFLPTIICKISYKC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4i_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4i', 'e_value': 0.0012, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4k_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4k', 'e_value': 0.0012, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6g4u_1', 'molecule_name': ['Brevinin-1BYa'], 'pdb_id': '6g4u', 'e_value': 0.0012, 'percentage_identity': 50.0, 'result_sequence': None}], 'AACSDRAHGHICESFKSFCKDSGRNGVKLRANCKKTCGLC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg4_1', 'molecule_name': ['Antimicrobial peptide aurelin'], 'pdb_id': '2lg4', 'e_value': 6.4e-21, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n6b_1', 'molecule_name': ['Hui1'], 'pdb_id': '2n6b', 'e_value': 0.075, 'percentage_identity': 42.5, 'result_sequence': None}], 'LRKFRNKIKEKLKKIGQKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 1e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'KWKSFIKKLTSKFLHLAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.029, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.029, 'percentage_identity': 65.0, 'result_sequence': None}], 'LEGRSISRIDELIQQRQEKD': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3j32_1', 'molecule_name': ['Hemocyanin isoform 1'], 'pdb_id': '3j32', 'e_value': 0.0047, 'percentage_identity': 70.0, 'result_sequence': None}], 'RWCVYAYVRVRGVSVRYRRCW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jsb_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '2jsb', 'e_value': 1.7e-11, 'percentage_identity': 95.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jni_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2jni', 'e_value': 2.3e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2l8x_1', 'molecule_name': ['Arenicin-2'], 'pdb_id': '2l8x', 'e_value': 2.3e-11, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5m9u_1', 'molecule_name': ['Arenicin-1'], 'pdb_id': '5m9u', 'e_value': 1.6e-10, 'percentage_identity': 90.5, 'result_sequence': None}], 'FLSLIPHIVSGVASLAKHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpy_1', 'molecule_name': ['Phylloseptin-H2'], 'pdb_id': '2jpy', 'e_value': 0.00093, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq0_1', 'molecule_name': ['Phylloseptin-H1'], 'pdb_id': '2jq0', 'e_value': 0.0013, 'percentage_identity': 61.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq1_1', 'molecule_name': ['Phyllospetin-H3'], 'pdb_id': '2jq1', 'e_value': 0.0031, 'percentage_identity': 61.1, 'result_sequence': None}], 'IGKEFKRIVQRIKDFLRNL': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 2.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 3e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 3e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'TSGKYPNCRYKEKRQNKSYVVACKPPQKKDSQQFHLVPVHLDRVL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2hky_1', 'molecule_name': ['Ribonuclease 7'], 'pdb_id': '2hky', 'e_value': 1.8e-22, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4x09_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '4x09', 'e_value': 1.2e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5oab_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '5oab', 'e_value': 1.2e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv7_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv7', 'e_value': 1.2e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mv6_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6mv6', 'e_value': 1.2e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6enp_1', 'molecule_name': ['Ribonuclease K6'], 'pdb_id': '6enp', 'e_value': 1.2e-10, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k2a_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '1k2a', 'e_value': 1.3e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '2bex_2', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bex', 'e_value': 1.3e-05, 'percentage_identity': 47.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2bzz_1', 'molecule_name': ['Non-secretory ribonuclease'], 'pdb_id': '2bzz', 'e_value': 1.3e-05, 'percentage_identity': 47.5, 'result_sequence': None}], 'VDKPPYLPRPRPPRRIYNRN': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezn_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '4ezn', 'e_value': 0.0001, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '3dpp_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpp', 'e_value': 0.0027, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'C', 'G'], 'entity_id': 2, 'entry_entity': '3dpq_2', 'molecule_name': ['Pyrrhocoricin'], 'pdb_id': '3dpq', 'e_value': 0.0027, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.0044, 'percentage_identity': 81.2, 'result_sequence': None}]}

In [52]:
%store res_5k

Stored 'res_5k' (dict)


In [61]:
len(res_5k)

173

## 5000-5250 

In [33]:
sec_5250=sec_1[5000:5250]

In [34]:
res_5250_raw=multiple_sequence_search(sec_5250)

1 sequences of 250
2 sequences of 250
3 sequences of 250
4 sequences of 250
5 sequences of 250
6 sequences of 250
7 sequences of 250
8 sequences of 250
9 sequences of 250
10 sequences of 250
11 sequences of 250
12 sequences of 250
13 sequences of 250
14 sequences of 250
15 sequences of 250
16 sequences of 250
17 sequences of 250
18 sequences of 250
19 sequences of 250
20 sequences of 250
21 sequences of 250
22 sequences of 250
23 sequences of 250
24 sequences of 250
25 sequences of 250
26 sequences of 250
27 sequences of 250
28 sequences of 250
29 sequences of 250
30 sequences of 250
31 sequences of 250
32 sequences of 250
33 sequences of 250
34 sequences of 250
35 sequences of 250
36 sequences of 250
37 sequences of 250
38 sequences of 250
39 sequences of 250
40 sequences of 250
41 sequences of 250
42 sequences of 250
43 sequences of 250
44 sequences of 250
45 sequences of 250
46 sequences of 250
47 sequences of 250
48 sequences of 250
49 sequences of 250
50 sequences of 250
51 sequen

In [35]:
res_5250=results_to_dict(res_5250_raw)

In [41]:
%store res_5250 res_5250_raw

Stored 'res_5250' (dict)
Stored 'res_5250_raw' (dict)


In [60]:
len(res_5250)

91

## 5250-5500 (colab)

In [37]:
res_5500={'GIMSIVKDVAKNAAKEAAKGALSTLSCKLAKTC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 5.3e-06, 'percentage_identity': 54.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.026, 'percentage_identity': 51.7, 'result_sequence': None}], 'EKCLRWQWRMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00027, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0059, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.0059, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.0059, 'percentage_identity': 76.9, 'result_sequence': None}], 'GLLNTFKDWAISIAKGAGKGVLTTLSCKLDKSC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 1.7e-05, 'percentage_identity': 48.6, 'result_sequence': None}], 'GIFSKLAGKKLKNLLISGLKNVGKEVGMDVVRTGIDIAGCKIKGEC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n6m_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '2n6m', 'e_value': 0.00058, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdj_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '5xdj', 'e_value': 0.00058, 'percentage_identity': 95.0, 'result_sequence': None}], 'RGLRRLGRKIAHGVKKYGPTVLRIIRIAG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fry_1', 'molecule_name': ['Cathelin-related peptide SC5'], 'pdb_id': '1fry', 'e_value': 4.8e-13, 'percentage_identity': 100.0, 'result_sequence': None}], 'FKCRRWQWAMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 5.1e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.3e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00068, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00068, 'percentage_identity': 80.0, 'result_sequence': None}], 'AKIPIKAIKTVGKAVGKGLRAINIASTANDVFNFLKPKKRKH': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kv4_1', 'molecule_name': ['Moricin-1'], 'pdb_id': '1kv4', 'e_value': 8.6e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1x22_1', 'molecule_name': ['Moricin'], 'pdb_id': '1x22', 'e_value': 1.2e-11, 'percentage_identity': 72.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jr8_1', 'molecule_name': ['Moricin'], 'pdb_id': '2jr8', 'e_value': 1.2e-11, 'percentage_identity': 68.3, 'result_sequence': None}], 'RRWSFRVSYRGFSYRKSR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 0.00028, 'percentage_identity': 73.7, 'result_sequence': None}], 'APRKNVRWCAISLPEWSKCYQWQRRMRKLGAPSITCIRRTS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 6.1e-21, 'percentage_identity': 97.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 3.9e-16, 'percentage_identity': 78.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 1.1e-15, 'percentage_identity': 79.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 2.2e-15, 'percentage_identity': 79.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 1.3e-11, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 1.3e-11, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1l5t_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1l5t', 'e_value': 7.3e-11, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h44_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h44', 'e_value': 7.3e-11, 'percentage_identity': 56.4, 'result_sequence': None}, {'chain_id': ['X', 'S'], 'entity_id': 2, 'entry_entity': '2j4u_2', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2j4u', 'e_value': 8.7e-11, 'percentage_identity': 56.1, 'result_sequence': None}], 'KWKSFIKKLLSKFLHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.00095, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.00095, 'percentage_identity': 70.0, 'result_sequence': None}], 'FKCRRWQWRAKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 8.4e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.9e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00087, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00087, 'percentage_identity': 80.0, 'result_sequence': None}], 'KFFRKLKKSVKKRAK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mwt_1', 'molecule_name': ['Crotalicidin'], 'pdb_id': '2mwt', 'e_value': 0.0076, 'percentage_identity': 80.0, 'result_sequence': None}], 'TCRYWCKTPENQTYCCEDEREIPSKVGLKPGKCPPVRPVCPPTRGFFEPPKTCSNDGSCYGADKCCFDRCLGEHVCKPIQTRG': [{'chain_id': ['D', 'B'], 'entity_id': 2, 'entry_entity': '4doq_2', 'molecule_name': ['Antileukoproteinase'], 'pdb_id': '4doq', 'e_value': 0.0094, 'percentage_identity': 36.5, 'result_sequence': None}, {'chain_id': ['I'], 'entity_id': 2, 'entry_entity': '2z7f_2', 'molecule_name': ['Antileukoproteinase'], 'pdb_id': '2z7f', 'e_value': 0.0098, 'percentage_identity': 36.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rel_1', 'molecule_name': ['Elafin'], 'pdb_id': '2rel', 'e_value': 0.013, 'percentage_identity': 32.7, 'result_sequence': None}, {'chain_id': ['I'], 'entity_id': 2, 'entry_entity': '1fle_2', 'molecule_name': ['Elafin'], 'pdb_id': '1fle', 'e_value': 0.013, 'percentage_identity': 32.7, 'result_sequence': None}, {'chain_id': ['H', 'R', 'P', 'I', 'K', 'D', 'M', 'Q', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '6atu_1', 'molecule_name': ['Elafin'], 'pdb_id': '6atu', 'e_value': 0.013, 'percentage_identity': 32.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3ngg_1', 'molecule_name': ['Omwaprin-a'], 'pdb_id': '3ngg', 'e_value': 0.016, 'percentage_identity': 46.2, 'result_sequence': None}], 'STALPVNSPMNKGDTEVMKCIVEVISDTLSKPSPMPVSQECFETLRGDERILSILRHQNLLKELQDLALQGAKERAHQQKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6r2x_1', 'molecule_name': ['Vasostatin-1'], 'pdb_id': '6r2x', 'e_value': 4.4e-07, 'percentage_identity': 100.0, 'result_sequence': None}], 'GIPCGESCVWIPCISAALGCSCKNKVCYRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 6.6e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 6.6e-14, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 2.9e-13, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 5.5e-13, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 4.5e-11, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 1.3e-10, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 3.8e-10, 'percentage_identity': 73.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 2.6e-08, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 7.5e-07, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 5.1e-06, 'percentage_identity': 64.3, 'result_sequence': None}], 'RKCLRWQWRMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00074, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.016, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.016, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.016, 'percentage_identity': 76.9, 'result_sequence': None}], 'AKCLRWQWEMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0011, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.029, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.029, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.029, 'percentage_identity': 69.2, 'result_sequence': None}], 'AKCLRWQWRMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0003, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0064, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.0064, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.0064, 'percentage_identity': 76.9, 'result_sequence': None}], 'GWKSVFRKAKKVGKTVGGLALDHYLG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 7.7e-05, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 7.9e-05, 'percentage_identity': 64.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 0.00017, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 0.0012, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rzc_1', 'molecule_name': ['Pleurocidin-like peptide WF4'], 'pdb_id': '6rzc', 'e_value': 0.0078, 'percentage_identity': 40.0, 'result_sequence': None}], 'EKCLRWQWRMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00081, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.018, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.018, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.018, 'percentage_identity': 76.9, 'result_sequence': None}], 'RKCLRWQWEMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0053, 'percentage_identity': 69.2, 'result_sequence': None}], 'FKCRRWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 6.2e-08, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.5e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 7.5e-05, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 7.5e-05, 'percentage_identity': 86.7, 'result_sequence': None}], 'GFMDTAKNVAKNVAVTLIDKLRCKVTGGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.0063, 'percentage_identity': 50.0, 'result_sequence': None}], 'GLGSVFGRLARILGRVIPKVAKKLGPKVAKVLPKVMKEAIPMAVEMAKSQEEQQPQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5x3l_1', 'molecule_name': ['M-myrmeciitoxin-Mp1e'], 'pdb_id': '5x3l', 'e_value': 0.00029, 'percentage_identity': 100.0, 'result_sequence': None}], 'SKCYQWQWRMRKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 6.4e-05, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 0.00034, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00077, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00077, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0018, 'percentage_identity': 78.6, 'result_sequence': None}], 'FKCARWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 9.4e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 2.1e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00014, 'percentage_identity': 86.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00014, 'percentage_identity': 86.7, 'result_sequence': None}], 'AKCLRWQWAMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.002, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.049, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.049, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.049, 'percentage_identity': 69.2, 'result_sequence': None}], 'LVNQLGISKSLANTILGAIAVGNLASWVLALVPGPGWATKAALATAETIVKHEGKAAAIAW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mp8_1', 'molecule_name': ['Nkr-5-3b'], 'pdb_id': '2mp8', 'e_value': 0.0026, 'percentage_identity': 34.4, 'result_sequence': None}], 'FKRIVQRIKDFLRNLV': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 2e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 2.8e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 2.8e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 2.8e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 2.8e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 2.8e-05, 'percentage_identity': 100.0, 'result_sequence': None}], 'RCRFCCRCCPRMRGCGICCRF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1s6w_1', 'molecule_name': ['Hepcidin'], 'pdb_id': '1s6w', 'e_value': 1.3e-08, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1m4e_1', 'molecule_name': ['Hepcidin-20'], 'pdb_id': '1m4e', 'e_value': 0.0019, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['R', 'U', 'P', 'Q', 'T', 'S'], 'entity_id': 1, 'entry_entity': '4qae_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '4qae', 'e_value': 0.002, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 3, 'entry_entity': '3h0t_3', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '3h0t', 'e_value': 0.002, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kef_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '2kef', 'e_value': 0.002, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1m4f_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '1m4f', 'e_value': 0.002, 'percentage_identity': 57.9, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 1, 'entry_entity': '6wbv_1', 'molecule_name': ['Hepcidin-25'], 'pdb_id': '6wbv', 'e_value': 0.002, 'percentage_identity': 57.9, 'result_sequence': None}], 'RKCLRWQWRMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00031, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0064, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.0064, 'percentage_identity': 76.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.0064, 'percentage_identity': 76.9, 'result_sequence': None}], 'GRKSDCFRKSGFCAFLKCPSLTLISGKCSRFYLCCKRIR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 8.3e-05, 'percentage_identity': 45.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.012, 'percentage_identity': 42.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg5_1', 'molecule_name': ['Gallinacin-2'], 'pdb_id': '2lg5', 'e_value': 0.012, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.047, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nlf_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlf', 'e_value': 0.047, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.047, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lg6_1', 'molecule_name': ['Gallinacin-2'], 'pdb_id': '2lg6', 'e_value': 0.047, 'percentage_identity': 44.8, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlq_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlq', 'e_value': 0.058, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nls_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nls', 'e_value': 0.058, 'percentage_identity': 39.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlg_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlg', 'e_value': 0.058, 'percentage_identity': 39.4, 'result_sequence': None}], 'ILGPILGLVSNALGGLL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 0.00016, 'percentage_identity': 76.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 0.00016, 'percentage_identity': 76.5, 'result_sequence': None}], 'QVYKGGYARPIPRPPPFVRPLPGGPIGPYNGCPVSCRGISFSQARSCCSRLGRCCHVGKGYSG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ueo_1', 'molecule_name': ['Penaeidin-3a'], 'pdb_id': '1ueo', 'e_value': 1.9e-29, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 8e-05, 'percentage_identity': 43.9, 'result_sequence': None}], 'IIGPVLGLVGSALGGLLKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 5.9e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 5.9e-07, 'percentage_identity': 100.0, 'result_sequence': None}], 'RKCLRWQWEMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0014, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.034, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.034, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.034, 'percentage_identity': 69.2, 'result_sequence': None}], 'IIGPVLGMVGSALGGLLKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 1.3e-06, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 1.3e-06, 'percentage_identity': 95.0, 'result_sequence': None}], 'KWKSFIKKLTTAVKKVLTTGLPALIS': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.08, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.08, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.082, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.082, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.082, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['D', 'C'], 'entity_id': 1, 'entry_entity': '6o4m_1', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.082, 'percentage_identity': 93.3, 'result_sequence': None}], 'YPPKPESPGEDASPEEMNKYLTALRHYINLVTRQRY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dez_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2dez', 'e_value': 9.2e-14, 'percentage_identity': 77.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2rlk_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2rlk', 'e_value': 2.3e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1qbf_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1qbf', 'e_value': 2.3e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ru5_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ru5', 'e_value': 2.3e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ruu_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '1ruu', 'e_value': 2.3e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oop_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oop', 'e_value': 4.7e-13, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2df0_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2df0', 'e_value': 2.9e-12, 'percentage_identity': 78.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2oon_1', 'molecule_name': ['Peptide YY'], 'pdb_id': '2oon', 'e_value': 3e-12, 'percentage_identity': 72.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ron_1', 'molecule_name': ['Neuropeptide Y'], 'pdb_id': '1ron', 'e_value': 7.7e-12, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2na5_1', 'molecule_name': ['Peptide YY(3-36)'], 'pdb_id': '2na5', 'e_value': 1.1e-11, 'percentage_identity': 78.1, 'result_sequence': None}], 'GLFSVVTGVLKAVGKNVAKNVGGSLLEQLKCKKISGGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.0067, 'percentage_identity': 51.5, 'result_sequence': None}], 'YQWQRRMRKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 0.021, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLLRKGGEKIGEKLKKIGQKIKNFFQKLVPQPE': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 0.00065, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 0.00065, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 0.00065, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 0.00065, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 0.00065, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 0.024, 'percentage_identity': 65.4, 'result_sequence': None}], 'LIGPVLGLVGSALGGLLKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap7_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap7', 'e_value': 1.5e-06, 'percentage_identity': 95.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ap8_1', 'molecule_name': ['Bombinin-H5'], 'pdb_id': '2ap8', 'e_value': 1.5e-06, 'percentage_identity': 95.0, 'result_sequence': None}], 'SYVGDCGSNGGSCVSSYCPYGNRLNYFCPLGRTCCRHAYV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.0063, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 0.049, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.057, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlp_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlp', 'e_value': 0.07, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj5_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1kj5', 'e_value': 0.07, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4s_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1e4s', 'e_value': 0.07, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1ijv_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1ijv', 'e_value': 0.07, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1iju_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '1iju', 'e_value': 0.07, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.085, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nle_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nle', 'e_value': 0.085, 'percentage_identity': 36.4, 'result_sequence': None}], 'GIGASILSAGKSALKGLAKGLAEHFAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 2.7e-06, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.056, 'percentage_identity': 60.0, 'result_sequence': None}], 'AKCRRWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 5.2e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 8.9e-05, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 8.9e-05, 'percentage_identity': 92.9, 'result_sequence': None}], 'FKCRRWQWRMKKAGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 6.5e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.4e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00067, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00067, 'percentage_identity': 80.0, 'result_sequence': None}], 'GLMSLFRGGVLKTAGKHIFKNVGGSLLDQAKCKITGEC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.034, 'percentage_identity': 42.1, 'result_sequence': None}], 'FKARRWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 3.1e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 7.3e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.0036, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.0036, 'percentage_identity': 80.0, 'result_sequence': None}], 'AKCLRWQWEMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0042, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.1, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.1, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.1, 'percentage_identity': 69.2, 'result_sequence': None}], 'GIGSAILSAGKSALKGLAKGLAEHFAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 1.1e-06, 'percentage_identity': 74.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.059, 'percentage_identity': 60.0, 'result_sequence': None}], 'ACYCRIGACVSGERLTGACGLNGRIYRLCCR': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e82_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e82', 'e_value': 4.1e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3i5w_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '3i5w', 'e_value': 4.1e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4rbx_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbx', 'e_value': 5.1e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mit_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2mit', 'e_value': 5.1e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e83_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e83', 'e_value': 5.1e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '1zmp_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '1zmp', 'e_value': 5.1e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxz_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2lxz', 'e_value': 5.1e-09, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4rbw_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbw', 'e_value': 1e-08, 'percentage_identity': 60.0, 'result_sequence': None}, {'chain_id': ['H', 'D', 'A', 'B', 'C', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '4e86_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e86', 'e_value': 2.6e-08, 'percentage_identity': 56.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4lbb_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '4lbb', 'e_value': 3.9e-08, 'percentage_identity': 56.7, 'result_sequence': None}], 'GIFSKLGRKKIKNLLISGLKNVGKEVGMDVVRTGIDIAGCKIKGEC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2n6m_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '2n6m', 'e_value': 0.0074, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xdj_1', 'molecule_name': ['Esculentin-1A'], 'pdb_id': '5xdj', 'e_value': 0.0074, 'percentage_identity': 90.0, 'result_sequence': None}], 'QCIGNGGRCNENVGPPYCCSGFCLRQPGQGYGYCKNR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dkc_1', 'molecule_name': ['Antimicrobial peptide 1'], 'pdb_id': '1dkc', 'e_value': 5.8e-11, 'percentage_identity': 66.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xbd_1', 'molecule_name': ['pB1'], 'pdb_id': '5xbd', 'e_value': 5.9e-06, 'percentage_identity': 54.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1q3j_1', 'molecule_name': ['Antimicrobial peptide Alo-3'], 'pdb_id': '1q3j', 'e_value': 1.6e-05, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1v91_1', 'molecule_name': ['Delta-amaurobitoxin-Pl1b'], 'pdb_id': '1v91', 'e_value': 0.014, 'percentage_identity': 42.1, 'result_sequence': None}], 'QLINSPVTCMSYGGSCQRSCNGGFRLGGHCGHPKIRCCRRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0014, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.017, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.017, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.017, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.017, 'percentage_identity': 35.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.04, 'percentage_identity': 36.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.073, 'percentage_identity': 44.4, 'result_sequence': None}], 'GILDTLKQFAKGVGKWLVKGAAQGVLSTVSCKLAKTC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 7.7e-16, 'percentage_identity': 97.3, 'result_sequence': None}], 'KKLGAPSITCVRRAFA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 1.4e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 6.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00042, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 0.0052, 'percentage_identity': 92.3, 'result_sequence': None}], 'AREASKSLIGTASCTCRRAWICRWGERHSGKCIDQKGSTYRLCCRR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mxq_1', 'molecule_name': ['Alpha-defensin 1'], 'pdb_id': '2mxq', 'e_value': 1.6e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '4rbw_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4rbw', 'e_value': 0.024, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3i5w_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '3i5w', 'e_value': 0.043, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mit_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2mit', 'e_value': 0.053, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e83_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e83', 'e_value': 0.053, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['D', 'A', 'C', 'B'], 'entity_id': 1, 'entry_entity': '1zmp_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '1zmp', 'e_value': 0.053, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxz_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '2lxz', 'e_value': 0.053, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3lvx_1', 'molecule_name': ['Neutrophil defensin 1'], 'pdb_id': '3lvx', 'e_value': 0.092, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4e82_1', 'molecule_name': ['HD5(63-94)'], 'pdb_id': '4e82', 'e_value': 0.096, 'percentage_identity': 47.1, 'result_sequence': None}], 'SLGNFFRKARKKIGEEFKRIVQRIKDFLQHLIPRTEA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 1.6e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 1.6e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 1.6e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 1.6e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 1.6e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 2.2e-06, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lmf_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2lmf', 'e_value': 9e-05, 'percentage_identity': 77.3, 'result_sequence': None}], 'GIGAAILSAGKSALKGLAKGLAEHF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 1.9e-05, 'percentage_identity': 72.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6hz2_1', 'molecule_name': ['Maximin-3'], 'pdb_id': '6hz2', 'e_value': 0.05, 'percentage_identity': 60.0, 'result_sequence': None}], 'FKCRRWQWRMKALGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 3.1e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 7.7e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00039, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00039, 'percentage_identity': 80.0, 'result_sequence': None}], 'SKCRQWQWKIRRTNP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5id5_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5id5', 'e_value': 2e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5i4g_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5i4g', 'e_value': 2e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.04, 'percentage_identity': 54.5, 'result_sequence': None}], 'GIGKFLKKAKKFGKAFVKILKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lsa_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2lsa', 'e_value': 0.00031, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mag_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '2mag', 'e_value': 0.00032, 'percentage_identity': 88.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '5cgn_1', 'molecule_name': ['D-Ala-Magainin Derivative'], 'pdb_id': '5cgn', 'e_value': 0.0039, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4mgp_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '4mgp', 'e_value': 0.0039, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5cgo_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '5cgo', 'e_value': 0.0068, 'percentage_identity': 82.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1dum_1', 'molecule_name': ['Magainin-2'], 'pdb_id': '1dum', 'e_value': 0.0068, 'percentage_identity': 76.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k98_1', 'molecule_name': ['MSI-594'], 'pdb_id': '2k98', 'e_value': 0.049, 'percentage_identity': 70.0, 'result_sequence': None}], 'ASVVKTTIKASKKLCKGATLTCGCNITGKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1w9n_1', 'molecule_name': ['Lantibiotic epilancin 15X'], 'pdb_id': '1w9n', 'e_value': 7.7e-11, 'percentage_identity': 80.0, 'result_sequence': None}], 'WNPFKELERAGQRVRDAIISAGPAVATVAQATALAK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2la2_1', 'molecule_name': ['Cecropin'], 'pdb_id': '2la2', 'e_value': 1.1e-07, 'percentage_identity': 55.6, 'result_sequence': None}], 'DPVTCLKSGAICHPVFCPRRYKQIGTCGLPGTKCCKKPN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 7.1e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 7.1e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 7.1e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 7.1e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 3.4e-19, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 2.6e-15, 'percentage_identity': 91.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 7.9e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0003, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.016, 'percentage_identity': 46.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nlf_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlf', 'e_value': 0.028, 'percentage_identity': 41.7, 'result_sequence': None}], 'FKCRRWQWRMKKLAA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 4.5e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.1e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00055, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00055, 'percentage_identity': 80.0, 'result_sequence': None}], 'KWFNEKSIQNKIDEKIGKNFLGGMAKAVVHKLAKNEFMCVANVDMTKSCDTHCQKASGEKGYCHGTKCKCGVPLSY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ipo_1', 'molecule_name': ['Hge36'], 'pdb_id': '5ipo', 'e_value': 1e-11, 'percentage_identity': 64.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5jyh_1', 'molecule_name': ['Hge36'], 'pdb_id': '5jyh', 'e_value': 6.7e-10, 'percentage_identity': 63.6, 'result_sequence': None}], 'KWKLFKKIGIGAVLKVLT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}], 'FQWQRNMRKV': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 0.0015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 0.0015, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1l5t_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1l5t', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lct_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lct', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h44_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h44', 'e_value': 0.01, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfi_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfi', 'e_value': 0.022, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bka_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1bka', 'e_value': 0.022, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfg_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfg', 'e_value': 0.022, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lcf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lcf', 'e_value': 0.022, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b0l_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b0l', 'e_value': 0.022, 'percentage_identity': 100.0, 'result_sequence': None}], 'ATCDLLSGTGINHSACAAHCLLRGNRGGYCNGKAVCVCRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 1.1e-20, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 3.3e-20, 'percentage_identity': 97.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 1.5e-19, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny8_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny8', 'e_value': 2e-13, 'percentage_identity': 72.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2nz3_1', 'molecule_name': ['Defensin'], 'pdb_id': '2nz3', 'e_value': 7.1e-13, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3g_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3g', 'e_value': 2.1e-12, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['X'], 'entity_id': 1, 'entry_entity': '2ny9_1', 'molecule_name': ['Defensin'], 'pdb_id': '2ny9', 'e_value': 5.2e-11, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 1.1e-09, 'percentage_identity': 62.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 7.9e-08, 'percentage_identity': 56.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3e_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3e', 'e_value': 8.1e-08, 'percentage_identity': 54.5, 'result_sequence': None}], 'GRFKRFRKKFKKLFKKLS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ket_1', 'molecule_name': ['Cathelicidin-6'], 'pdb_id': '2ket', 'e_value': 2.2e-06, 'percentage_identity': 100.0, 'result_sequence': None}], 'GIPCAESCVWIPPCTITALMGCSCKNNVCYNN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 1.2e-10, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 2.6e-09, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 2.6e-09, 'percentage_identity': 78.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 2.6e-09, 'percentage_identity': 68.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 1.2e-08, 'percentage_identity': 65.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 3.6e-08, 'percentage_identity': 72.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 3.2e-07, 'percentage_identity': 65.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1df6_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1df6', 'e_value': 5e-07, 'percentage_identity': 76.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 5.1e-07, 'percentage_identity': 69.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 7.7e-07, 'percentage_identity': 59.4, 'result_sequence': None}], 'QVYKGGYTRPIPRPPPFVRPLPGGPIGPYNGCPVSCRGISFSQARSCCSRLGRCCHVGKGYS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ueo_1', 'molecule_name': ['Penaeidin-3a'], 'pdb_id': '1ueo', 'e_value': 3.4e-29, 'percentage_identity': 98.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 2.1e-05, 'percentage_identity': 45.6, 'result_sequence': None}], 'EFELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5ki9_1', 'molecule_name': ['Beta-defensin 104'], 'pdb_id': '5ki9', 'e_value': 9.7e-23, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mjv_1', 'molecule_name': ['Human beta-defensin'], 'pdb_id': '6mjv', 'e_value': 0.0038, 'percentage_identity': 50.0, 'result_sequence': None}], 'QRYFCRVRGGRCAALTCLPRETQIGRCSVKGRKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 9.3e-16, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 1.5e-07, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.0057, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.0062, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.0062, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.0062, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.0062, 'percentage_identity': 40.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.014, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6kra_1', 'molecule_name': ['Neutrophil defensin 4'], 'pdb_id': '6kra', 'e_value': 0.02, 'percentage_identity': 42.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6qet_1', 'molecule_name': ['Gallinacin-11'], 'pdb_id': '6qet', 'e_value': 0.047, 'percentage_identity': 38.2, 'result_sequence': None}], 'INLKAIAALAKKLLG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6duu_1', 'molecule_name': ['Mastoparan-L'], 'pdb_id': '6duu', 'e_value': 0.0015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1smz_1', 'molecule_name': ['Transportan in bicellar solution with [DMPC]/[DHPC]=0.33'], 'pdb_id': '1smz', 'e_value': 0.0021, 'percentage_identity': 85.7, 'result_sequence': None}], 'RRWQWRMKKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00044, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.012, 'percentage_identity': 100.0, 'result_sequence': None}], 'HSSGYTRPLPKPSRPIFIRPIGCDVCYGIPSSTARLCCFRYGDCCHR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1xv3_1', 'molecule_name': ['Penaeidin-4d'], 'pdb_id': '1xv3', 'e_value': 6.2e-23, 'percentage_identity': 97.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ueo_1', 'molecule_name': ['Penaeidin-3a'], 'pdb_id': '1ueo', 'e_value': 2.7e-07, 'percentage_identity': 46.4, 'result_sequence': None}], 'SKCYQWQRRMRKLGAPSITCIRRTS': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 9.2e-13, 'percentage_identity': 96.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 1.5e-08, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 1.6e-08, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 6.3e-08, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['X', 'S'], 'entity_id': 2, 'entry_entity': '2j4u_2', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2j4u', 'e_value': 6.4e-08, 'percentage_identity': 62.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.4e-07, 'percentage_identity': 77.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 3.5e-07, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 3.5e-07, 'percentage_identity': 56.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1dtz_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1dtz', 'e_value': 4.2e-07, 'percentage_identity': 62.5, 'result_sequence': None}], 'SKCYQWQRRMRKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jw1_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1jw1', 'e_value': 4.9e-06, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 0.0054, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 0.0054, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0054, 'percentage_identity': 71.4, 'result_sequence': None}, {'chain_id': ['X', 'S'], 'entity_id': 2, 'entry_entity': '2j4u_2', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '2j4u', 'e_value': 0.016, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1l5t_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1l5t', 'e_value': 0.029, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lct_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lct', 'e_value': 0.029, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h44_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h44', 'e_value': 0.029, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.041, 'percentage_identity': 71.4, 'result_sequence': None}], 'RGFRKHFNKLVKKVKHTISETAHVAKDTAVIAGSGAAVVAAT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1zrx_1', 'molecule_name': ['Stomoxyn'], 'pdb_id': '1zrx', 'e_value': 3.1e-17, 'percentage_identity': 100.0, 'result_sequence': None}], 'DFGCARGMIFVCMRRCARMYPGSTGYCQGFRCMCDTMIPIRRPPFIMG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jzb_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jzb', 'e_value': 0.054, 'percentage_identity': 34.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1jza_1', 'molecule_name': ['Toxin CsEv2'], 'pdb_id': '1jza', 'e_value': 0.054, 'percentage_identity': 34.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kja_1', 'molecule_name': ['Beta-toxin Cn5'], 'pdb_id': '2kja', 'e_value': 0.079, 'percentage_identity': 32.7, 'result_sequence': None}], 'GILDTLKQFAKGVGKFLVKGAAQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 7.3e-09, 'percentage_identity': 95.7, 'result_sequence': None}], 'GILDTLKQFAKGVGKWLVKGAAQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 5.5e-08, 'percentage_identity': 95.7, 'result_sequence': None}], 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLR': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6djw_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '6djw', 'e_value': 8.1e-27, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4rf0_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '4rf0', 'e_value': 8.2e-27, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '5khy_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '5khy', 'e_value': 8.2e-27, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3kw5_2', 'molecule_name': ['Ubiquitin'], 'pdb_id': '3kw5', 'e_value': 8.2e-27, 'percentage_identity': 100.0, 'result_sequence': None}], 'GLFDKLKSLVSDF': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '2mp0_2', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '2mp0', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['P', 'M', 'N', 'O'], 'entity_id': 4, 'entry_entity': '4jbw_4', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '4jbw', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1o2f_1', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '1o2f', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '1gla_1', 'molecule_name': ['PTS system glucose-specific EIIA component'], 'pdb_id': '1gla', 'e_value': 0.016, 'percentage_identity': 100.0, 'result_sequence': None}], 'FKCRAWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 9.2e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 2.1e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.001, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.001, 'percentage_identity': 80.0, 'result_sequence': None}], 'GWKKWFNRAKKVGKTVGGLAVDHYLG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ls9_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '2ls9', 'e_value': 0.0063, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z64_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '1z64', 'e_value': 0.0065, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsf_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsf', 'e_value': 0.0082, 'percentage_identity': 52.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6rsg_1', 'molecule_name': ['Pleurocidin'], 'pdb_id': '6rsg', 'e_value': 0.083, 'percentage_identity': 44.0, 'result_sequence': None}], 'GIINTLQKYYCRVRGGRCAVLSCLPKEEQIGKCSTRGRKCCRRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 1.6e-22, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 5.5e-10, 'percentage_identity': 58.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.017, 'percentage_identity': 48.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 0.031, 'percentage_identity': 43.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.056, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.061, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.061, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.061, 'percentage_identity': 36.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.061, 'percentage_identity': 36.4, 'result_sequence': None}], 'FKCRRWQWRMAKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 2.4e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 7e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00036, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00036, 'percentage_identity': 80.0, 'result_sequence': None}], 'GIMDTLKNLAKTAGKGALQSLVKMASCKLSGQC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 0.00019, 'percentage_identity': 48.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6wpo_1', 'molecule_name': ['Hylaseptin-4'], 'pdb_id': '6wpo', 'e_value': 0.036, 'percentage_identity': 66.7, 'result_sequence': None}], 'GIMDTLKNLAKTAGKGALQSLLNKASCKLSGQC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2g9l_1', 'molecule_name': ['Gaegurin-4'], 'pdb_id': '2g9l', 'e_value': 5.2e-05, 'percentage_identity': 51.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6wpo_1', 'molecule_name': ['Hylaseptin-4'], 'pdb_id': '6wpo', 'e_value': 0.036, 'percentage_identity': 66.7, 'result_sequence': None}], 'RKCLRWQWAMRKVGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0025, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.058, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.058, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.058, 'percentage_identity': 69.2, 'result_sequence': None}], 'GIGALSAKGALKGLAKGLAEHFAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mhw_1', 'molecule_name': ['Maximin-4'], 'pdb_id': '2mhw', 'e_value': 0.044, 'percentage_identity': 70.4, 'result_sequence': None}], 'RQWQSKIRRT': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5id5_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5id5', 'e_value': 0.0022, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5i4g_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '5i4g', 'e_value': 0.0022, 'percentage_identity': 100.0, 'result_sequence': None}], 'RRWCFRVCYRGFCYRKCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2b5k_1', 'molecule_name': ['Polyphemusin-1'], 'pdb_id': '2b5k', 'e_value': 2.5e-09, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7ald_1', 'molecule_name': ['BRICHOS domain-containing protein'], 'pdb_id': '7ald', 'e_value': 0.0041, 'percentage_identity': 62.5, 'result_sequence': None}], 'FACRRWQWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 4e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 9.7e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00048, 'percentage_identity': 92.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00048, 'percentage_identity': 92.3, 'result_sequence': None}], 'KWKLFKKIGIGAVLKVLTTGLPALIS': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 3.3e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 3.4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 3.4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 3.4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 3.4e-05, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.021, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.021, 'percentage_identity': 85.7, 'result_sequence': None}], 'FLSLIPHAINAVSALANHG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq1_1', 'molecule_name': ['Phyllospetin-H3'], 'pdb_id': '2jq1', 'e_value': 7.6e-09, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq0_1', 'molecule_name': ['Phylloseptin-H1'], 'pdb_id': '2jq0', 'e_value': 1.9e-06, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpy_1', 'molecule_name': ['Phylloseptin-H2'], 'pdb_id': '2jpy', 'e_value': 1.2e-05, 'percentage_identity': 83.3, 'result_sequence': None}], 'GVIPCGESCVFIPCISTLLGCSCKNKVCYRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2eri_1', 'molecule_name': ['Circulin-B'], 'pdb_id': '2eri', 'e_value': 5.3e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '7kpd_1', 'molecule_name': ['Acyclotide ribe 31'], 'pdb_id': '7kpd', 'e_value': 6.5e-13, 'percentage_identity': 82.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knm_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knm', 'e_value': 6.6e-13, 'percentage_identity': 82.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kcg_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2kcg', 'e_value': 6.6e-13, 'percentage_identity': 82.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2knn_1', 'molecule_name': ['Cycloviolacin-O2'], 'pdb_id': '2knn', 'e_value': 3.4e-12, 'percentage_identity': 79.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh4_1', 'molecule_name': ['Circulin-A'], 'pdb_id': '1bh4', 'e_value': 4.2e-12, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2kux_1', 'molecule_name': ['Kalata-B5'], 'pdb_id': '2kux', 'e_value': 5.4e-11, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vb8_1', 'molecule_name': ['Root cyclotide 1'], 'pdb_id': '1vb8', 'e_value': 2.3e-08, 'percentage_identity': 73.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nbj_1', 'molecule_name': ['Cycloviolacin-O1'], 'pdb_id': '1nbj', 'e_value': 2.3e-08, 'percentage_identity': 69.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1za8_1', 'molecule_name': ['Leaf cyclotide 1'], 'pdb_id': '1za8', 'e_value': 2.9e-07, 'percentage_identity': 64.3, 'result_sequence': None}], 'TKCFQWQWNMRKVRG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 6.5e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 6.5e-06, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '1l5t_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1l5t', 'e_value': 4.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lct_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lct', 'e_value': 4.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1h44_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1h44', 'e_value': 4.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfi_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfi', 'e_value': 9.4e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bka_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1bka', 'e_value': 9.4e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfg_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lfg', 'e_value': 9.4e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lcf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1lcf', 'e_value': 9.4e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1b0l_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1b0l', 'e_value': 9.4e-05, 'percentage_identity': 93.3, 'result_sequence': None}], 'FKCRRWAWRMKKLGA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 6.5e-07, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 1.5e-05, 'percentage_identity': 93.3, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ce2_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1ce2', 'e_value': 0.00073, 'percentage_identity': 80.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1biy_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1biy', 'e_value': 0.00073, 'percentage_identity': 80.0, 'result_sequence': None}], 'GLLSAVKGVLKGAGKNVAGSLMDKLKCKLFGGC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 9.9e-06, 'percentage_identity': 54.5, 'result_sequence': None}], 'FLSLIPHAINAVSAIAKHFG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq0_1', 'molecule_name': ['Phylloseptin-H1'], 'pdb_id': '2jq0', 'e_value': 5.2e-07, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jq1_1', 'molecule_name': ['Phyllospetin-H3'], 'pdb_id': '2jq1', 'e_value': 7.4e-06, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jpy_1', 'molecule_name': ['Phylloseptin-H2'], 'pdb_id': '2jpy', 'e_value': 1.3e-05, 'percentage_identity': 78.9, 'result_sequence': None}], 'KWKSFIKKLTSKALHSAKKF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.029, 'percentage_identity': 65.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.029, 'percentage_identity': 65.0, 'result_sequence': None}], 'GLICESCRKIIQKLEDMVGPQPNEDTVTQAASRVCDKMKILRGVCKKIMRTFLRRISKDILTGKKPQAICVDIKICKE': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nkl_1', 'molecule_name': ['Antimicrobial peptide NK-lysin'], 'pdb_id': '1nkl', 'e_value': 3e-34, 'percentage_identity': 91.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l9l_1', 'molecule_name': ['Granulysin'], 'pdb_id': '1l9l', 'e_value': 2.3e-07, 'percentage_identity': 35.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ddj_1', 'molecule_name': ['Saposin-A'], 'pdb_id': '4ddj', 'e_value': 0.0042, 'percentage_identity': 23.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sn6_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '1sn6', 'e_value': 0.0042, 'percentage_identity': 21.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gtg_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2gtg', 'e_value': 0.0042, 'percentage_identity': 21.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2dob_1', 'molecule_name': ['Saposin-A'], 'pdb_id': '2dob', 'e_value': 0.0042, 'percentage_identity': 23.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1m12_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '1m12', 'e_value': 0.0042, 'percentage_identity': 21.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '4uex_1', 'molecule_name': ['Saposin-A'], 'pdb_id': '4uex', 'e_value': 0.0043, 'percentage_identity': 23.8, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2z9a_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2z9a', 'e_value': 0.0044, 'percentage_identity': 21.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2qyp_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2qyp', 'e_value': 0.0046, 'percentage_identity': 21.6, 'result_sequence': None}], 'GLPVCGETCVGGTCNTPGCTCSWPVCTRN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttm_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttm', 'e_value': 2.1e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2khb_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2khb', 'e_value': 2.1e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4tto_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4tto', 'e_value': 5.7e-14, 'percentage_identity': 96.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jjz_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1jjz', 'e_value': 8.6e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k48_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '1k48', 'e_value': 8.6e-14, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4ttn_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '4ttn', 'e_value': 1.1e-13, 'percentage_identity': 96.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mn1_1', 'molecule_name': ['Kalata-B1'], 'pdb_id': '2mn1', 'e_value': 2.4e-13, 'percentage_identity': 96.7, 'result_sequence': None}], 'KQATVGDINTERPGILDLKGKAKWDAWNGLKGTSKEDAMKAYINKVEELKKKYGI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nvl_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '1nvl', 'e_value': 1.2e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1nti_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '1nti', 'e_value': 1.2e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2abd_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '2abd', 'e_value': 1.2e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1hb8_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '1hb8', 'e_value': 1.2e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1hb6_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '1hb6', 'e_value': 1.2e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1aca_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '1aca', 'e_value': 1.2e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2fj9_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '2fj9', 'e_value': 1.7e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2cb8_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '2cb8', 'e_value': 1.7e-21, 'percentage_identity': 92.7, 'result_sequence': None}, {'chain_id': ['A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2fdq_1', 'molecule_name': ['Acyl-CoA-binding protein'], 'pdb_id': '2fdq', 'e_value': 2.1e-21, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3epy_1', 'molecule_name': ['Acyl-CoA-binding domain-containing protein 7'], 'pdb_id': '3epy', 'e_value': 7.7e-14, 'percentage_identity': 69.1, 'result_sequence': None}], 'RKCLRWQWAMRKYGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0065, 'percentage_identity': 69.2, 'result_sequence': None}], 'ARLKKCFNKVTGYCRKKCKVGERYEIGCLSGKLCCAN': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6mjv_1', 'molecule_name': ['Human beta-defensin'], 'pdb_id': '6mjv', 'e_value': 7.3e-05, 'percentage_identity': 57.1, 'result_sequence': None}], 'KWKLFKKIGIGAVLKVLTTGLIALIS': [{'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '3qrx_2', 'molecule_name': ['Melittin'], 'pdb_id': '3qrx', 'e_value': 0.0006, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6dst_1', 'molecule_name': ['Melittin'], 'pdb_id': '6dst', 'e_value': 0.0006, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mlt_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mlt', 'e_value': 0.00062, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mw6_1', 'molecule_name': ['Melittin'], 'pdb_id': '2mw6', 'e_value': 0.00062, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bh1_1', 'molecule_name': ['Melittin'], 'pdb_id': '1bh1', 'e_value': 0.00062, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 2, 'entry_entity': '6o4m_2', 'molecule_name': ['Melittin'], 'pdb_id': '6o4m', 'e_value': 0.00062, 'percentage_identity': 94.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.012, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.012, 'percentage_identity': 85.7, 'result_sequence': None}], 'DFASCHTNGGICLPNRCPGHMIQIGICFRPRVKCCRSW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 3.1e-10, 'percentage_identity': 69.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 0.00012, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 0.00012, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 0.00012, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 0.00012, 'percentage_identity': 44.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.0002, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00025, 'percentage_identity': 45.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.00028, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '2nlh_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nlh', 'e_value': 0.0003, 'percentage_identity': 38.9, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2nld_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2nld', 'e_value': 0.0003, 'percentage_identity': 38.9, 'result_sequence': None}], 'DAECEICKFVIQQVEAFIESNHSQAEIQKELNKL': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1sn6_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '1sn6', 'e_value': 0.002, 'percentage_identity': 32.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gtg_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2gtg', 'e_value': 0.002, 'percentage_identity': 32.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1m12_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '1m12', 'e_value': 0.002, 'percentage_identity': 32.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2qyp_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2qyp', 'e_value': 0.0021, 'percentage_identity': 32.4, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2z9a_1', 'molecule_name': ['Saposin-C'], 'pdb_id': '2z9a', 'e_value': 0.0021, 'percentage_identity': 32.4, 'result_sequence': None}], 'AKIPIKAIKTVGKAVGKGLRAINIASTANDVFNFLKPKKRKA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kv4_1', 'molecule_name': ['Moricin-1'], 'pdb_id': '1kv4', 'e_value': 1.6e-17, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1x22_1', 'molecule_name': ['Moricin'], 'pdb_id': '1x22', 'e_value': 7.5e-12, 'percentage_identity': 72.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jr8_1', 'molecule_name': ['Moricin'], 'pdb_id': '2jr8', 'e_value': 6.6e-11, 'percentage_identity': 67.5, 'result_sequence': None}], 'GPLSCRRNGGVCIPIRCPGPMRQIGTCFGRPVKCCRSW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 1.9e-18, 'percentage_identity': 94.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 1.8e-06, 'percentage_identity': 57.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 6.6e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 7.1e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 7.1e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 7.1e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 7.1e-06, 'percentage_identity': 47.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 7.2e-05, 'percentage_identity': 51.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lcs_1', 'molecule_name': ['Gallinacin-7'], 'pdb_id': '5lcs', 'e_value': 0.0038, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.0056, 'percentage_identity': 46.9, 'result_sequence': None}], 'EGVRNFVTCRINRGFCVPIRCPGHRRQIGTCLGPRIKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 7e-08, 'percentage_identity': 58.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 3.5e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 3.5e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 3.5e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 3.5e-05, 'percentage_identity': 43.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 5.5e-05, 'percentage_identity': 46.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00013, 'percentage_identity': 47.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0023, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ut3_1', 'molecule_name': ['Spheniscin-2'], 'pdb_id': '1ut3', 'e_value': 0.015, 'percentage_identity': 41.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1kj6_1', 'molecule_name': ['Beta-defensin 103'], 'pdb_id': '1kj6', 'e_value': 0.06, 'percentage_identity': 40.6, 'result_sequence': None}], 'KNLRRITRKIIHIIKKYG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fry_1', 'molecule_name': ['Cathelin-related peptide SC5'], 'pdb_id': '1fry', 'e_value': 0.024, 'percentage_identity': 61.1, 'result_sequence': None}], 'FFHHIFRGIVHVGRTIHKLVTGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 5.5e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 5.5e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 5.6e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 5.6e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 5.6e-10, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 8.3e-09, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 4.2e-08, 'percentage_identity': 81.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 3.4e-06, 'percentage_identity': 68.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 3.4e-06, 'percentage_identity': 68.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 3.4e-06, 'percentage_identity': 68.2, 'result_sequence': None}], 'FFHHIFRGIVHVGRTIHRLVTGG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2jos_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2jos', 'e_value': 2.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojm_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2ojm', 'e_value': 2.4e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcv_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcv', 'e_value': 2.5e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcu_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '2mcu', 'e_value': 2.5e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pf0_1', 'molecule_name': ['Moronecidin'], 'pdb_id': '6pf0', 'e_value': 2.5e-10, 'percentage_identity': 95.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojn_1', 'molecule_name': ['Piscidin_PG'], 'pdb_id': '2ojn', 'e_value': 3.5e-09, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ojo_1', 'molecule_name': ['Piscidin_AA'], 'pdb_id': '2ojo', 'e_value': 1.8e-08, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcw_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcw', 'e_value': 1.4e-06, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mcx_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '2mcx', 'e_value': 1.4e-06, 'percentage_identity': 72.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6pez_1', 'molecule_name': ['Piscidin-3'], 'pdb_id': '6pez', 'e_value': 1.4e-06, 'percentage_identity': 72.7, 'result_sequence': None}], 'GSKKPVPIIYCNRRKCQRM': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xo9_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo9', 'e_value': 2.8e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo4_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo4', 'e_value': 2.8e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '6gd5_2', 'molecule_name': ['Thanatin'], 'pdb_id': '6gd5', 'e_value': 2.8e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6aab_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6aab', 'e_value': 2.8e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '8tfv_1', 'molecule_name': ['Thanatin'], 'pdb_id': '8tfv', 'e_value': 2.8e-08, 'percentage_identity': 90.5, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5xo3_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xo3', 'e_value': 2e-07, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xoa_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xoa', 'e_value': 2e-07, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6afq_1', 'molecule_name': ['Thanatin'], 'pdb_id': '6afq', 'e_value': 2e-07, 'percentage_identity': 90.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5xok_1', 'molecule_name': ['Thanatin'], 'pdb_id': '5xok', 'e_value': 3.2e-06, 'percentage_identity': 85.0, 'result_sequence': None}], 'KWKLFKKILLKFLHLAKLF': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.00035, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.00035, 'percentage_identity': 75.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0g_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0g', 'e_value': 0.071, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9p_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9p', 'e_value': 0.071, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0h_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1f0h', 'e_value': 0.096, 'percentage_identity': 70.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9o_1', 'molecule_name': ['Magainin-2', 'Cecropin-A'], 'pdb_id': '1d9o', 'e_value': 0.096, 'percentage_identity': 70.0, 'result_sequence': None}], 'CKQSCSFGPFTFVCDGNTK': [{'chain_id': ['C'], 'entity_id': 2, 'entry_entity': '6c0h_2', 'molecule_name': ['Gln-dal-cys-ala-phe-gly-pro-phe-dbb-phe-val-cys-bh2-gly'], 'pdb_id': '6c0h', 'e_value': 1.4e-08, 'percentage_identity': 89.5, 'result_sequence': None}], 'QGVRSYLSCWGNRGICLLNRCPGRMRQIGTCLAPRVKCCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1bnb_1', 'molecule_name': ['Beta-defensin 12'], 'pdb_id': '1bnb', 'e_value': 9.8e-08, 'percentage_identity': 61.8, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fqq_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fqq', 'e_value': 7.3e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['H', 'P', 'I', 'K', 'D', 'M', 'A', 'B', 'C', 'J', 'N', 'O', 'E', 'F', 'G', 'L'], 'entity_id': 1, 'entry_entity': '1fd4_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd4', 'e_value': 7.3e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['D', 'A', 'B', 'C'], 'entity_id': 1, 'entry_entity': '1fd3_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1fd3', 'e_value': 7.3e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6cs9_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '6cs9', 'e_value': 7.3e-05, 'percentage_identity': 41.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4q_1', 'molecule_name': ['Beta-defensin 4A'], 'pdb_id': '1e4q', 'e_value': 0.00025, 'percentage_identity': 44.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lxo_1', 'molecule_name': ['Chimeric Peptide'], 'pdb_id': '2lxo', 'e_value': 0.0013, 'percentage_identity': 55.2, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2plz_1', 'molecule_name': ['Beta-defensin 1'], 'pdb_id': '2plz', 'e_value': 0.0014, 'percentage_identity': 40.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1e4r_1', 'molecule_name': ['Beta-defensin 8'], 'pdb_id': '1e4r', 'e_value': 0.0017, 'percentage_identity': 50.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6m56_1', 'molecule_name': ['Beta-defensin 4'], 'pdb_id': '6m56', 'e_value': 0.0045, 'percentage_identity': 46.9, 'result_sequence': None}], 'AERVGAGAPVYL': [{'chain_id': ['I', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '2xql_1', 'molecule_name': ['Histone H2A-IV'], 'pdb_id': '2xql', 'e_value': 0.00082, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['B'], 'entity_id': 2, 'entry_entity': '4kha_2', 'molecule_name': ['Histone H2A'], 'pdb_id': '4kha', 'e_value': 0.00083, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C'], 'entity_id': 1, 'entry_entity': '6k01_1', 'molecule_name': ['Histone H2A'], 'pdb_id': '6k01', 'e_value': 0.00083, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'A'], 'entity_id': 1, 'entry_entity': '7c7x_1', 'molecule_name': ['Histone H2A.6'], 'pdb_id': '7c7x', 'e_value': 0.00084, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['I', 'K', 'A', 'C', 'E', 'G'], 'entity_id': 1, 'entry_entity': '6m2m_1', 'molecule_name': ['Probable histone H2A.3'], 'pdb_id': '6m2m', 'e_value': 0.00084, 'percentage_identity': 100.0, 'result_sequence': None}], 'GKPRPYLPRPTSHPRPIRV': [{'chain_id': ['D', 'C'], 'entity_id': 2, 'entry_entity': '4ezp_2', 'molecule_name': ['APO-monomer'], 'pdb_id': '4ezp', 'e_value': 0.0031, 'percentage_identity': 76.5, 'result_sequence': None}], 'WGRKKRRQRRRPPQ': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1jfw_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1jfw', 'e_value': 0.0009, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1k5k_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1k5k', 'e_value': 0.00091, 'percentage_identity': 92.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tiv_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1tiv', 'e_value': 0.015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tbc_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1tbc', 'e_value': 0.015, 'percentage_identity': 85.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1tac_1', 'molecule_name': ['Protein Tat'], 'pdb_id': '1tac', 'e_value': 0.015, 'percentage_identity': 85.7, 'result_sequence': None}], 'GLLRRLRDFLKKIGEKFKKIGY': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lyp_1', 'molecule_name': ['Antimicrobial protein CAP7'], 'pdb_id': '1lyp', 'e_value': 0.089, 'percentage_identity': 66.7, 'result_sequence': None}], 'DKLIGSCVWGAVNYTSDCAAEKRRGYKGGYCGSFANVNCWCET': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2u_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2u', 'e_value': 8.7e-20, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ozz_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1ozz', 'e_value': 1.1e-19, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p00_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p00', 'e_value': 1.3e-19, 'percentage_identity': 90.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1p0a_1', 'molecule_name': ['Defensin ARD1'], 'pdb_id': '1p0a', 'e_value': 4.5e-19, 'percentage_identity': 88.6, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1i2v_1', 'molecule_name': ['Defensin heliomicin'], 'pdb_id': '1i2v', 'e_value': 4.3e-18, 'percentage_identity': 86.4, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vnb_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vnb', 'e_value': 0.054, 'percentage_identity': 42.1, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1vna_1', 'molecule_name': ['Toxin CsEv1'], 'pdb_id': '1vna', 'e_value': 0.054, 'percentage_identity': 42.1, 'result_sequence': None}], 'IGKKFKRIVQRIKDFLRNL': [{'chain_id': ['F'], 'entity_id': 1, 'entry_entity': '2fcg_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2fcg', 'e_value': 5.8e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k6o_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '2k6o', 'e_value': 7.3e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nnk_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnk', 'e_value': 7.3e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5nmn_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nmn', 'e_value': 7.3e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnm_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnm', 'e_value': 7.3e-06, 'percentage_identity': 94.7, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5nnt_1', 'molecule_name': ['Antibacterial peptide LL-37'], 'pdb_id': '5nnt', 'e_value': 7.3e-06, 'percentage_identity': 94.7, 'result_sequence': None}], 'GFMDTAKNVAKNVAVTLIDNLKCKITKAC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2k10_1', 'molecule_name': ['Antimicrobial_2 domain-containing protein'], 'pdb_id': '2k10', 'e_value': 0.014, 'percentage_identity': 53.6, 'result_sequence': None}]}

In [55]:
%store res_5500

Stored 'res_5500' (dict)


In [59]:
len(res_5500)

131

## 5500-5638

In [39]:
sec_5638=sec_1[5500:]

In [42]:
res_5638_raw=multiple_sequence_search(sec_5638)

1 sequences of 138
2 sequences of 138
3 sequences of 138
4 sequences of 138
5 sequences of 138
6 sequences of 138
7 sequences of 138
8 sequences of 138
9 sequences of 138
10 sequences of 138
11 sequences of 138
12 sequences of 138
13 sequences of 138
14 sequences of 138
15 sequences of 138
16 sequences of 138
17 sequences of 138
18 sequences of 138
19 sequences of 138
20 sequences of 138
21 sequences of 138
22 sequences of 138
23 sequences of 138
24 sequences of 138
25 sequences of 138
26 sequences of 138
27 sequences of 138
28 sequences of 138
29 sequences of 138
30 sequences of 138
31 sequences of 138
32 sequences of 138
33 sequences of 138
34 sequences of 138
35 sequences of 138
36 sequences of 138
37 sequences of 138
38 sequences of 138
39 sequences of 138
40 sequences of 138
41 sequences of 138
42 sequences of 138
43 sequences of 138
44 sequences of 138
45 sequences of 138
46 sequences of 138
47 sequences of 138
48 sequences of 138
49 sequences of 138
50 sequences of 138
51 sequen

In [43]:
res_5638=results_to_dict(res_5638_raw)

In [44]:
%store res_5638 res_5638_raw

Stored 'res_5638' (dict)
Stored 'res_5638_raw' (dict)


In [45]:
len(res_5638)

65

## Sequence length below 10 residues (runned in colab)

In [4]:
res_max10={'RAWVAWRNR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2meb_1', 'molecule_name': ['Lysozyme C'], 'pdb_id': '2meb', 'e_value': 0.021, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mee_1', 'molecule_name': ['Lysozyme C'], 'pdb_id': '2mee', 'e_value': 0.021, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2meh_1', 'molecule_name': ['Lysozyme C'], 'pdb_id': '2meh', 'e_value': 0.021, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mec_1', 'molecule_name': ['Lysozyme C'], 'pdb_id': '2mec', 'e_value': 0.021, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mei_1', 'molecule_name': ['Lysozyme C'], 'pdb_id': '2mei', 'e_value': 0.021, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '5lvk_1', 'molecule_name': ['Lysozyme C'], 'pdb_id': '5lvk', 'e_value': 0.021, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '5lsh_1', 'molecule_name': ['Lysozyme C'], 'pdb_id': '5lsh', 'e_value': 0.021, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2mef_1', 'molecule_name': ['Lysozyme C'], 'pdb_id': '2mef', 'e_value': 0.021, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '2mea_1', 'molecule_name': ['Lysozyme C'], 'pdb_id': '2mea', 'e_value': 0.021, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2med_1', 'molecule_name': ['Lysozyme C'], 'pdb_id': '2med', 'e_value': 0.021, 'percentage_identity': 100.0, 'result_sequence': None}], 'LHCIALRKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2ln4_1', 'molecule_name': ['Defensin coprisin'], 'pdb_id': '2ln4', 'e_value': 0.011, 'percentage_identity': 100.0, 'result_sequence': None}], 'GRCRDDFRC': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '3psm_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '3psm', 'e_value': 0.00045, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lj7_1', 'molecule_name': ['Defensin Lc-def'], 'pdb_id': '2lj7', 'e_value': 0.00045, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2gl1_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2gl1', 'e_value': 0.00045, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6nom_1', 'molecule_name': ['Defensin-2'], 'pdb_id': '6nom', 'e_value': 0.00045, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '6vpn_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '6vpn', 'e_value': 0.00045, 'percentage_identity': 100.0, 'result_sequence': None}], 'CNGKRVCVC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 0.0006, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.011, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.016, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lld_1', 'molecule_name': ['Defensin Lucifensin'], 'pdb_id': '2lld', 'e_value': 0.043, 'percentage_identity': 66.7, 'result_sequence': None}], 'NGKRVCVCR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2e3f_1', 'molecule_name': ['Defensin'], 'pdb_id': '2e3f', 'e_value': 0.0023, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1l4v_1', 'molecule_name': ['Sapecin'], 'pdb_id': '1l4v', 'e_value': 0.047, 'percentage_identity': 88.9, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1ica_1', 'molecule_name': ['Phormicin'], 'pdb_id': '1ica', 'e_value': 0.067, 'percentage_identity': 88.9, 'result_sequence': None}], 'CGGWKRKRC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 0.03, 'percentage_identity': 77.8, 'result_sequence': None}], 'RWRWRWRW': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4r4h_1', 'molecule_name': ['Surface protein gp120'], 'pdb_id': '4r4h', 'e_value': 0.081, 'percentage_identity': 85.7, 'result_sequence': None}], 'RRWQWRMKK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.0018, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.051, 'percentage_identity': 100.0, 'result_sequence': None}], 'CGGWHRLRC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 0.00014, 'percentage_identity': 100.0, 'result_sequence': None}], 'CKSIPGRCG': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 0.0022, 'percentage_identity': 100.0, 'result_sequence': None}], 'GPCASDHNC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr3_1', 'molecule_name': ['Knot1 domain-containing protein'], 'pdb_id': '2lr3', 'e_value': 0.00028, 'percentage_identity': 100.0, 'result_sequence': None}], 'KWKLFKKI': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1f0d_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1f0d', 'e_value': 0.056, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1d9j_1', 'molecule_name': ['Cecropin-A', 'Magainin-2'], 'pdb_id': '1d9j', 'e_value': 0.056, 'percentage_identity': 100.0, 'result_sequence': None}], 'WWHSWWSTW': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6vpd_1', 'molecule_name': ['Glutathione peroxidase'], 'pdb_id': '6vpd', 'e_value': 0.091, 'percentage_identity': 55.6, 'result_sequence': None}], 'WRWKWRWK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4r4h_1', 'molecule_name': ['Surface protein gp120'], 'pdb_id': '4r4h', 'e_value': 0.094, 'percentage_identity': 85.7, 'result_sequence': None}], 'VGRIHRHLK': [{'chain_id': ['D', 'A', 'J', 'G'], 'entity_id': 1, 'entry_entity': '5fug_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '5fug', 'e_value': 0.068, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '4cay_1', 'molecule_name': ['Histone H2A.Z'], 'pdb_id': '4cay', 'e_value': 0.069, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['C', 'G'], 'entity_id': 3, 'entry_entity': '6m4d_3', 'molecule_name': ['Histone H2A.V'], 'pdb_id': '6m4d', 'e_value': 0.071, 'percentage_identity': 100.0, 'result_sequence': None}], 'AHCLAIGRK': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2lr5_1', 'molecule_name': ['micasin'], 'pdb_id': '2lr5', 'e_value': 0.037, 'percentage_identity': 88.9, 'result_sequence': None}], 'CGGWKRLRC': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1fjn_1', 'molecule_name': ['Defensin MGD-1'], 'pdb_id': '1fjn', 'e_value': 0.0022, 'percentage_identity': 88.9, 'result_sequence': None}], 'WWSYWWTQW': [{'chain_id': ['A', 'B'], 'entity_id': 1, 'entry_entity': '6vpd_1', 'molecule_name': ['Glutathione peroxidase'], 'pdb_id': '6vpd', 'e_value': 0.037, 'percentage_identity': 66.7, 'result_sequence': None}], 'TFFRLFNR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '2i2j_1', 'molecule_name': ['Competence stimulating protein'], 'pdb_id': '2i2j', 'e_value': 0.055, 'percentage_identity': 100.0, 'result_sequence': None}], 'RRSVQWCA': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6w_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6w', 'e_value': 0.028, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1z6v_1', 'molecule_name': ['Lactoferricin-H'], 'pdb_id': '1z6v', 'e_value': 0.028, 'percentage_identity': 100.0, 'result_sequence': None}], 'FKCRRWQWR': [{'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1lfc_1', 'molecule_name': ['Lactoferricin-B'], 'pdb_id': '1lfc', 'e_value': 0.00012, 'percentage_identity': 100.0, 'result_sequence': None}, {'chain_id': ['A'], 'entity_id': 1, 'entry_entity': '1blf_1', 'molecule_name': ['Lactotransferrin'], 'pdb_id': '1blf', 'e_value': 0.0032, 'percentage_identity': 100.0, 'result_sequence': None}]}

In [32]:
len(res_max10)

21

In [8]:
%store res_max10

Stored 'res_max10' (dict)


# Final Results

In [1]:
%store -r res_5h res_1k res_15h res_2k res_25h res_3k res_35h res_3750 res_4k res_4250 res_45h res_5k res_5250 res_5500 res_5638

In [7]:
print(len(res_5h)+len(res_1k)+ len(res_15h) +len(res_2k) +len(res_25h) +len(res_3k)+ len(res_35h)+len(res_3750)+len(res_4k)+len(res_4250)+len(res_45h)+len(res_5k)+len(res_5250)+len(res_5500)+len(res_5638)+len(res_max10))

2135


In [5]:
total_res={**res_5h,
           **res_1k,
           **res_15h,
           **res_2k,
           **res_25h,
           **res_3k,
           **res_35h,
           **res_3750,
           **res_4k,
           **res_4250,
           **res_45h,
           **res_5k,
           **res_5250,
          **res_5500,
          **res_5638,
          **res_max10}

In [6]:
len(total_res)

2135

In [8]:
%store -r total_res

In [10]:
len(sec_0)

6169